In [1]:
from keras.models import Model, Sequential
from keras.layers import Dense
import numpy as np
import random
from tqdm import tqdm
from typing import List, Dict

from src.model.players.GeneticBot import GeneticBot
from src.GameController import GameController
from src.model.Game import Game
from src.view.NoGraphics import NoGraphics

import os
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/lib/cuda'

2024-06-27 17:31:04.245342: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-27 17:31:04.269550: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-27 17:31:04.680148: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
blank_model = GeneticBot("").model

def model_to_genotype(model):
    weights = model.get_weights()
    # flatten weights
    res = []
    for layer in weights:
        res.extend(layer.flatten())
    return res

def genotype_to_model(genotype:list):
    model = blank_model
    weights = []
    i = 0
    for layer in model.get_weights():
        layer_size = layer.size
        new_layer = np.array([genotype[i:i+layer_size]]).reshape(layer.shape)
        weights.append(new_layer)
        i += layer_size
    model.set_weights(weights)
    return model

GENOTYPE_SIZE = len(model_to_genotype(blank_model))
print(GENOTYPE_SIZE)

def random_genotype():
    return [random.uniform(-1,1) for _ in range(GENOTYPE_SIZE)]

10049


/home/useria/Documents/sk/skull-king-bots/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-06-27 17:31:05.026264: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-27 17:31:05.046094: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation

In [3]:
class Individual:
    def __init__(self, genotype):
        self.genotype = genotype
        self.player = GeneticBot(str(genotype), delay=0.0001)
        self.player.set_model(genotype_to_model(genotype))
        self.fitness = 0

def history_to_fitness(history:List[Dict[str, Dict[str, int]]], population:List[Individual]) -> None:
    for individual in population:
        individual.fitness = history[-1][individual.player.name]["score"]

def crossover(parent1:Individual, parent2:Individual) -> Individual:
    genotype1 = parent1.genotype
    genotype2 = parent2.genotype
    split_point = random.randint(0, GENOTYPE_SIZE)
    child_genotype = genotype1[:split_point] + genotype2[split_point:]
    return Individual(child_genotype)

def mutate(individual:Individual) -> None:
    mutation_rate = 0.1
    for i in range(GENOTYPE_SIZE):
        if random.random() < mutation_rate:
            individual.genotype[i] = random.uniform(-1,1)

In [4]:

def genetic_algorithm(output_file:str):
    population = [Individual(random_genotype()) for _ in range(50)]
    best_individual = None
    for generation in range(10):
        with open(output_file, "a") as f:
            f.write(f"Generation {generation}\n")
        # make batches of 5 players
        batches = [population[i:i+5] for i in range(0, len(population), 5)]
        for batch in tqdm(batches):
            game = Game([individual.player for individual in batch], False)
            game_controller = GameController(game, NoGraphics())
            game_controller.run()
            history = game_controller.get_history()
            history_to_fitness(history, batch)
        
        population.sort(key=lambda x: x.fitness, reverse=True)
        with open(output_file, "a") as f:
            f.write(f"  Best genotype: {population[0].genotype}\n")
            f.write(f"  Best fitness: {population[0].fitness}\n")
        best_individual = population[0]
        new_population = []
        for i in range(0, len(population)//2, 2):
            parent1 = population[i]
            parent2 = population[i+1]
            child = crossover(parent1, parent2)
            mutate(child)
            new_population.append(child)
        population = new_population
    return best_individual


In [5]:
best_individual = genetic_algorithm(output_file="out.txt")

print(f"Best individual: {best_individual.fitness}")

  0%|          | 0/10 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


I0000 00:00:1719502266.557305  439052 service.cc:145] XLA service 0x7dad34004ac0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719502266.557323  439052 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-06-27 17:31:06.560205: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-27 17:31:06.578121: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902
I0000 00:00:1719502266.627488  439052 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/

 10%|█         | 1/10 [00:19<02:54, 19.43s/it]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/st

100%|██████████| 1/1 [00:04<00:00,  4.86s/it]

cccccc

100%|██████████| 1/1 [00:04<00:00,  4.86s/it]
0it [00:00, ?it/s]


IndexError: list index out of range

In [ ]:
# save model
model = best_individual.player.model
model.save("best_model.h5")

In [7]:
genotype = [0.35989233099637996, 0.18081765256249072, 0.8551571004824032, 0.8358475372737033, 0.7374674025183876, 0.4950025362426207, -0.06203500681500884, -0.809819640515095, 0.013768847598115919, 0.37152771439118504, -0.16762436975301886, -0.31894054878302236, 0.5901712257395819, -0.1166755254455274, -0.41405136871452863, -0.20664532661145385, -0.19029426213179135, 0.953716533381409, 0.3603486664162838, -0.6626274092779076, 0.7767705566473566, -0.31282269576109467, 0.5495084064776381, -0.2510611101318423, -0.8619356124459019, -0.41704901817398254, -0.9962423290459659, 0.15046652020689355, 0.6512710501546233, -0.6760542777058973, -0.5704836316814503, -0.19150509569157026, 0.16471543732371874, -0.4700464552902488, 0.028345787480581475, 0.8468885315603105, -0.2679416994465569, 0.3229729627515423, 0.5260691529554473, 0.6865680487401282, 0.7067557994254929, 0.1478635441400915, 0.6267923015413566, 0.48911496567997337, 0.4903965723242174, 0.9743249860530707, 0.0387856903268855, 0.798842431384027, 0.8319902138095321, -0.12899296813801997, -0.30063230590115175, -0.16780594619969902, 0.7366974388076024, 0.6085663617154677, 0.939675530918179, -0.8119346039371873, 0.8611097665341425, 0.517551848710156, -0.0002837050973376609, -0.2726432868980264, 0.6869488677147697, 0.7634560171419766, -0.9071737864623046, -0.9277337680505364, -0.7684087431583726, 0.9378735900902235, 0.7530266643825103, -0.4917161595116708, 0.5706526836211692, -0.21986431439053433, 0.6163479601853712, -0.5309200086200685, -0.36900989026917697, 0.43582311602446167, 0.7091686536184623, 0.4399892567000634, 0.8970209842671448, 0.7410353434859966, 0.9267193485075611, 0.06486355356533569, 0.443700981762152, 0.19648489861666518, 0.9899822156332556, -0.5005448511118986, -0.5430566522979625, 0.6018008921194549, 0.39397853414477835, -0.5824113381457627, 0.18450294454391214, 0.7293363090751144, -0.030070858717923032, -0.3096434132139996, 0.09770332683933325, 0.7662885417060752, 0.8164790534382742, -0.6769591785524403, 0.6607176207799097, -0.9961688454901421, 0.8588261781621847, -0.032387474869295385, 0.12186439202955857, -0.5365178145404832, -0.37780812735378544, -0.8953743351026349, -0.3519320067588474, 0.3052008234211627, 0.2913423873468939, -0.636943606643015, -0.7132980624383891, -0.9913045663635347, 0.3166536452136961, -0.9047174726570764, 0.8480971813354481, 0.2525240978460197, 0.8777130031040432, 0.34301135405275285, 0.9023318777956126, 0.5458934819608614, 0.4385636782905853, 0.19707556411388438, 0.844863369001231, -0.12256318878290795, 0.8519004243033106, -0.1496575735907324, -0.9352773177824052, 0.13239851787465096, -0.8458339027873487, -0.5474300300407606, 0.6731207177354419, 0.6043951875707325, 0.11698350647024425, -0.5288328705204313, -0.47698090472772137, -0.5050074169264658, 0.36923486950446094, 0.7539596340103838, -0.8767548413956401, 0.3513808894744228, -0.5435239376030221, 0.2926323873876078, 0.37860897846618013, 0.32411221889329056, -0.6313001048635125, -0.27702174669514035, 0.6453502719796047, 0.015607605602013797, -0.9386933844040077, -0.03841264785793652, 0.8817993981420282, -0.7307478934576233, 0.11663952689783752, -0.6181812327103768, -0.7607117102389831, 0.09078267204882051, -0.9744332806487441, 0.0447393116718493, 0.5475768906316452, -0.7435110605804784, -0.9816066537110057, -0.31086577290457984, 0.3474468443370553, 0.7218563593687517, 0.18996118046870092, 0.7157500124040248, -0.24640877610630452, 0.4296143397042236, 0.27564788409683016, 0.8443478387320422, -0.7053396437843771, -0.5863213192259364, -0.2563921274382852, -0.5795455525922042, 0.2840458032123119, -0.3930281348295852, -0.4714108122629368, 0.48586555973907, -0.8391689348161346, 0.6610987825234629, -0.9878193404515114, 0.23936175307885144, -0.059812727030866375, 0.7027570928227858, 0.5390500527021151, -0.6442575479499344, -0.07969737549438105, -0.046282083685259545, -0.015314449636396832, -0.010396999499476811, -0.8788434392656836, -0.987295400401069, 0.22072781332086966, 0.1694191541582546, 0.4959628837070513, -0.9480405987976119, 0.4002627646416692, 0.5526940057579859, 0.4179028820758661, -0.7030898748495964, -0.9167750272792512, -0.9264498126314262, 0.7341925302138279, -0.38634899069435336, -0.15762812642409552, -0.47561617569005876, 0.23171318993403367, -0.7185933087974101, 0.5041856942062826, 0.1597149624545271, -0.0073638938342655, -0.7103663801051694, -0.5047540132803321, 0.9636945174658287, 0.5639435622627658, -0.2320088312465196, -0.8751966895328145, -0.4432880924977549, -0.4049244953063291, 0.8703470114203378, 0.1601280346624243, -0.10296533773444394, 0.9156745942728142, -0.9787644695416282, 0.7776945089433951, -0.06371153844417776, 0.3396731231785002, 0.9217544980724623, -0.712507850165131, 0.0827889887191946, -0.9212110016658417, 0.9553464891301848, 0.38912108582250027, -0.5930871074857802, -0.864590129367597, -0.29109618435587215, 0.23937552405113616, -0.4277223690040799, -0.27541626826995236, -0.7548510973443745, 0.3498348091571317, -0.13552287134500718, 0.7477783596969187, 0.4710811667132595, 0.7762463678404106, 0.7846281678063898, -0.6448875184462197, -0.5790649959224823, 0.8475664375585423, -0.8542514289406338, 0.6491655946943853, -0.04741734816085019, 0.052335449172395965, -0.37525832004205184, 0.8849394407818771, -0.9013053139126561, -0.3333975117420791, -0.4326851595106589, -0.8620139995849054, 0.39186567148468177, -0.890706379066385, -0.7595948175286797, 0.5666557905363534, 0.5107785586175762, -0.25315086129301534, -0.4455798536818478, 0.4074706527423029, -0.06932498649588026, -0.3447012065774713, 0.7202280770959608, -0.3371582293454325, 0.6291512925524416, -0.33161854141939173, 0.3787548476339835, -0.9186659394281134, 0.0018782678029720667, 0.12924442749661647, 0.3510773938223617, -0.7148602912082771, 0.05216054703506545, 0.5056599823710308, 0.7010211198258918, 0.8506136747153048, 0.4431373571680177, -0.7320459554574898, 0.5727743237680984, -0.26120279663372736, 0.8401067180544486, 0.27424455526710134, -0.6031046405289, 0.3023784881156668, -0.08389996598725125, 0.4510896164298628, 0.08100691076330002, -0.026225754824831915, -0.08919474329145238, -0.21459975642566476, 0.7916282438014921, -0.3160268318933279, -0.9285312660764509, 0.8050243508962651, -0.08818550395261804, 0.809729456310903, -0.7224017280460928, 0.05140703635969546, 0.9852799097279816, 0.9502077391488151, 0.6854433360225856, 0.16615397595569426, -0.8029021824871734, -0.2790085627311254, -0.7523824960532648, 0.025369860744804962, -0.921295921390924, 0.8382931795501383, 0.967868693278013, -0.47449445172628835, 0.7346908247937252, -0.04764943380414288, -0.6486287528176169, -0.9838381672287062, 0.3589059705061355, -0.9001026669965917, -0.550775458436219, -0.01691961352555671, -0.6572140473888792, -0.8945435469129661, 0.6078685576506051, 0.27455623293625897, -0.3354282653786056, 0.10719482262051083, -0.7960619322976286, 0.6781567866868872, -0.4331028296403665, 0.4952352954188135, -0.19746524823056522, -0.9291691490829288, -0.5477069779857859, -0.22445900165188215, -0.22581297163823, -0.7787515395573821, 0.8921173077311133, -0.03233438265087707, 0.6105939241291398, 0.43641638495571455, 0.9586699843400719, -0.01784026200418043, 0.41582921351010294, 0.8486700952550592, 0.41313288514754976, 0.6176770435483094, -0.15448031122758366, -0.37164030532658376, -0.4159430264470394, -0.33646610311421843, 0.14854153012757298, -0.9531262254458688, 0.338745048775122, 0.4254037894632854, -0.40568435131720904, -0.004170803610670681, -0.32927915995855406, 0.10444516210076316, -0.8844683845420087, 0.883769236251049, 0.7602444399645418, -0.9597861772128917, 0.601176124706537, 0.7066168766130929, -0.9239871204004493, 0.06564438787110105, 0.34735419137415446, 0.2662511019809637, -0.6304171937872973, -0.20332988673470065, -0.778188383665124, 0.8791186248339431, -0.9511375096221226, 0.30411463625756996, -0.45329369421647137, -0.8439192770442465, -0.44039718892261837, 0.8779773435478493, 0.623650534405946, -0.8185152075047621, 0.7524395016873124, -0.5356722741319115, -0.1807733528298332, -0.18691606839826802, -0.6949547797468547, -0.3069050459620857, -0.9818891321954004, -0.8140317436901632, 0.2933403475817975, -0.24322413017194888, -0.029287931213195284, 0.9744688036108984, 0.5693741973903312, -0.3996612565072517, 0.6158776213104746, 0.4341401647777021, 0.945623854666354, -0.9539706110191308, 0.5986020362233346, -0.8393045392926419, -0.8534645911725705, 0.07596782976179517, -0.03513820776253973, -0.8623265385032199, 0.8969890119667276, 0.7366309605424304, 0.9176708843852077, -0.0067815550241177025, -0.2221298316354836, 0.3826225428431236, 0.4747685256482197, -0.15817778958867557, -0.8954336831572403, -0.8109367446396565, -0.027684069430949476, -0.6185010086525955, -0.9468253506128161, -0.0006658196373594549, -0.4939462218314554, 0.05134516126314059, -0.6319650075036403, -0.24809232763326716, -0.9601097459895593, -0.9702594693837281, 0.00030899501389192174, 0.026728813551226693, 0.7297273486137694, -0.9707754444246401, 0.4792291027676474, 0.8302778754125204, 0.4131631168499159, -0.3546295094342191, -0.3299307241624607, 0.523590057246526, 0.4532577051897715, -0.1168338936353599, 0.31085083129741786, -0.3296088742587464, 0.4374996264117139, 0.035553854801414264, -0.19676536111915532, 0.16250937738224258, -0.24561332342092368, 0.20414339250259772, 0.911376097257268, 0.15404058663465836, 0.5154110376742653, -0.18130304774774952, -0.7014793044814247, -0.59884803341293, 0.1768306488464042, -0.5392616376570123, 0.8077422251551067, -0.9258008346332369, -0.6211114255381873, 0.6090943055903835, -0.37684205597107856, -0.9669163909607519, -0.080132980538131, 0.8196304585082483, -0.5684632395231279, 0.9551061014269571, 0.8529660653531237, 0.6585478562746085, -0.6582491034001721, 0.41345448834607734, -0.07270256311104428, 0.7084856222974296, 0.05226339346436659, 0.343889840911886, -0.08829355006051243, -0.2547629132437632, -0.647246221178434, -0.8041582224009061, -0.8534188984669249, -0.1821109644071428, -0.7650612459305683, 0.9534221882289093, 0.0388585550128715, 0.0124051889662542, -0.19201296303459925, 0.6678281848889924, -0.9126826060879336, -0.014529934422591317, 0.09342856809339284, 0.8126019715166373, 0.5046974883716633, -0.957425047819009, -0.6447959087661956, 0.6185714958188331, -0.6146501807665072, -0.6717630913715631, 0.05442969347852156, -0.556294502916673, 0.9901261828367951, 0.8792539316443471, -0.033594882689974126, 0.3426114396256308, 0.5555119644659507, -0.17551829012363274, 0.5121922565353936, 0.18131406535425088, -0.3992867479636504, 0.8254766684992285, -0.7986646565144326, 0.1748028098286769, 0.26127607885174986, -0.599552446809569, -0.3913083041866179, 0.0934492588358824, -0.43535596892695283, 0.26951761868575486, -0.8622914782638, 0.7875227726682192, 0.6682506438448983, 0.004762905654827954, 0.5749882370192028, 0.8630549852981331, 0.8835850330072899, 0.0008489964112483595, 0.8364521476724025, 0.4335461337737705, 0.6275456453027803, 0.8564358000975993, 0.5254038582860083, 0.9114985259196642, 0.7459757254586501, -0.6509380138545571, 0.582084195652375, -0.12968473011377202, 0.4775852182830198, 0.8025974579452102, -0.8638211050518501, -0.009289918662011898, 0.30245321444987483, 0.10770143076209315, 0.6881150307520802, 0.9739909250743493, 0.8744888199411285, -0.2796406071341251, 0.2399260422452738, 0.8222213921424768, -0.26234907436621, -0.8190971483912353, 0.9497043051477752, 0.5867995576180072, 0.8756407749285104, -0.7636845300761104, 0.04488067912191185, 0.06542418749581036, -0.43256751197044885, 0.634603270135143, 0.879445222636881, -0.39968180834470624, -0.5243196666399523, -0.5656339403721757, 0.7153674366565976, -0.3921627821957705, -0.5234501395576099, 0.1018251976483584, 0.9649140082631309, 0.9177808186316045, 0.8808297632830746, 0.45185732687042135, 0.2801718818374457, -0.9386565365400683, -0.29660434066126484, -0.10885374949152649, -0.23958819403779041, -0.8296485902896182, 0.9833282696493875, 0.09510974841404551, -0.23552294168244958, 0.281711438974396, 0.5618035441355831, -0.22946017425271115, 0.36270926437710016, 0.5887254454593998, -0.661887385879778, 0.25268212521194355, -0.8764325710293073, -0.6449607839019207, 0.07405121706956397, 0.009695991318353636, 0.4928498786786426, 0.09791828420941195, 0.9599915020833065, -0.3941516736954147, 0.3311647449180002, -0.8456325106780185, 0.9831933083882909, -0.40668369965233575, -0.4209013768062042, 0.059382948719785444, 0.07315892418258896, 0.8863964569095839, -0.20579977581050635, 0.27007086472925, 0.5490438507214566, -0.3624285185928129, -0.726954390885582, 0.9516044819925367, 0.5624198942816607, -0.3042117740147259, -0.6703984292817509, -0.4068722270915026, 0.7169753169761988, -0.48462207953369485, 0.621557859350337, -0.1250188124640199, 0.7888247965942992, -0.40831102684690146, -0.3960624331944629, 0.39709821021847436, 0.8447466789285314, 0.2838159317526081, -0.8217199776458806, -0.2696886739771418, -0.19853942858253437, 0.0036872274987711506, -0.6362343328993243, -0.3224570814833765, -0.45869717148591405, 0.5363525503450359, 0.4172258420627448, -0.14033144432996436, -0.09211753165978576, 0.7708136873474154, -0.9215834302487944, -0.8069434695894251, -0.1313527795522098, 0.060262338501055224, 0.3334387039241504, 0.9034513555576291, -0.669819618166251, 0.5681790210976614, -0.5743438174271851, 0.6625433381353214, 0.9053881979361724, -0.7173627028264555, -0.22503043787703958, 0.6948532197679951, -0.9954937435680002, -0.8201791619788226, -0.3843107471715752, -0.6950343575923297, -0.8731976297583226, 0.7737847913887532, -0.9611151973970955, 0.19323396646991364, -0.14503246340302356, 0.3478415596357891, 0.19299468346246718, -0.7207357705187187, 0.1832191032613819, -0.3552786375202339, -0.9912293637484451, -0.35629760218284967, -0.7702766480248053, -0.781562762910806, -0.4230151965216473, -0.5279102426745372, 0.9782464090944929, 0.3077328416723435, -0.7390035525350862, -0.9673215195615903, -0.5520601657653634, 0.6259859935245053, 0.5154545839235509, -0.24041231119528828, 0.5493385355654021, 0.621121283507948, 0.864319148324193, -0.6415590060597622, -0.5531300523822902, -0.28984603760152994, 0.97669281786725, 0.08582167983447464, -0.34512880140287483, -0.22648624601689504, -0.24566069583471872, -0.2437126211427194, 0.44484440544551074, 0.8606602492955722, 0.8905257438702012, 0.9357780816569616, 0.9644716494124053, -0.09679207660016953, 0.8804918563995925, -0.6186938682935796, -0.782473517345847, -0.24546205810626875, 0.04307318770145563, 0.964437754066084, -0.9970998056904188, 0.2530057164472521, 0.30870514359243173, -0.15990107116525665, 0.56505310964391, -0.8226981028460436, 0.9322112191729246, -0.2773758012318177, 0.004480276676205985, -0.9260424570349728, -0.6910897646021508, -0.046797260486871384, 0.8592470061680071, 0.946350500404834, -0.7995029048197124, -0.8166085114980985, 0.4162948415428902, -0.5230030641734875, -0.34591225234441136, -0.44607014110537313, 0.02370252336379708, -0.46924641649131926, 0.8575065703173677, 0.3903488438119558, 0.7877189882586642, -0.7930495038654435, -0.7266122846719913, -0.9313192736199982, 0.5355669905213916, -0.26788459345830073, 0.5160676278531999, 0.28821970082018433, -0.39445381289925385, 0.8388876582163158, 0.3251707651937712, -0.09605818754043449, 0.5481029159119204, -0.3520252957608867, -0.07580277098404165, -0.6188379570409541, 0.11660897088573341, -0.7099547569213649, -0.6767362148580756, -0.44768616879991985, -0.5846224451326729, -0.2999003097111772, 0.5999841195428299, 0.7120212240484578, -0.026697034092605154, -0.33033078977893493, -0.012076185619578794, -0.5047940106406559, -0.23369539100835235, 0.1812963782064736, 0.7879956704921123, -0.8287699066868508, -0.4616613847876343, -0.3302117911909368, -0.8589833998253964, 0.4955982581936864, -0.6316680907670493, -0.3368627752458555, 0.026216380939699357, -0.34272187463033355, -0.8346991089588092, -0.38893543230501404, -0.9968701047218933, 0.894910768197555, 0.9030639550268702, 0.12355194008298254, 0.27889366308500274, 0.12049496686948391, -0.9870339217162305, 0.534803653057824, 0.6304370955716947, -0.23685822083243413, -0.7118210564922569, 0.6089592515123368, -0.9044740661054824, -0.44243169574079344, -0.46472500192803956, 0.060199376579337605, -0.16364132172197277, -0.9654983807566557, 0.08217794662435352, -0.276916828865049, 0.9293979638089549, 0.9525009996776794, 0.10092436514950043, 0.7263726384943172, 0.2503620961776871, -0.12312214062649396, 0.13277259914989226, -0.6051313892221686, 0.9880396440333543, 0.09916870726533955, 0.09251461774689362, 0.2658203482982502, -0.21747989996829342, 0.9627302381254275, 0.03502874375238574, -0.5286554215450325, 0.5079182544800718, 0.6806972933156676, 0.013164857540611807, -0.6347021493799685, 0.9117534166602712, -0.27370583401708704, -0.12131915210372002, -0.47225636817847483, 0.3702002516704084, 0.47816322951955215, 0.10207960334563237, 0.7120780963049971, 0.005009105825287685, 0.45493451452838585, -0.5126836235128076, -0.19796231153638066, -0.008239473061858904, 0.9224824894800572, 0.08591887558088462, -0.1335064497739502, -0.23841268114910275, 0.6119603036405474, 0.3259943016639315, -0.46946822180226655, -0.6714359086499251, -0.16312737515425058, -0.0541541725139989, -0.07534703155739564, -0.009485472809715567, 0.7387350126985359, 0.8291959099960899, -0.573477526155983, 0.6401895412250618, 0.9704995495987985, -0.3058978642133503, -0.341279651181692, 0.7776368981483044, -0.9907648336218802, -0.3295514573696414, -0.9225813135812542, 0.796975393996715, -0.7273042539671368, -0.9565683647781194, 0.9812611450664324, -0.4783233568531753, -0.6255363928068893, -0.2834790376249201, -0.7833716246265434, 0.20754645605996358, 0.3974159526554988, -0.5306083462104809, 0.8546233966877452, 0.3615789192907417, -0.20750319675315154, -0.9658995559034123, 0.7100866625856659, 0.29543121477845125, -0.7279905512688059, 0.17291087840053732, 0.8251575131249298, 0.25041023626069436, 0.4502069868054617, -0.767844953291914, 0.5333733502312104, -0.9346321545924381, 0.5780690849509937, -0.8986227116107375, -0.6921508813243198, 0.6213680347714703, 0.7028183531843557, -0.9493797399213901, 0.14739089088688928, 0.1281908595501322, -0.02458351202037412, -0.6351938768146184, 0.24040106665667293, -0.5566187147425581, -0.5212610423979203, 0.4379731741252888, -0.322654541683588, -0.7216191427223475, 0.24970248519574834, 0.13441662558490886, 0.2422017969928285, -0.8741760375104359, -0.7821622530173138, 0.41718575019663917, -0.9434685264623162, 0.2199668374303878, -0.5128013038742281, -0.019786319721528445, -0.4284249571456513, 0.15217515926745828, -0.22029917140783262, 0.5216434745059426, -0.8027203761077681, -0.03238622751120013, 0.08372729561938241, -0.7167916878011595, -0.1788379043883761, -0.6962359690180442, -0.00909925746400253, 0.7972846798183901, 0.24360779831116908, -0.2768170994526773, 0.6414137265398352, -0.7253798514143588, -0.10930667770752756, -0.6954155675854479, -0.7304245144403756, -0.2970525327025719, -0.005480997207145455, -0.6326129014501018, 0.92858443023441, -0.5895340025843563, -0.16720769602282282, -0.4455836716474817, 0.5992376791743581, 0.4368243131316143, 0.2796614783936293, 0.2580825349502949, -0.7765326592242776, -0.05290300460573594, -0.865678343052303, -0.701542118292463, -0.24775619023472362, 0.6605032237704034, -0.2870105322189682, -0.5138231229244625, 0.8988214358764262, 0.8518930603819717, 0.30925068841963377, -0.0461969340983126, -0.5312192685414534, -0.04139676080848331, 0.7664305280639128, -0.5305221727586105, 0.6057771936004552, -0.5575373209719432, 0.4493215923205369, 0.052958222423417745, -0.687613334927798, 0.31860861098045024, 0.9053407764555048, 0.5711689333450962, 0.8463420894050442, -0.45631898977822805, 0.5079374837977106, 0.38894581778580495, -0.8496481200472086, -0.20235474859052505, 0.10546785550838367, -0.85672322448173, 0.8810597715670241, 0.8443212789721248, -0.813817532619191, 0.5249719059515539, 0.6236020575283325, -0.7925202043302702, -0.29942429777305435, 0.5800730315286462, -0.41661931559115395, 0.925309376702572, -0.110735291111701, -0.7583399183991137, 0.5463773731883717, 0.1430677335549695, -0.8801475622239174, -0.8386412315548508, -0.33979005368168647, 0.3684658623860695, -0.03743554759556278, -0.29561756633171, -0.11644516225812862, 0.7060615882841039, -0.3935925543765091, 0.8061520161610489, 0.562118129850699, 0.6347454277732261, 0.39480583699457195, 0.9618063892320532, 0.987994286917816, -0.09340311907461007, 0.5712028724587244, -0.5034889237838636, 0.9340846817897592, -0.8952114196186836, -0.8763247645134997, 0.5983347325990691, 0.03757958416581353, -0.4713035874528466, 0.7979445872340167, 0.355092080234819, 0.6655905331377148, -0.19970487959337846, -0.42984200648753323, 0.6829287377294171, 0.3445496565312167, -0.0031928756890862164, 0.06204349517454211, -0.09975636259125542, -0.9250849879031557, 0.03825003381330894, 0.26675663076484546, 0.20600847540836265, 0.8048674685144916, -0.2830024105086948, -0.41321768177935314, 0.7090497895985735, -0.292056203114184, 0.8753118719253132, 0.24803784235511084, 0.23582579240700086, -0.7267388671645705, 0.21819255455443698, 0.7789261151427711, 0.4045790921786818, 0.41825942213460343, 0.5253112770175137, 0.9517005412011756, -0.5014483203874331, 0.05763484140622488, -0.5484825547627199, 0.887688440158384, 0.9978998247995576, 0.5484541426064216, -0.12490098567305008, -0.510111172082637, -0.8040327701573535, -0.05240022807874456, -0.9033240888463017, 0.24966245310749802, 0.16701959967095403, 0.8056219408245964, 0.9175827254507991, -0.019727029565643095, 0.1026479222637815, -0.5349243430536099, -0.3453095013736289, -0.6880777918255623, -0.059616836374396964, -0.45370959888179163, 0.08638386793713648, -0.41845516809151406, 0.28163099516401746, 0.8328142503511633, 0.9914444798386826, -0.6794320993425087, -0.35468627757169213, -0.5311171429465971, -0.7105558516980475, -0.36080420619857745, -0.970352159949748, -0.9143260508857973, 0.34894868441391913, -0.81020629640008, 0.1476045041097207, -0.7578268255183642, -0.8539455768084152, -0.6993518100413612, 0.3781459041992472, 0.13346750814820063, -0.77555807860368, 0.8155542379417655, 0.5492981493383391, -0.05804614507179995, 0.7974440960972531, -0.3136898547516267, 0.2601340137660386, 0.779920415267092, 0.21266753031739838, 0.9836280453273374, -0.45349594894242773, -0.2135178107583573, 0.04640980557412, 0.07229684519957691, -0.9364260550637273, 0.4433419258227742, -0.8935955226751251, -0.40748226584697567, 0.9109461219228892, -0.7512441482201013, 0.12031335167261359, 0.5036860039109465, 0.7089554432944594, -0.8757838270825857, -0.8671614723040952, 0.23983008979249676, -0.5314420144584824, 0.28556877324625063, -0.09124390288421269, -0.8204914749707899, 0.6222586009243696, -0.2265538181990927, 0.14377658571138285, 0.1524762321634563, 0.6964887911575761, -0.650603717103067, 0.9395842048408665, 0.9820744571796702, 0.7125093583478008, -0.13817067140781147, -0.4066045104228655, 0.4942216314219168, -0.12869123930997195, -0.7391724889527573, 0.4895364928790742, 0.7777196833057114, -0.682440862716988, -0.9244455285198114, -0.5514811475226389, -0.22395769471534632, -0.6991851200822712, 0.761761429641338, -0.18157810863616297, -0.6024896608712933, -0.07927188121231898, 0.6678396596556309, -0.8609796641988543, 0.22499709037465832, -0.19899754656134294, -0.9305115482993691, 0.7762919658925318, 0.4421967136522058, 0.07285179526943875, 0.7712553669017572, -0.09911033157186533, 0.8449109496232647, -0.7655887046442669, -0.27774633848739505, 0.466842395603138, 0.38523777790575586, 0.9789967092380158, -0.9413888605769265, -0.5875252973211675, -0.6959974776598983, 0.5728006274155732, 0.9233519185984826, 0.7339025910111976, -0.7812066044361443, -0.23784110721752838, -0.7645871093950043, -0.20884789299695639, 0.31653207299741415, 0.45699139020059465, 0.6576122470103309, -0.8307442576636579, -0.5463190318428928, -0.16182354229820062, -0.9805948347659197, 0.4398797499104894, -0.5679669501320388, 0.3285892154706458, 0.5485239743481773, 0.6996848358516399, -0.9745455596406774, 0.4849488312397998, -0.07006336238424415, 0.858778135002698, -0.46937164281086075, -0.917939400372888, 0.8759175689041463, -0.7185656424621576, 0.33523715280379873, 0.8784628252203723, -0.6095212285956932, -0.012135573697436453, 0.48036589391938556, 0.495340601434739, 0.9784731134600175, 0.3347400743035793, 0.6269087051656139, -0.2262865719407663, -0.64703594855232, -0.0641811266911041, -0.21421189909628602, 0.6036954667468166, -0.6906370451936417, -0.24697663028107253, 0.42029331503237044, 0.35729051454831073, 0.6994565620256432, -0.5478448734522561, -0.3379506012597251, 0.9846571171983931, 0.05249947320265713, -0.7945669525595436, 0.8918118894736322, 0.06696579756348853, -0.10577524169244623, 0.39234475340703745, -0.441408639741542, -0.20011370549094387, 0.9048179369414855, -0.7029583161826363, -0.9502635861517943, 0.28492898040650494, 0.6040589616851748, -0.831022774612824, 0.25707079488696416, 0.9469265701990213, 0.8915359307614859, 0.2682849536327676, -0.06846092928098124, 0.4599188214955299, 0.3521440505478277, 0.47079918914668406, -0.21406849335209555, -0.8057164003636104, -0.49849076736987774, -0.6547270566504384, -0.05136843123904766, 0.6630037181090145, -0.827851852576831, -0.7215483175118618, 0.8341547269179297, 0.8916902375334776, 0.7696426180376941, -0.31697423210950104, 0.5430857988392837, -0.4769515227585963, 0.7476355869163089, 0.45545421986594437, -0.47635972324234066, 0.54331840378809, -0.30985002400526485, 0.44466275418186463, -0.95343514466169, -0.9907006083706509, -0.8514720350366298, 0.25379668250190224, -0.001404555156322207, -0.622943463879899, 0.6910333828029587, 0.4113038255256487, -0.1162717331623595, -0.48557888812292904, 0.08249436119484099, 0.7487549640032318, 0.6065941113860911, 0.14550088333830735, -0.242174964691368, -0.3187874303217717, -0.7049890592333052, -0.254523377543761, 0.9173470258133054, -0.4977499511611947, -0.3028980863871624, -0.38423501667299487, -0.31987920552195215, 0.878972078216159, -0.06928474714938293, 0.287759631635899, 0.059394215461105127, -0.7027774784116758, 0.5550748086211215, -0.0619434421241436, -0.11532259127143685, 0.2810082629935251, 0.566683404726301, 0.5164404643474216, 0.09594157438381679, 0.6319782658112603, 0.7846504897622533, 0.049652000138030505, 0.34192301614481413, 0.7541279720463308, 0.24201836743664185, 0.455751432288098, -0.6393904630823568, 0.3546360182139059, 0.543382045268646, 0.5736938800114537, -0.5525317532539513, -0.5195915611189168, 0.910942633619716, -0.030984220872942725, -0.3296820696479994, -0.5523886001826415, 0.1994677430712528, 0.373375973092559, -0.945766793875449, -0.2903371189295778, 0.1012760391489671, -0.08365403088689072, -0.7317588120567977, -0.10003291269988979, 0.8248916182958561, -0.1425297374443768, -0.6560360971013395, 0.8053703576617683, 0.3616455380424297, -0.6808214214067951, 0.5257552121274462, 0.9432527171707987, 0.7181504351877515, -0.8693052061150182, -0.7895636244865041, -0.6234138595679228, -0.0016519494801829904, -0.5429751765523692, -0.5208044299355232, -0.7612003319376337, -0.4195546004552211, 0.38193276884860383, 0.16267540153455862, 0.7650009930135122, -0.8334759254449862, 0.7326271516410969, 0.36810090722497923, 0.7188017890463432, -0.8354939172632048, 0.1281116807367495, 0.4854275427352206, 0.7765667767351907, 0.018336597416116263, 0.905660217903185, 0.8448687884902306, 0.3274220833895347, -0.22930589682136504, -0.13343716620782775, -0.5814556327698042, 0.5380409901092005, 0.13425077087835624, 0.3906370636596057, -0.486810551555102, 0.020392911818217563, -0.3143805883594053, -0.4952942561003242, 0.853699805213985, -0.8292492796353994, 0.599403475991277, 0.31479731440571745, -0.9938107678448038, -0.9190499335631868, -0.24234487355886203, -0.5914959637283004, -0.422643002294566, -0.20565383178799435, 0.17347462474982978, -0.2263854086666226, 0.24705633752618583, -0.4836669493717425, -0.9776249558579266, -0.14803093839010684, 0.00980272004835303, 0.9947988453245848, -0.020241526418528943, -0.77042400590866, 0.2078992587273525, -0.8289956223753676, 0.6045675456898283, 0.25900076726407484, 0.6495546412365218, 0.9509194180523488, -0.08946551506488754, 0.021223479539181245, 0.1585612218958583, 0.999733926057174, -0.634967015318169, 0.6454641616165557, 0.15813024632316108, -0.5023038342084489, -0.4912726195756354, 0.1555907253774631, 0.057319514233804325, 0.3748151158658788, 0.6692091499132373, -0.25225296705200084, 0.46237385525097974, 0.4155593085428402, 0.5795703879518614, 0.6940553889307102, 0.44733628008948245, 0.030258017318009367, -0.7304239521129994, -0.6450372862961073, 0.2552740739441641, -0.6079579445661194, 0.9152684791371037, -0.49276700017540875, 0.9871310029249158, 0.290863071519464, -0.894628866254984, -0.12898469982716665, -0.8998725577946596, -0.6012402965828911, -0.4668694466257324, -0.02752586458984818, 0.04165285653533113, -0.8577687018242757, -0.9219858629330397, -0.5291496806411793, 0.45844553941092125, -0.005274471891569155, -0.39300456324178734, 0.5776249419974482, -0.4987000868461595, -0.40282414306984515, -0.7495731084558093, -0.2577562000618323, -0.8709165221339765, 0.19097796376649367, 0.7805885437237818, 0.34877413075039065, -0.9164938095372586, -0.375786179014717, 0.58061232840829, 0.06096547678984421, -0.42260230390150166, -0.5365633513690178, -0.5236715695796945, -0.05869954845770864, 0.1459509822522771, 0.721629731766086, 0.7970218914191896, 0.3035533974670961, -0.9901141779507223, 0.9016355934909472, 0.4376436058132085, -0.6504965332059367, 0.8562180843209275, 0.417237049636628, 0.07420060371198467, -0.22963742869343684, -0.6342316789618456, 0.36064259373384866, -0.6595225157488231, 0.31972322475056836, -0.926010677528081, 0.8539428024071771, -0.46054114147279535, 0.6668239529928732, 0.9423421324164398, -0.42898332536540273, 0.13428253500044018, -0.027674506949361133, 0.2086705513089726, 0.8776910563760143, 0.6887707152343967, 0.23536585236019314, 0.4716990809674795, 0.6126531178471635, 0.4231596699482669, 0.7626862110691508, -0.97415161873127, 0.6103427143554583, -0.9015434983068815, 0.373620801810137, 0.7553663161482598, -0.6852248492065676, 0.4271031581436382, 0.14150562351283558, -0.6038025242863048, 0.9758914084990404, 0.02345664899662947, -0.10553485704182108, 0.513145231004728, -0.3115580339013819, 0.7607245688980959, 0.888168671035594, -0.39943670664444086, -0.35946053970242065, 0.05062737530247263, 0.9378015956121788, 0.5443282270879617, -0.30385105402343116, -0.9900927954413496, -0.8887353759461554, 0.5356744866504664, 0.0835710748593459, -0.2809326044996401, -0.31055867643342494, -0.8060958878348203, 0.683198520213312, -0.6783791773029639, 0.14621712288822897, -0.40982082395145003, 0.3667664265378219, 0.6578457729062182, -0.34927908110787387, 0.3003814029348453, 0.15597298433132667, 0.3452584775785228, 0.3954737068450993, 0.8472584219977504, -0.41653907208163843, -0.47675209284395526, -0.9022991221371872, -0.07876622215841911, -0.8391042579067973, 0.874694176849961, -0.8902404669683479, 0.5197143720433142, -0.7907644782694652, 0.3005428800982082, 0.47919861853759094, 0.1671362405912009, 0.5962784533046273, 0.7222803320461861, 0.38953541264312963, 0.4956606261937948, 0.036520493556040146, -0.8153493499302031, 0.7228555257949554, -0.02530219770230402, -0.3613674961781226, 0.9349799534205809, -0.0845891432689374, -0.7945972202689235, -0.14049004056379122, 0.3954912021324253, -0.21480153256065226, 0.5907083569702769, -0.48378322470788815, -0.45994605335485894, -0.4457385818167765, -0.7663102094556562, -0.8436903232891819, 0.10784602341098037, -0.3316296631995803, -0.1597252558598361, 0.26015379336206856, -0.8259643062819728, 0.6655537678249905, -0.5579716395977012, -0.6140018134724292, -0.40597562669302034, 0.3030213824442536, 0.1915267829482139, -0.7922036472395908, -0.6419140982415041, -0.4921108556258751, -0.8601644565457269, -0.08790701659142686, -0.3340682305602709, -0.5201966303383745, 0.36673513250749434, 0.32973049446957137, 0.9437896727186268, 0.9032421784028049, -0.576487565312253, 0.9876725478656174, 0.19986187184664073, 0.10998638920187087, -0.756582144149635, -0.3956529936661637, 0.9779231610798398, -0.8570842173287623, 0.40363376605425993, 0.27765325060052914, -0.39606747110304985, 0.9214187469592869, 0.2714684607100568, 0.6825708086066962, 0.6381293464556492, 0.673117334820392, 0.14969587670809492, 0.10161004566602383, -0.4039345391606308, -0.21410125088905474, 0.08423849325295274, 0.0781993301834254, 0.5509502299903353, 0.9263315634732041, 0.21256454943444614, -0.7182930268980618, -0.6708945948971183, 0.24979652042147604, 0.2600806449151867, 0.41752878624671497, -0.4804958961954917, 0.4054922123975784, -0.726405284771211, -0.9714255749639034, -0.31982369006307954, 0.9496778006839213, -0.3467471425756017, 0.5731581985140992, -0.9088559067690178, 0.4707511251005936, 0.3983787381037349, 0.7876054586395931, -0.8695885098891971, 0.07419228361157426, -0.7657202927063811, -0.06545184981482133, 0.648740224859476, -0.24238839278745128, 0.12067448101259148, 0.8663931692276052, -0.5799877993272842, 0.7228019255984215, 0.34023274471869036, -0.23759956669504212, -0.9447533784844524, 0.34494028664095544, 0.20416903199687964, -0.8941321533077147, -0.02387198094195586, -0.04174081940985874, 0.5038299757816751, -0.19031084699586054, 0.7340587910960084, -0.5186026799888277, -0.9881216595152984, -0.009171754817946187, -0.25093419822906915, 0.6054800685276822, -0.7892617664565225, 0.4101112802925131, -0.12228374177875945, -0.15072341906861486, -0.5436662613230248, 0.23480074888011893, -0.2741458308439719, 0.5754354220383346, -0.6253362287820754, -0.2417922147847691, 0.6887853404393349, -0.07681845656232045, 0.7421736760614537, -0.19610404424298555, -0.21915132254261938, -0.11231469182787102, 0.8288200108311039, 0.43827975618540815, -0.7331433093756166, 0.4071415928687889, -0.311014697941558, -0.47397116318651467, -0.9970013734290517, 0.31782732495726584, -0.9129352206543926, 0.17438186382625886, 0.2277741971301972, -0.7286132139212642, 0.9567691575670825, -0.13825409621560758, -0.17633485366049828, -0.8221391844778585, 0.025118358391746254, -0.8069332592345035, -0.7787885029545942, -0.6652286020971312, 0.4670890648969326, -0.8274754900169625, -0.6247579595674497, -0.1707382381256437, 0.488514281826846, 0.4421250978849156, -0.8740411768965255, -0.870272392279402, 0.1345268713676666, -0.557609169716349, -0.33566398527566244, -0.19675017467902456, 0.254311649278137, -0.8988910339007725, 0.4426509799994973, 0.017939151924978525, -0.9016152196995904, 0.5081732873395357, 0.2373357607621791, -0.038711467333228944, -0.9730242418570347, -0.29742413789173106, -0.807534791648173, 0.4925762740181532, 0.5841883107743324, 0.9876587022283898, -0.7816173432578037, 0.4964215572428474, 0.48052445028400115, 0.6046315234380517, 0.38220933319464967, 0.5535154374021991, 0.9557647615563791, 0.9080192709406938, -0.8541940402432584, -0.0014929704899155904, 0.9500316814970502, 0.38560223074822164, -0.6106624108592393, -0.6254787078292869, 0.829922501060619, -0.8570457099104005, 0.9348652598318825, 0.5440636332596926, -0.6581713043461472, -0.9286771116473036, 0.8630146403644843, 0.7214251525140445, -0.44466819651901135, 0.6434271778824721, 0.32696342983035365, -0.7156234938859909, 0.452203919432423, -0.7132475707142507, -0.5086366169848109, -0.9400312267841597, 0.3213796148804453, -0.44028497458942484, -0.7608895531550204, 0.9567872568069302, 0.6457635408865692, -0.766132454958506, -0.9364985902863636, 0.17957577970634353, 0.7956494564732359, -0.06433480295147831, -0.3481844924505353, 0.0023961745452638628, -0.3031891603192136, 0.01917068943684841, 0.5111217787521039, -0.9908302491173202, 0.9919820226663636, 0.9983312030793112, 0.7072559665060958, 0.6503315708974542, -0.08720836864438608, 0.9182995581575228, -0.8174340118864973, -0.7809551937616963, -0.5077469540919395, -0.10566595748671204, -0.6180601451512191, 0.42670656980203114, -0.11958120947023088, -0.33472641444422035, -0.726331483026643, 0.8227432363008254, 0.29042859175063485, -0.40816128755242875, -0.9136387369434875, -0.8172249486395564, -0.10348104180794215, -0.296099395033119, 0.026187740170759977, 0.9509217548658702, -0.8806009891359006, 0.2096165480023271, 0.12540370113886357, 0.9785165658061248, 0.5299123845701135, -0.8523487560447902, 0.6842398349541183, -0.9862732055637808, -0.24025475693586107, 0.5860590582694734, -0.8370764737757497, -0.28818219249111543, 0.07261047682072364, -0.043298545444105496, 0.43129082866289026, 0.03004042114555605, 0.7762005296222318, -0.1765773884692805, -0.6139323340473182, 0.9297304865309337, 0.5715538751064806, -0.6805181031898597, 0.1331563570337282, 0.8621724103594612, 0.3034571539111741, 0.9506816099513811, 0.3209777441230024, -0.31869958367936535, 0.710282985988965, 0.9962703715797874, -0.10681307184901634, -0.564464476058862, 0.3203200816893792, 0.5662517512734195, 0.32410979736809953, -0.3406778318092052, 0.4979036684019753, -0.7248418308559317, 0.6050176639109437, -0.5101752751335915, -0.1149108238292591, 0.7178112820657176, 0.261523793540432, 0.6947936138477346, -0.9477699234133385, 0.7754106049935561, -0.6721190561620851, 0.6034029784220971, -0.08427508984975329, 0.5773740132790479, 0.8475430597316818, -0.5648248748530875, -0.7743123857266299, -0.8853182184046307, -0.6933853040414764, 0.49616127417242284, 0.7115598480240006, 0.8466576832558104, -0.712505010474678, -0.3156057526679861, 0.9067652863690405, 0.1303216749568703, -0.21747460895483472, 0.6805650456138608, -0.8756029602622126, 0.09223954961331926, 0.077921162630471, -0.7880420919665994, -0.4495132054607074, -0.22288078579847426, 0.9647813888857575, 0.21616029379863155, -0.37262529606530537, -0.4439080739785872, -0.3876998006205179, 0.1853517323864371, -0.09865270398060688, -0.4413371935037078, -0.027357874389055903, 0.8351918290198297, -0.4715690498263063, 0.7282395500283994, 0.6893829595218803, -0.059070101902040095, -0.8072403286649286, -0.1619821834007067, -0.25126328361374894, 0.7275342860793379, 0.5675912392919291, -0.6782352152683637, -0.35252530056086506, -0.3396863546283735, 0.021391345051685606, 0.05908011151112569, 0.347085604492007, -0.5482633485084123, 0.2863019390601276, 0.5342047451423118, 0.8688447345646628, 0.38876128645525854, -0.8789073504001523, 0.5386379994269779, 0.5444852360648511, -0.5208233585997384, 0.5165337564743728, -0.7475494832310572, 0.5644217442142989, 0.12517477304309765, -0.31016901526961993, -0.5474431792758192, -0.16346555721687617, -0.5117312132206344, -0.8384437886513936, 0.96996501976782, 0.45947120344260783, -0.5858214784369582, -0.6790292869318812, -0.9550910240975199, 0.46908547939257117, 0.5512713325375551, 0.5144392882468989, 0.20719450952161544, 0.5890351447325861, 0.3469597373823996, -0.32679423933460305, -0.6742345868895976, -0.5837858145332409, 0.4577529303255876, -0.9813831300689475, -0.12097553113985038, 0.37541660688482303, -0.7837007254104582, 0.29209041198995256, -0.8642937039334058, 0.5630794859174959, -0.1274881071972993, -0.22606087002185715, -0.742301303302215, 0.5061062067362692, -0.8511479299185865, 0.2000869949219688, 0.9879244461030325, -0.8438342953136242, 0.5876300920941466, -0.5660622748273716, -0.09527657500104869, -0.6682959172557608, 0.6333320261902993, 0.1678878630355607, -0.711212450189113, 0.07295690583668812, 0.08903056462840264, -0.35823105729572813, -0.368269232033704, -0.2966017316394296, -0.24994759852357218, 0.6301382528486197, -0.9514843164174807, 0.5208518171039409, -0.1314445915912985, -0.5558800154301182, 0.8622638320112004, 0.6209086310308667, 0.39081546319319815, 0.8145805883371195, -0.4432860036195605, -0.2959232355045107, -0.3840959372242412, -0.6070304837989287, -0.27476912839765544, 0.624309255145961, -0.5891864807018763, 0.8866572660134551, 0.012326237546182783, -0.7713860113314113, -0.7101534672941223, -0.46871225370416214, -0.15550315016374539, 0.17444161531246527, 0.2889665022752794, 0.3745268563113051, -0.6905220541549613, -0.3004265602902163, -0.5847583705468107, -0.859466659211249, 0.21604994092533203, -0.004465167020117455, -0.8499239290660321, 0.05730076423659569, -0.5342371630597289, -0.9330104462948383, 0.3993398846534921, 0.16492685716300204, 0.5492138713370551, -0.22993094275803627, -0.5569238422760874, -0.7857571996786847, -0.6435765954452008, -0.2541694843810891, 0.31445474979902643, -0.8029407357371781, -0.43153943506600556, -0.318533786577089, 0.10654220960822469, 0.04745382986430946, -0.8722005214047868, -0.7668718063193913, 0.28121917599916424, -0.8868354825610627, -0.025207368267379904, -0.9736278883432214, -0.6786069761563249, 0.5791057942900248, 0.5661048428453113, 0.35078331038585087, 0.6039760677136674, -0.7908392868429717, 0.07567791320233841, 0.07299137437406866, -0.7734971905139263, 0.7097146031236383, 0.7218100977216919, -0.5811505046117549, -0.19306685164482573, 0.30378154500298726, -0.23861158290805284, 0.15286624082313005, -0.9272948071099143, -0.7917557269145847, 0.01072151755615569, 0.9777801587298609, 0.629529598757028, 0.6506598390717384, -0.9151023351267376, 0.9640453506982405, 0.6944975031722471, 0.9369980515058762, 0.2953071252331303, -0.36219609720887025, -0.04872731481672421, -0.7760931603339323, 0.008920888589291076, -0.046841121022831, 0.9273584375872921, 0.3362456720261662, -0.36641334183489715, 0.6278718918737547, 0.1454005555116431, 0.32603312260203077, -0.545103502487138, -0.5636232391705738, -0.3337061391020173, -0.7698887064744766, -0.49956857768014085, -0.2729512325633652, 0.757203866407868, 0.7080337718714353, 0.25296177653213037, 0.41828471073099327, 0.49766753519494067, 0.6943868152712274, -0.377756292633616, 0.419321644041335, 0.43887435649756257, -0.7342179201701795, 0.11301601001460049, 0.3672306979395732, -0.36864369474567904, -0.7329977085386032, -0.19553112361189973, -0.8460745583371956, 0.9526508155658133, 0.3762893679533461, 0.6348848946819834, 0.9503169059200882, 0.5431068576777491, 0.7035631322461269, -0.6245608433583769, -0.7649163836910757, -0.09862865170147006, -0.6562843223071912, -0.08851331131601481, -0.5435283829627022, -0.09836126937830802, 0.5433037821208071, 0.7240255791857928, 0.8887106217040612, 0.594057253126764, -0.40441675856291703, -0.581919483802759, 0.9285255223518094, -0.6422738409041324, 0.65140823525771, -0.13772340954222573, 0.0362542615808199, -0.5763267717655407, -0.12397792212730274, 0.13568154431161972, -0.2078872760643229, -0.5972144376547741, -0.14219210304740693, -0.21917044085420279, -0.4647482019154634, 0.15910342159726865, 0.35254341232435604, 0.8976953170016049, -0.4289842954532048, -0.23879037155302707, 0.40606152860434785, 0.5182517772787563, -0.6786392590085844, -0.5380247379020962, 0.5235530621318576, -0.1890806682707895, 0.43273408626229704, -0.30968232685832464, 0.8216065092687843, 0.8238073277639866, 0.7344219431180039, -0.984650428202513, 0.08697950797010612, 0.09494639882853484, -0.9657564951527162, -0.5898509788247928, -0.32900377270186754, -0.07243357845654463, -0.6532672896610754, -0.3014319817906699, 0.04359910718484228, -0.781701481272292, -0.26543833154304064, -0.7190395786769654, -0.47937915074602966, -0.9847626605471191, -0.26370341308440826, -0.6680831143525625, 0.8669466179085317, -0.7508199896945824, 0.9982713217277137, 0.95569073384144, 0.6605565878512949, -0.6008372585283832, -0.006592041816220862, -0.11031148859101991, -0.2777569997295972, -0.6146320058062686, 0.1177670797544681, -0.7441583398795129, -0.19155188697514292, 0.8595241633794151, 0.40699982970087456, -0.6816369763484262, 0.6902548900005212, -0.5090475276922932, -0.8898645420314013, -0.15535908773781304, 0.2313629047275445, 0.23562908871749233, 0.9836879074548599, -0.777612087253128, 0.7408803223163654, -0.875436209345025, -0.12233290661873797, 0.41818753611302295, 0.19726747160224378, -0.19859408266492862, 0.10286254911593495, 0.9702767119370224, -0.831699707904936, 0.9780799715522157, 0.8269310647779315, 0.730935127665028, 0.9299151793506713, -0.1316072454273569, -0.548483276188604, -0.7316913728882923, 0.00932555583392558, 0.8585124560478343, 0.5997308988458963, 0.04537254942508473, 0.4022248005560072, -0.6307660028313016, -0.8662447440046623, 0.5892624996479783, 0.6746913904085967, 0.8000036946013112, -0.5861147784434626, -0.516023535600088, 0.7945885503862318, 0.019060174041824585, -0.05838431187480109, -0.6164173350412923, 0.7642603914368242, 0.037957963941768647, -0.8925645350942126, 0.3530215950290876, -0.30960693769979786, -0.6973832341452271, 0.49343651123995835, -0.8332231450951764, -0.54556442053485, 0.30136852220374233, 0.7773161302826219, 0.6821701751157023, 0.9750509266386309, -0.6984703047565013, 0.13355643472507217, -0.9541441178325225, 0.822882095951593, 0.39117039598218195, -0.24808828913486924, 0.2740831275763653, 0.22684663479697198, -0.08098398344517399, -0.5074149247579931, 0.9707909048212764, -0.4678649798301231, 0.29923640524512507, 0.9539949107325632, -0.32588304197117, 0.29005441780745156, -0.028524380151205397, -0.5705593103830833, 0.5080435150008706, 0.46722054781810174, 0.07447672743047629, 0.6908369413254551, -0.08058958024834362, 0.045613388750406436, 0.8132492674843836, -0.3958419794918453, -0.4489329095595951, -0.2531007608799427, 0.3173727160858417, -0.30904875606853555, -0.4630526851999568, -0.3243148150471318, -0.340631459008899, -0.9874796648188655, -0.021343255236360825, -0.2938510923807316, -0.9718113654499685, 0.2861733290513526, 0.8418026276834871, -0.47342640126061375, 0.7029392544822206, 0.9079106309434566, 0.2855647827610506, -0.278312540690133, -0.13431951358959338, 0.17487120244541376, -0.8333076607022718, 0.1478241515231582, -0.4933867517287003, -0.011774522267931298, -0.32402346849026586, -0.7804845153017126, -0.8125759012780602, 0.6543391733917148, -0.07409348774295554, 0.5804807778658623, -0.6184079930238235, -0.07279044316816896, 0.02707814312697132, -0.4771209213877077, 0.1670149570274797, -0.10089556549514733, 0.2792985258854819, 0.35387748340780467, -0.8891206232273607, -0.6900233311568578, 0.5404396488308458, 0.6041573755501892, -0.689985571488235, -0.8732230136090207, 0.9874962200810347, -0.13655419244370193, 0.4523975689269286, -0.9827317139943912, -0.33664508748454525, -0.49301413425773366, -0.31422258104984313, -0.3342852507799208, -0.33513393706874806, 0.14826792123860666, -0.10500778888442164, -0.5507585294133397, -0.2948979973469641, 0.8600012913666535, -0.8306094934656945, -0.9910630796706192, 0.1487757561427836, -0.5456229026632171, -0.9575616942012126, 0.6761215063821713, 0.739398207098914, -0.2983801609839132, -0.18796060058381214, -0.06950440494677723, 0.835079790249257, -0.51330874642661, -0.6137210480850885, 0.6675171599062895, -0.25944335063394064, -0.3187345375878321, 0.111350235723225, 0.7990947638837769, -0.8294339029004436, 0.05901786711003254, -0.2511997380088862, -0.7987677024105739, 0.2299807245443548, -0.5289901518800304, 0.7029121387018469, -0.8387493281354432, 0.8230830459947958, -0.9249818354036854, 0.16982381589856166, -0.7842706109867144, 0.23571371870189695, -0.4870289208211984, -0.7780950290946178, 0.03277958253500102, -0.17509227569797758, -0.16270689015680895, 0.5010140895791815, -0.5376489260617456, 0.3907973017560973, 0.4776191542557453, 0.7198941858391537, -0.735955911952586, -0.1807820530339439, -0.5005885001111265, 0.19889271591669444, -0.058871829586365054, 0.39425886662505816, 0.575762793277176, 0.9386763614369509, -0.9265023041994567, 0.021176930211581624, 0.23982436695109755, -0.07761822194809054, 0.8726954146106229, -0.012256580753023272, -0.46854145084664234, -0.05353179369839833, -0.3754857198222512, -0.8128991153868799, 0.7100040756871009, 0.17506009468089268, 0.47942460062925263, -0.09138328040476185, -0.3847782435395828, 0.48456298433576905, -0.6043459291375297, -0.18621880454384065, -0.14176895633830888, 0.6378456813125812, 0.7829361725502619, 0.9731176531160581, -0.4449177233352455, -0.23026942939603612, -0.5890493656405922, 0.24452568796024776, 0.7348803225967593, -0.6951047531003618, 0.5960806475608238, 0.40529750814377397, -0.08307513596748839, 0.5058941371432271, -0.449763652106967, 0.2794112407106557, 0.581371124041091, 0.2507138883057287, 0.6051911820040572, -0.2896306433424263, 0.4247360736608672, -0.11518575544230836, 0.9612438743476193, -0.0902322298886511, -0.39311860932282205, -0.7694754922102212, 0.6764990940915372, -0.2209330775398768, -0.3334097035776178, 0.921570516166746, -0.5831466248970161, -0.55693688059742, 0.9142692904581198, -0.6051671297291688, 0.5372160370212589, -0.1342306027164717, 0.5159899833644006, -0.7162322774526013, -0.6105698494937664, 0.42681504411301296, 0.16271513352931466, 0.38587931980339674, 0.8568604188046953, -0.5910921809493375, -0.4993130426867418, 0.5543874385248837, -0.0814608247686408, -0.949168537324474, -0.3397626233897506, 0.5801929258871961, 0.9801485217186361, 0.7969182134092192, -0.7323992003439459, 0.7474658198750319, -0.5680739358046918, 0.6547542280512639, 0.37621289093990873, -0.015692725944810748, -0.7550991403020837, -0.29210712161967756, -0.14811296966610543, 0.014183700977699276, -0.45411944484506606, -0.890651154447204, -0.7335522793547689, 0.9689817036438186, -0.433812904393317, -0.2358765801885414, -0.6757645233327993, -0.03975660561012084, 0.815847610842118, -0.40308621733240346, -0.42539152723167617, -0.2567472883454609, 0.3538250794867437, 0.17787766255884185, 0.8594229706486267, -0.04790369671459982, 0.28867651613440737, 0.6149383385245131, 0.4919221596767902, 0.038250325570082344, -0.46323840787549964, -0.8680469791043028, 0.3623762158606916, 0.36184799528501843, -0.049183687530755194, 0.0536327137393533, -0.5457111500466756, 0.9568053903283036, 0.6258054085231719, -0.6278760662132243, 0.6796741203509116, -0.3396572080782021, 0.9988315675897299, -0.1374981153989121, -0.595263948148979, 0.06062122816178128, 0.3823446254950469, 0.1730855058177443, -0.9519932482705602, 0.06644244398450305, 0.831985911276937, -0.6777059311460154, 0.41392451269813413, 0.6131637374438568, 0.05493657365342641, -0.9272265596761069, 0.04804595120464339, 0.759937014385061, 0.681285719022622, -0.5662518808039851, 0.3930281490196079, 0.5475192695940863, -0.46705531365169994, 0.7788817806346395, -0.8457003538184502, 0.623853401568349, -0.8902973498013387, 0.9792388793542615, 0.9740066564391392, 0.24940301549755572, 0.06429303744030301, 0.022566715367457357, -0.9112183418846531, 0.27044428686498767, -0.3068641337147531, -0.9053995138720634, 0.15209796682385401, 0.016373087034466716, -0.9783042059523814, 0.3468243704135494, 0.8285020612170604, -0.9123747593700811, -0.6717618373028087, -0.5298090147689016, -0.41688711736698014, -0.435762202508835, 0.8344264721651331, 0.13937271475918656, 0.12020947426027306, 0.3153396818830263, -0.3661120366501798, 0.8910024341956835, 0.33644011236129057, -0.6277008643954416, 0.17806191469227617, 0.8842485558707647, 0.34769014723293035, 0.4227539805957794, -0.4575031142821151, 0.9384113510540033, 0.01032882489420417, -0.8044492479285525, -0.5219892126157935, 0.9629676122175455, -0.9619911032941588, 0.8488627599351621, 0.083112729489762, -0.77837980665254, -0.5741575406939945, -0.24780417741716532, -0.3313881950731472, 0.3348322685404519, 0.983394092892139, -0.3305254438723735, 0.28421725997984093, 0.5658403620916408, -0.4303374855897193, -0.9963310502876466, -0.29535743123323943, 0.2896497415545427, 0.3521095657163942, 0.047587650134715664, -0.6667340069282472, -0.09305144979761026, -0.4585557382795591, 0.5961669430268455, 0.10308432766947506, 0.3325758210724199, 0.24261323396699663, -0.7002974975367113, -0.7878231767329078, 0.3309359298361283, -0.6729105206440251, -0.7034939297427072, 0.21938603539631885, -0.20849953214551986, -0.6332964315899556, 0.3630622429849213, 0.4811656099685202, 0.8552411676424789, -0.837866319514426, -0.8098753859293164, -0.44850071887213394, 0.3156842765075627, 0.9857111112615617, 0.1401087568724917, 0.20948210215212448, -0.3820816768023978, -0.1286013505513146, 0.909247371986869, 0.4949719082335855, -0.9125239553273752, 0.29376243512367384, 0.9752057639731058, -0.7605341600250635, -0.4357168149703008, 0.27946696899415113, 0.9353083352213745, -0.000805802950061496, -0.66758749588792, 0.38483477167311886, -0.7610182394165814, 0.3226419818978692, 0.9083301908686674, -0.6856009789905195, 0.04138307853005552, 0.23325236658776216, -0.835346843185389, -0.362981324665014, 0.4250766656381413, -0.7492538057669926, -0.025503442564866363, 0.03702030105897025, 0.7705606679997634, -0.8904638405501557, -0.6766992437305355, 0.1364801186053426, 0.7515159188801759, -0.03848401598298201, -0.5348441630805194, -0.6513803673663554, -0.8635679460220107, 0.7781824713000707, 0.4265969817276827, 0.34038994131730815, 0.44854983307883534, -0.5795342078755055, 0.5962174957328767, -0.004916258571632026, -0.5573480989379738, -0.005555949370304569, 0.5172067991651803, -0.9696549847379232, -0.23715818808137867, 0.03973219126477634, 0.07780736023446977, 0.8234131977365284, 0.2769324823120125, -0.5672175820180743, -0.24707785135967097, -0.5754104440678156, 0.6997512065988034, 0.107741485116021, 0.6392166791976288, 0.26895173847120124, 0.871255680589281, 0.1016320400474644, 0.12698551773501077, -0.4413118922963295, 0.27615706150327535, -0.6895611511290922, -0.6557582332730496, 0.6050160758647003, -0.912047107809602, -0.32272000048243843, -0.018151403443503078, -0.42894394396580715, -0.41492506238775606, -0.765319138741622, 0.10398582923520672, -0.7535022527648791, -0.9052582946662313, -0.055493466558753424, -0.5649663420566002, 0.572113548159797, 0.8946645678702481, 0.07129002730211154, 0.8462468664740082, 0.47837915935356135, -0.8135209358463431, 0.694928990497711, 0.055712824638677105, -0.49565940738765835, -0.7291374561320894, -0.5988997904663684, 0.014601521301982778, -0.4038544841093541, 0.010959876266958934, 0.9266645077915769, -0.6691767892586362, 0.020844003399424382, 0.2934393584813062, -0.7510656229199835, 0.2850521821742551, -0.942837964605719, 0.1658153537735818, 0.9935706867942606, 0.8932576387848195, -0.5431347020603683, 0.3398315264535061, -0.40063188722899645, 0.8551390235461951, 0.04614050422586069, 0.1879095369536492, -0.8370305450949256, 0.9559305822358497, -0.3693860367610422, 0.854206819177848, 0.6066524020335966, 0.271212734650794, 0.5297641357547005, 0.06922824245341963, -0.318687963356282, 0.5578396291228693, -0.6673728835321358, 0.31059153905879433, 0.19071145450100402, 0.11626463636553575, -0.09147633024068158, 0.19291895676432214, -0.6199835875343402, 0.9540838057418739, 0.1435280652696167, 0.446829511043372, 0.7131212148862063, 0.4131354674028893, -0.38529366473555626, 0.6598668149239257, -0.002835982182744168, 0.8221402644364186, 0.8650721119624927, 0.3766455914833462, -0.840533516630267, 0.46625062411772067, -0.5859882199216297, 0.023962421273650003, -0.18630644214222625, -0.4990295231008357, 0.19169434722930934, -0.30117207323350104, 0.8804045534270053, 0.3096146678053091, -0.7908906021496191, -0.647355176301136, -0.5938432215299945, 0.593688231872231, -0.7903918985792313, 0.29975839531053095, 0.3460796925486884, -0.4103837339489227, 0.9341755411378372, -0.6502096292896602, -0.7381581263008348, -0.14611486036836197, -0.7271165119453962, 0.28468921798498137, 0.8072783407395572, -0.9981022079582011, 0.371242954386366, 0.9303795308868612, -0.14441541500411437, -0.6113987527731766, 0.32479374397697014, -0.5499336626189972, -0.6368759128072248, -0.3633435686677471, 0.3223366219171693, 0.94029262572244, -0.5547295589557737, 0.8292617587841242, -0.5123445483092783, 0.1778802055833777, -0.5831465719320801, 0.409260808416936, -0.33831821061765543, 0.7065469492524463, -0.88652093286962, 0.45703768772825093, 0.959569145377356, -0.6260954058432682, 0.019395934669648396, 0.6536942196680287, -0.005872151484949573, -0.1441221902200407, -0.57985120858984, 0.6100514608692613, -0.3521766297660023, 0.4789103111715629, -0.7289562847238427, 0.038118080612560945, 0.5209339771806285, 0.9091660183911845, 0.10256254453348856, -0.27805785394277804, 0.9816904905952766, -0.5757163173687929, 0.1692269478155628, -0.09269230714150978, 0.7800759098876227, -0.8032277375276307, -0.8748882708710439, -0.8584791617038356, 0.709380720110623, 0.2839689721448304, 0.5397680525670627, 0.22213602168000013, -0.14783419554862776, -0.6141141753262622, -0.3960236305161964, -0.9450247241211218, -0.06856621070664093, -0.8581984199166364, 0.171610190102639, -0.7983170258923584, -0.5715850209394953, 0.5626272758295188, -0.4790330384873258, 0.59345133368509, -0.023255045503078886, 0.9542476623277094, 0.30382731050929435, 0.364846837699379, 0.8870635066672636, -0.07258319855347639, -0.30578717154116375, 0.3272891298350935, -0.43174948682454595, 0.9433653283723715, -0.15093528140134205, -0.5341210193756256, -0.840327355235138, -0.4902904918474902, 0.5447241471601829, 0.3049238975429833, -0.26983011655147804, -0.2320200661197318, -0.05334613499464935, -0.5942267446256277, -0.6514985819417995, 0.3151878997158857, -0.3273094259424907, 0.004071267762002062, 0.5611536372964292, 0.8795147864684107, 0.5243999246059505, 0.5145701196903458, 0.05539532083957255, 0.8127190845019534, 0.03637974952742029, -0.956462024037251, -0.1849413205831003, -0.7523345575663523, 0.7387494357108202, 0.25628871207481074, -0.9323293924272988, 0.9003131762909766, 0.015195882927892423, 0.04840841861618328, 0.58437624448116, -0.5656961790153141, -0.9851782300085505, 0.612985720024563, 0.36972345111950666, 0.30686241779773527, 0.7058144186906294, -0.8633801898626277, 0.38860379667685496, 0.7580421177145604, -0.29354100683322915, -0.08826966820328908, 0.39802108354699506, -0.592055914863054, -0.7293773976365703, 0.8029500785794308, -0.5021393795950484, -0.6183359610018535, -0.9036640448881674, 0.6124726815951005, 0.14026471456112422, 0.37901985366094504, 0.691913858308776, 0.21455728939380503, 0.00011679514457729745, -0.29888564115485816, -0.9772419418284892, -0.7247953697743845, -0.9842254187275334, 0.7582138240865057, -0.6321063151042383, 0.4406837971374873, 0.7973913082105726, -0.648940959460226, -0.22226502671256387, 0.5020991808459752, -0.1742068754103101, -0.24323142922023333, -0.8144937735856712, -0.1736165007034287, 0.15629497553773786, 0.1324440756801173, -0.1726048186960698, 0.21659004249064862, -0.041222665024596594, -0.9670956316354942, -0.2743659377381631, 0.23107697237750768, 0.5658900523862107, -0.8487608181054342, -0.682360211084108, -0.9499575385698502, -0.3020818495146229, -0.1941424756642156, -0.31153634754674053, -0.6932833289471179, -0.6143834621812418, -0.16308794674716953, -0.19000550628626356, -0.6112643747990141, -0.29725817630504925, -0.5639256470055038, -0.7672123412738974, -0.194847208548248, 0.9038860921027645, -0.6530281279375203, 0.8628645186246506, 0.6171081545714063, -0.9876966118761201, 0.15798972282562618, 0.8885105772023736, -0.1459945743705604, 0.9856677006693184, -0.29309690341266736, -0.8181740757560929, 0.11664606161295521, -0.6428495867704282, 0.53752470270337, 0.4161272114661063, -0.9657852402008045, -0.03179358552159384, 0.9954936890711894, -0.7856847951009966, -0.036268051197987594, -0.10680897110593501, 0.6665203065142475, 0.18747704799693898, 0.3736990744785602, 0.045010577498427295, 0.10721757989373115, -0.8737517415140454, -0.873205473916701, 0.3769804203909983, -0.27367393223235714, 0.14672013598207156, -0.027503110723179125, 0.8311913564116502, 0.8302574005101511, -0.08090030821869143, -0.5220992235880844, 0.5161668347096198, 0.8916728619967871, -0.4019986658654344, -0.798959822185151, -0.8956241995085705, 0.4472195234157841, -0.11186586629545836, -0.9034278853514033, -0.9758909303365799, -0.9473954091835575, -0.6709321524839447, 0.3576617661495438, -0.12155740974252227, 0.1900078988308076, 0.9646908288603753, 0.038098356902443786, -0.0044971848728032615, -0.9070249946542004, 0.6782041438288013, -0.4328796147830334, -0.7466362257203065, 0.5248662559429147, 0.9829592352220349, 0.13503149150117988, -0.5311745231937015, 0.7068060714600926, 0.8931482957071837, 0.009543783277131235, 0.8674160293733377, 0.8346674252077018, 0.38268298262701883, 0.2740879481542382, -0.5410744907926872, -0.3390864410767691, 0.4183721185851639, 0.2500888852460219, -0.06017364351637111, -0.9009850700493438, 0.37374540267590595, -0.5925893549230354, 0.6239336687354935, -0.5117836275422754, -0.9478787509281661, 0.28877857759229375, -0.8624069849395408, -0.6267113745600204, 0.7316596710134495, -0.034257198858873794, -0.5483269257375925, 0.7385329316036988, -0.4976811371884393, 0.45473725652635544, -0.583875910837798, -0.11992664015250787, -0.7135646735955423, 0.6110051335840085, -0.7506924877695889, 0.6781170082243491, 0.5085365766423957, 0.2038888218658974, 0.6280921411743912, 0.9566924753667974, -0.23653942993568688, -0.7174453647413948, 0.7227849350673241, -0.17180319995440652, 0.5602670826194929, 0.8094781216958731, -0.07002262615593491, -0.9697504284843543, -0.613449412359159, 0.8823260595975724, 0.5945710119967149, -0.6551590825105953, -0.3514575717502144, 0.6277565476030738, -0.7941765082036305, 0.938712529463015, 0.25086744713619225, 0.8859991323803371, 0.8393229489878884, -0.6097778489516341, 0.7541135952930726, 0.19219572236077576, 0.19498247335914565, -0.13247651422075202, 0.026211498545461565, -0.63639914637168, 0.8767237778978196, -0.6713694808648081, 0.9378235963689447, 0.7705679830655943, -0.7755326450619076, -0.5695824643147351, -0.30990763125807286, 0.7188632567585034, -0.9501291136530507, -0.5535209778624179, -0.8996944029150209, 0.23680787872305165, -0.3119019193144037, -0.15510950011332336, -0.652832520330429, 0.3560627302545085, -0.7004795991391524, 0.39296516307991425, -0.38591989313807695, 0.722409006605724, 0.1810153064656974, 0.9026707832174965, 0.6858522687931223, -0.4165587385180789, 0.47603073519205763, -0.6283496297803957, 0.39516348799384615, -0.04342727883308317, -0.7959139614313013, 0.6877611187118278, 0.76754697500429, 0.7344677695190993, 0.8787538272063109, -0.6182326981886328, 0.47547115197093115, -0.5680829729219155, -0.45875596991617096, 0.7513895477359787, 0.9710614714918491, -0.1791654296102465, 0.6370559853057369, -0.42033355281399576, -0.8075403083435826, 0.5387765763929089, -0.2704032909795535, -0.562469906511293, -0.3566593120383661, 0.34405431382425644, 0.22931509444608533, 0.3394267007111229, -0.4796929717454297, 0.04105812604613135, -0.5683002557126413, -0.28468846567005945, -0.8946616782308803, 0.34897871926283996, 0.5794190234765824, 0.6903759400127341, -0.5534708426613264, -0.406700003718796, -0.25052556129764314, -0.25468866790832356, 0.5046927249476176, 0.823443159079134, 0.5349970799899668, 0.15858711102381307, 0.9327153163593911, -0.3727917037171733, -0.021573051663041465, 0.7286972664825162, 0.43284081281014775, 0.9627520286236395, 0.13641804949562975, 0.16751353192475693, -0.6721561621716048, 0.9251319943744019, -0.161687648107107, 0.44229815586400956, 0.40923049309119675, -0.20211219279234016, 0.09299877865278816, -0.00938538631402186, -0.5128713896450161, 0.8435235452659302, 0.8028231357870428, -0.7072874608250284, 0.5863862992040316, 0.758838850491734, 0.07851294923012864, 0.793486010388186, 0.4443495681308838, 0.8012653010246318, -0.4210649534151194, -0.20388372835771573, 0.7016353325086169, 0.38614101113344557, 0.31132435446364526, 0.4811168162956221, 0.16808926970756444, 0.3631076800254145, 0.7360940239254787, -0.26836319228698424, 0.20110242129566203, 0.28700073252813607, 0.7752500702722829, -0.4046899383914857, -0.8643363762695464, 0.9764738085664773, -0.542870984697831, -0.7312548252454325, -0.631505953541555, 0.09480075107697306, 0.5286875460534479, -0.6231449811947964, -0.8776684190693673, -0.8584940285734075, 0.8437813910826271, -0.8011454762734158, -0.13929921686519586, -0.7651321700388083, 0.5566880012306465, 0.3991204162733524, 0.667629570702702, 0.5492165374714755, 0.3097904812615613, -0.831531268367885, -0.8706045398082729, 0.12276917352519368, 0.6409503200492388, -0.49814199642976553, -0.26976030975762333, -0.7299550530706749, -0.7152484852544785, -0.9017930437622945, -0.31907621976646827, 0.704069889365623, 0.5631630170400552, 0.12819188685541705, -0.9832617698365465, 0.9557785992302881, 0.7214993075235401, -0.5679902964360535, 0.538629080250616, 0.1602009390464305, -0.8020230213867072, -0.22613968375703086, -0.43328927260596006, 0.7458502066032566, -0.9424538219363201, 0.04454933478320422, 0.40851411675025573, -0.8282836353480636, -0.9420285758409894, -0.7038380308314185, 0.2574268143478302, -0.23731012747457214, -0.9707032325061684, -0.1870062701633135, -0.2685112165867978, -0.7411426260619247, 0.5048658027610315, 0.3975288722630197, 0.8883614010937062, -0.21983905264251313, -0.2507979963547604, 0.43401646410517847, 0.3531934754296646, 0.5831001891120573, -0.15925571339350975, 0.6480301819265877, -0.5590615544093538, -0.5706944838802488, 0.4728240274432032, -0.9062711015695648, -0.866753030151775, -0.25958773996604934, 0.9182701583758714, 0.44180849735594063, -0.7313686802702941, 0.6926957467136197, 0.8931812884425157, 0.15360891598864468, -0.6093814153836805, 0.8396673673095612, -0.6120316456172925, -0.10305629952142814, -0.06579883717825297, -0.8514411527784349, -0.22140561181192853, 0.6520162911236345, -0.7682441451746045, 0.529043313462807, 0.8542274060827868, -0.04635843640142934, 0.37075631866900904, -0.07665257412846005, 0.6907411633409146, -0.34426848143047395, -0.013521047168294942, 0.9664170313340794, 0.03802646327921155, -0.7987276031423991, -0.27129629475688843, 0.07076185980652983, -0.16325100922902158, 0.9134585704250906, -0.31325246373466076, -0.45930170353638533, 0.6045192125898935, 0.6684189988156837, 0.6261659638151444, -0.8847380130696123, 0.5692540273104505, -0.1531896431925659, 0.7491683319496825, 0.31649498937127496, -0.1901508271746084, -0.1291992717030912, 0.38185358342556697, -0.008330393697321137, -0.5009786573958208, 0.07547701043248622, -0.24756370669485195, -0.2965054452535123, 0.8172802039499096, 0.20860716002420787, 0.9867317276634964, -0.3762905939424004, -0.7301327362171368, 0.9700197331593696, -0.060005500975445214, -0.6879250520908602, 0.7502169302963562, 0.9558794276689344, 0.10074427292397803, -0.8050390245726335, -0.055941118101787035, 0.3459673842633171, -0.43459723193106603, -0.9381937768305317, 0.6677823272994843, 0.43439858599766645, 0.171209782001555, 0.7338511292599008, 0.49591933681882283, -0.42882774577863825, 0.5076019666745242, 0.9040001411703165, -0.41935500338676657, -0.3723420250578744, -0.4743806445965417, -0.6452217064461003, 0.12447244869953811, -0.6166345124492993, 0.9902612203983074, -0.6895874038702818, -0.395405060214147, -0.6917991658595437, 0.2703515240178227, -0.04538240766690027, 0.39777566994745395, 0.007426250948370239, -0.9309217664616896, -0.5097504207266974, 0.9336502888993452, -0.4166460207835361, -0.03200406662691835, -0.6083466308928789, 0.04639300679926839, 0.6213570240268165, -0.39765547854171257, -0.326463228457639, -0.4071677823160764, -0.24109909443254773, -0.14093480939738057, 0.9758387147030267, -0.7308702298467535, -0.6020299715064308, 0.9941766320164622, 0.5580256245494617, -0.8960318795483506, -0.8867201838122714, 0.683979994595177, -0.8211429787666, -0.65120264299435, 0.32827811577923716, -0.5203514743906441, 0.19064031673005943, -0.8895714872594127, 0.6737987348405459, -0.8796390547682171, 0.9987614796835282, 0.12223249118339052, 0.7391725256248456, -0.31666055123633474, -0.24035218862654473, -0.7130695485664911, -0.20135792146458042, -0.36519598445724033, -0.5426697426932559, -0.8799943089151596, -0.4193652132028223, -0.9019196956205593, 0.5386595564996124, -0.190971892222759, 0.8014144773279157, 0.7380392995153466, -0.471223101475295, 0.5031179218376121, 0.7639114444843988, 0.991227581317143, 0.814941273437582, -0.21604148045186222, -0.13759216261944007, -0.15946327106841007, 0.08815644225907016, 0.5133106903652935, -0.45770794897795875, -0.31228877967753244, 0.7862396666878175, -0.08803731141047577, -0.6720436713706233, -0.21925802212003465, -0.6502828538463534, -0.2149559134830854, -0.35944235373485367, 0.8998456211132646, -0.06385404610345269, -0.8598863278543192, 0.8255077167048281, 0.5071334632115241, 0.8041634257667527, -0.6304740093200594, -0.23661337742491217, 0.8021102843872647, 0.836898779937324, 0.06416120277989545, -0.030668387720498735, -0.8375171245342163, 0.2139397708410804, -0.7264606236246605, 0.21368478598983454, -0.5939902638287218, -0.1478816533884515, 0.7209793213205438, 0.7841839616148214, 0.33433127976068455, -0.23362195304249211, 0.8157381503291152, 0.019918638690424872, -0.9240458971158467, -0.6351095664200175, -0.05794901851261813, -0.37049014131880065, 0.3578305747782511, -0.6202897759116683, 0.44229437295059215, 0.560025903100493, 0.4522163450020671, -0.27967033909015826, 0.7997261436391665, 0.6666094188895906, 0.988021158618962, 0.026454052772433734, 0.8940439291528102, -0.3595327627047924, -0.9105269300990841, 0.6900614758134471, 0.13728420413528997, -0.38866182842973807, -0.8220529672310477, 0.831532424690153, -0.7062867801843471, 0.7653458395118531, 0.626469385576369, -0.06893028832872106, 0.19447125966619838, -0.3772454488731496, 0.675436238145757, -0.21853697776040804, -0.908101720173917, -0.8896408179676387, -0.13651314888612687, 0.4446469521271883, -0.4071800971814481, 0.15043050795699764, 0.9401941886826679, -0.3484555417434454, 0.4025145492797373, 0.17824073791907113, -0.9973619384606354, -0.9538033519156017, -0.79829623245398, -0.4493268963968007, -0.47761859427361864, 0.2986837448150095, -0.08462335031247004, -0.7926164375514344, -0.9442770054280685, -0.4143729898416215, -0.36348045534343987, 0.1923060734132611, -0.769014047484756, -0.7726252432196343, 0.4742089638145217, -0.5637476705928068, -0.2598999649912641, -0.5494345201388853, 0.5815623347573711, 0.6254658669883388, -0.7907217041490993, 0.36491593829945623, 0.05654893722597287, -0.2223866672143444, -0.20769370411910115, 0.050733308110991304, -0.13817142565012341, 0.5048520555937228, -0.07440034520081484, -0.6089992320897799, -0.5715074002025935, -0.616954766099338, 0.6008107935933804, -0.507750505236237, -0.6256546709826496, 0.8315004979079774, -0.5227557885609775, -0.2026608197029116, 0.286987856599497, -0.18731192046531864, -0.3411220339487113, -0.7670878350973165, -0.9395368781179225, -0.30058382199509737, -0.6164228753574346, 0.621437394447306, -0.7635276714529589, 0.9381271103993143, -0.7768948943130545, 0.7900179562767085, -0.6146114342826354, 0.6145398137960969, 0.8274238130634228, -0.24433579595386634, 0.2817290112962041, 0.21575684634446302, -0.9161234575633144, -0.9292173759317617, -0.49446330665907734, 0.400264491155748, 0.10410196567720198, -0.9649666495178522, -0.8450466381928474, -0.7483072625127563, -0.31907540580712546, 0.9635590260302396, 0.9499785820478623, -0.5956415852630985, 0.21407554598043377, -0.10338324301599067, 0.47538086826037596, -0.6733650322194904, 0.8547186710502486, 0.3510388900959096, -0.9810008159980903, 0.25379228621399674, -0.20132283938024798, 0.1475678986500315, 0.06523640119047736, -0.40391719427503414, 0.7617107376273573, -0.33303490124055446, 0.2169015562640071, -0.8604102918896024, 0.6712000958383062, -0.9112074400792214, -0.1529889940159732, -0.588527572646482, -0.05642973338140078, 0.21570502795799262, -0.8226948549735009, -0.8790614794658449, -0.17957507602136347, -0.3816885266084151, 0.6354849236961109, 0.45580813630119676, 0.5684877892411577, 0.35830755015678406, 0.11955727921900317, -0.12256809056503282, -0.25203297850406803, 0.30263456707398695, -0.0032710802605375733, -0.5001527929487031, 0.6142907101976172, 0.36342451476755144, 0.6436809017745988, 0.3926001636945071, -0.5773383501509939, 0.528394470782009, -0.7884723549246127, 0.9599734096863444, 0.29400366999822136, -0.11234709904062723, -0.4454816515521751, -0.2387571058658935, -0.8874039345451865, -0.39747457380406725, 0.6999710564170938, -0.8070512495851263, 0.3423522450580274, -0.9926132764487288, -0.025144091734114138, 0.6910943200630881, -0.0028148122045390522, -0.7420264761749009, 0.3300321372665349, -0.48345815122329916, -0.9187412340270993, -0.5411698930826339, -0.5304603426780248, -0.25048920926777307, 0.9793094784288345, -0.0192300279882065, -0.7331661467458312, 0.08252747840071639, 0.13276156136985806, -0.5643483713067408, 0.5081900324341246, -0.8740352862637499, -0.9656201650251655, 0.44958842274208255, -0.6689488964187547, 0.15092805550729782, 0.17472166434264835, -0.7174841042825781, 0.4096848602493188, -0.04679362514437124, 0.4650850018069257, 0.24754259083580266, 0.1511273104249331, -0.8207189095320673, 0.8607306198158506, 0.927703545656464, -0.08969149005731292, 0.9732247362025428, -0.30950220344442925, 0.13325300035151688, -0.9791490329099752, 0.9961639147323051, 0.9671778496586592, -0.678165013065565, -0.5252355273539204, -0.5906890522461736, -0.25637261830033786, -0.088981891467379, 0.8370067775341856, -0.5561691278416616, 0.4937632579571922, 0.7608286591306002, -0.628592046091131, -0.7899014523568284, 0.6798205036365761, 0.015432511097690726, 0.3574808047624536, -0.8324671983498069, -0.14056336321052254, 0.18412824533193795, 0.6138063144863688, -0.5471130677039211, 0.4285658491880242, 0.9133187405057497, -0.9743810179000658, -0.6210169778443362, -0.24406022614080558, 0.1241455935790261, 0.8549740588840731, -0.4362013066011381, 0.6876215308309608, 0.36822142131824775, 0.03469423612523426, -0.8428183242504323, 0.32300561365572, 0.43312867791154464, -0.10835849327028857, 0.8596191509977236, -0.7847552417557284, 0.9657004726373735, -0.9820928827937494, 0.001618858050356442, -0.2736075645377769, -0.30954240878522454, -0.14443307535834338, 0.6310525916928056, -0.04405656791384094, -0.21320705039705223, -0.9751000598417492, -0.8477512352720065, -0.8076784562775878, 0.34243765787590275, -0.6067814109133589, -0.8557609537576791, -0.45050873005396164, -0.29417976071486707, 0.24074616034175644, -0.9615094823654018, -0.8914219145445557, 0.9771828824261553, -0.8770257033985995, -0.5735727035924887, -0.244558034109015, 0.7977428391368138, 0.9078017904427662, -0.5056330338906485, 0.8149302481765088, 0.8990688359067311, -0.6920818508228457, 0.07121815818797428, 0.6768371094502172, -0.13580917777521373, 0.7061907058944366, 0.8507384252341144, -0.5926849403148409, -0.15291108676136123, -0.7666631618513406, 0.9876612794242228, 0.07424511649168197, 0.557399374807305, 0.47409385776506685, 0.8793929721852507, -0.8622073775834356, 0.9820245253658795, -0.6848891681698996, -0.7989377395275292, 0.4638867711317509, 0.9320347465703078, 0.16856564244542538, 0.7983180748439183, 0.8722551129163922, -0.5490118503573593, 0.5948812642425909, 0.5507799208055022, 0.055659442143183746, 0.07067011186407779, -0.714533914993535, 0.25090448233293094, -0.02091018594987215, 0.9152000656458112, -0.6804334315934861, 0.8868929556733554, 0.33062153223697344, -0.7451286860065522, -0.18524780633216342, -0.15312995971746246, 0.10823825223067773, -0.8917214997837619, -0.591480971304444, -0.6382494945854862, -0.3148941740012843, -0.7655840627334594, 0.584945611780257, 0.3088389490445944, -0.47247673124721223, -0.04340424990019276, 0.3051500397301703, -0.09509017761183647, 0.7094037530451527, -0.9959303023420949, -0.07939990262418761, 0.22508809145967734, -0.22519875315544446, -0.5768280802134824, -0.11177083452471215, 0.48127071025866197, 0.5585797782950896, -0.6622005760713117, -0.9796468471318027, -0.7225955520966929, -0.909496017311539, 0.7983511224451865, -0.6734404497478614, 0.1087287775336716, 0.3357892560114577, 0.8659301801445032, 0.5278211127562835, -0.1895183051900149, 0.8263779095316983, 0.836477831756125, -0.13340416123018461, 0.8802633041706416, 0.122589269244048, 0.4120611612417071, -0.7209200794789132, 0.22730941597318743, -0.32542475718778285, 0.7486388426112447, 0.8013091836475565, 0.48361778115712184, 0.6692435776286738, 0.7065582303696512, 0.6270289668776694, -0.5790803377646956, 0.40618368771010327, 0.26729810471545945, 0.674667452740642, -0.15532749816119074, 0.5561986347981795, -0.032257838247754966, 0.6971554261765125, -0.11827484257893528, -0.4667587622669387, -0.6265092691711691, 0.0005736501824928908, -0.9608040061700782, 0.18734931359146456, 0.8080841890871766, -0.6955591810296442, 0.39166469304856766, -0.9881415663015016, -0.06368049026019929, 0.35481460717767677, 0.5765900204068612, -0.8802489566472993, -0.4565949825626676, 0.8293875367737809, -0.8180364303652257, 0.4965487650571285, 0.5058200045673109, -0.8576485142792758, 0.34107444185118063, 0.4928637682154824, -0.6681531920755042, 0.32916473595201623, -0.6062627780903693, -0.851556315391103, 0.2755198186502805, 0.9245682419219381, -0.885131429122678, -0.4846270164569302, -0.9461279340947608, 0.45091551847810285, 0.4015776669383726, 0.19592979732980598, 0.6111014981444214, 0.8220186493411117, -0.44675082384124076, 0.17132430431398227, -0.9130631362682329, 0.47605433725563673, 0.6847479253751789, 0.21695724519505322, -0.8105445197143275, 0.8141019657476476, -0.6725870195558141, -0.7919583246610638, -0.1364974212052037, 0.9304326635625573, -0.8760073709775771, -0.3758511428508622, -0.0972873623654229, 0.8055913064615987, 0.03880849458566682, -0.07718448367659025, -0.41579661232173915, -0.05011336695055313, -0.0900446837316431, -0.8507598465518598, -0.37949760519748743, -0.7469522447158996, -0.5233918456311826, 0.2937002538610698, 0.14076015949327592, -0.46936154650307516, 0.23617578575837972, -0.6780604365897902, 0.7449562352579262, 0.19309831030127023, 0.7582894922998955, 0.8137730567810408, -0.19600089455174796, -0.9107512806508775, -0.9971876555116035, 0.10052767528538964, -0.44950899198753125, 0.5549659620302849, 0.19535255449184086, 0.377922582708631, -0.8647320721404086, 0.6543071087716881, 0.098672703960341, 0.7318341867250857, -0.551958398629067, 0.348988980560615, -0.17145856096546086, 0.9957136673624019, 0.08829935403067135, 0.7639025619627351, -0.23731764644665287, -0.14387093924019578, -0.8534556383929455, 0.8323958591709943, 0.3331602759588479, 0.7984333382679099, -0.4985492247678869, 0.9709377166738384, -0.49463266863761324, 0.10898559899395477, 0.9589901811112869, -0.7182725935144256, -0.011552306202743834, -0.10856361903695544, 0.34491218037793314, 0.5635255535585746, -0.8477177421156437, 0.10914515296207772, -0.33334778699299106, -0.582557602508136, -0.45940809656532333, -0.2716152021139817, -0.21606039641551855, 0.9361404886432887, 0.08984483252186948, 0.17205495874181875, 0.5207553140022017, 0.2240764255036387, 0.33089807474608257, -0.4929769723441504, -0.5538916934413214, 0.7862279349557846, -0.32816660124577446, 0.9028182642980918, 0.34459389915855176, -0.6919394919558863, -0.3400965301083605, 0.17326245197005719, 0.5818316854065699, 0.40658639887248893, 0.8507038412531216, 0.35445672034542586, 0.41707691070883435, -0.08856459121292759, 0.14373063375866368, 0.8388566127494321, 0.0146135343251097, 0.04371966327542287, 0.3032283352126728, -0.16008615538072068, -0.6487508415493055, 0.5555652109550724, 0.5270465486660334, -0.13451978290073008, -0.7916270795806339, -0.5820248500389584, -0.9422727911461652, -0.8958688345650765, -0.7341317613510303, 0.6115931339674028, 0.7388924836135629, -0.34229123623921476, -0.7387492821058768, 0.14145414945728274, 0.9747736433500862, 0.2629708662845074, 0.34228407748351275, -0.5696859024499383, -0.3959683237645817, 0.1952027901512816, -0.05798632673709747, 0.7770330287884399, 0.1639199906121429, 0.6565297270488475, -0.0904975828750072, 0.42272902960560255, 0.45587020577374027, 0.5261242507089983, -0.8139194431936478, 0.8301840289095308, 0.6316218975552395, -0.137705546069413, 0.264583787465295, 0.412303096103978, -0.04668613660750953, 0.9894355436249589, -0.3155725540327936, 0.24958476480950487, 0.4575137733869479, -0.9337599435901942, -0.8310854508526884, -0.13301888493929725, -0.8264562827630262, 0.8802633025387969, -0.5346645108620747, -0.09098231981603311, 0.6655919989491104, -0.5633593197960658, 0.43195756392201456, 0.6310817629882712, -0.15504350310861525, -0.31828101882631543, -0.09139197847343294, 0.21032395408785587, -0.6073380185478987, -0.1702948425589117, 0.11843743939626239, 0.5003800879256912, -0.5022414686523724, 0.9169849223277413, 0.5518288801720688, -0.6351496323042305, -0.576300396735727, 0.20114838334590313, -0.6601581235245211, 0.016273405643804972, -0.5292169191801799, 0.036947060357659156, 0.7443151726671018, -0.25223346280377057, -0.912251245130373, 0.34833421263640063, -0.9812419054387542, 0.5037312702128065, -0.3694541435418037, -0.048752086771304715, -0.005633609371476833, 0.43935530136765055, 0.7459527863606525, -0.10022434154611237, 0.5565991559222285, 0.12532486752966343, 0.10277115592307418, -0.0035057129476727678, -0.5317766070404175, -0.7506473834350156, -0.5866379338203274, 0.8412766984299123, -0.9509044461490124, 0.0914105332799724, -0.4916355487186159, 0.7968479635333254, -0.9397153332191674, 0.18178610790737326, -0.7896968554278494, 0.2096123702921482, -0.692299877752077, 0.06675139794322216, 0.31960153107735034, -0.014917383580957289, 0.7933843858990115, -0.46394524162452955, -0.21403383543918664, -0.544876661200963, -0.2738906233190481, -0.4168294972427582, -0.786789561890411, -0.38987794096701656, 0.6110078809002462, -0.4383659237428319, -0.8215725384119243, -0.22770901008806743, 0.9922587663021456, 0.42541080696846456, 0.8423949628893177, 0.34845489811916064, 0.19917454981737093, 0.8414695902712355, 0.8582011148718964, -0.528280626534603, -0.7079573641203529, 0.619381902663193, -0.30838759442688746, -0.9445387444626017, -0.3203161329550428, 0.6415137072442687, 0.5146044423819558, -0.18178180894229512, 0.0865470848964065, 0.0820003508941991, 0.6549307681768453, -0.5196349773275364, 0.11041242055475076, -0.4662068859181021, -0.11425780481632275, 0.5834158225394759, -0.07404873569595805, -0.8328504554296174, 0.9783696616388156, 0.12172203994821351, -0.26782513235360095, -0.1481265856120575, -0.8723618305298722, 0.18035686460714806, -0.11486468216029522, -0.8262107655731372, -0.7636439750278872, 0.22333040168833684, -0.2042921325405267, -0.1120834080023847, -0.10944109218550979, -0.21025651680608726, -0.3170099521543557, -0.7389832175877971, 0.9801430263192479, -0.6234227895145754, -0.8965365421777354, -0.7667685675289493, -0.27272806009401873, 0.8061857484639023, -0.2896425094806625, -0.9072721593762214, -0.018221213749971765, 0.8214634763872268, -0.14709631284654323, 0.5776136951907465, 0.4747647809510045, 0.8302667431044948, 0.8920019504128118, 0.5061845515168604, 0.5635738465985074, -0.22582016687115747, 0.7912182349916534, -0.8408506575845642, -0.9300353604808, 0.08793021160988168, -0.4281869715586144, 0.8784860697094201, 0.9201577066045605, 0.09488294905890649, -0.2724799775571798, 0.768714747379575, -0.4955189387405916, 0.9107250848424431, -0.5527122596722855, 0.3452615150354439, 0.18271779676701327, 0.9382421337282871, -0.3800694733714811, -0.23817922562303195, -0.07688203881619282, -0.18869242652575546, -0.7590959599402796, 0.9757299099680998, -0.8303419337736992, -0.6263425590044032, -0.42140355016169173, -0.7688024585031135, -0.19985260403305727, 0.6093175308829979, -0.2209008335295819, 0.7264012920704437, 0.4447764268989516, -0.2860516607972172, 0.21244845087752995, 0.8558456716849803, 0.8964422401673238, -0.05593022927517821, -0.9597661723402398, 0.17944571035770585, 0.48179239190860024, -0.7144824540652408, 0.34826535466766173, -0.8427721198584701, -0.8641697980000513, -0.4355292952070957, -0.35453874352842973, -0.38730000565449574, 0.842897428740156, -0.9709765263524697, 0.08019185465130341, 0.8082602462179946, 0.5498632906194072, -0.5331062081134692, 0.927638367891799, -0.5692529540620872, 0.24982369863473264, 0.06898436483530057, 0.8896397772677951, 0.3818667978781398, -0.19250737553716335, -0.25994759875828977, -0.3847184459626545, 0.9640959787844998, 0.19135725344986398, 0.9710333337886226, -0.683677312522561, 0.7643207452343455, -0.9195311848949153, -0.7475391367441167, 0.9118303626162578, 0.6948521845551001, 0.28864470711382983, -0.208270829951668, -0.05742848853856142, -0.7851742511070752, 0.3960902490888978, -0.9684775653347413, -0.8461090377986706, -0.48971050873869504, -0.022531874989190293, 0.0527574609582937, -0.8408879570229097, 0.46395865395611047, -0.2568658936117514, 0.30065406644675363, 0.3477756866244075, 0.47554900937198297, 0.4156468474388062, -0.8432942053904429, 0.7007413781688372, -0.44233123638480465, -0.9942421245583914, 0.11201255186534231, 0.8331714015634053, -0.7153675200677974, -0.8374913127174619, 0.6328580767428913, 0.3710984961800132, 0.0851188050293743, -0.607969590438213, 0.5921946943410912, -0.9335581737946415, 0.003171586344274724, 0.9370951234079514, 0.4676494466769392, -0.9344758056652334, 0.16535731127800934, -0.8632163794423082, -0.1638039868803729, -0.4808648011001482, 0.46715050798123436, -0.1734130615525047, -0.19844323018326304, -0.31083029788938843, -0.6131171864218881, 0.14757040827727197, -0.9790139501084083, 0.23372706761683526, 0.4111053645080216, -0.9456430583916884, 0.12034072990131328, 0.3115530712449486, -0.9373214526069915, -0.6600363268798797, -0.23414856059271583, -0.6460350022376171, 0.6083700079572578, 0.22968060742835772, 0.7915692164427741, -0.9154017674661317, -0.247148725653477, -0.5909809932878278, -0.25718879942331063, 0.7934300241437371, -0.10711991747235383, -0.06414119061018342, 0.17303216168800284, -0.9068996188039633, -0.49352815326908095, 0.09177892952766031, 0.6064006350527447, 0.9645098571375348, 0.4609939312082454, 0.08179688070235058, -0.5973399826852928, 0.8718903710006343, -0.259705169830837, -0.20472243532904866, -0.01826564657987606, 0.08725660445547967, -0.22252985035303263, 0.8670375411118818, 0.19477726898965542, 0.4815472838644994, -0.43406660911462946, 0.6511249218493731, -0.26448005107773387, -0.3526228523281638, -0.2832917578844969, 0.7691666661527661, 0.2830007428886476, -0.5112216517697037, 0.44916689748476135, 0.3835586535142439, 0.2161089686996045, -0.042691052027009846, 0.8807506143671398, -0.5726477964120789, 0.9902876915162138, -0.2378966867329173, 0.583600241192044, -0.45870671317327605, -0.20574022880997367, -0.5021924255933687, -0.16186778288257564, -0.9608741858191971, -0.9011818440985846, -0.5771936899435874, -0.16837376277385485, 0.8228762328457304, -0.8546521635284012, -0.4908868806818014, -0.26929157195264053, -0.512759957248003, -0.5602494252764081, -0.17876385916396687, 0.24736707657128965, -0.5735081361731171, 0.1788955810495103, -0.9579454152635798, -0.8265132998275879, -0.17354629794779775, 0.044760060308953964, -0.02964479460303937, 0.8001701485877164, 0.7736445755287178, 0.43592783596939055, 0.15950916886119648, 0.3430427425897056, 0.5551122702463862, 0.3281536709264923, -0.6802338608411849, 0.5290969365206006, 0.28040043082794197, -0.048303357450902196, 0.5535825637713452, 0.7537076087097994, -0.7353976783744953, -0.6867849055034971, -0.9608366729943212, -0.5653945901088104, -0.45226938982763176, 0.7054120371829748, -0.136819677675039, 0.12028800951747853, -0.9921203776512599, -0.0020533727190885642, -0.5616220598038149, -0.628098217831103, 0.13616210306279553, -0.8238006899725676, 0.6817936340167952, 0.018133003219115817, -0.3931435756658679, 0.6745904903654616, -0.2705269153552943, 0.23498805254728738, 0.0895864667166204, -0.5177652445653671, -0.43780063312248974, -0.6530256275003967, 0.16760075137957786, 0.8554601212676407, 0.2343435807564338, -0.9984609774222446, 0.6905832036542823, 0.9730520816579862, 0.9688822342724781, 0.17890335403328295, 0.8858432543115138, -0.7399174255186767, -0.8752511941824084, 0.28896738741243566, -0.4175431048887237, 0.13334618548757926, 0.2872967148038772, 0.5364859086654901, -0.017538146797164478, 0.9340518400554645, 0.5404103591458775, 0.7834485867098651, 0.31386646099261717, -0.9086641553162138, 0.7889202401819873, -0.2929317976362733, -0.3711518719121203, -0.41643076339763385, 0.928666184633711, -0.9369395305148847, 0.4710965264739837, -0.3761005083750917, 0.48916010796608456, 0.15550962980251803, 0.6475764341324697, 0.5625269668381747, 0.27395053157520355, 0.368075964146118, 0.5052893889584815, -0.6187633247491191, -0.9902707236228216, 0.3062072412512955, 0.43948378049264036, -0.8478473436097247, 0.9511601442256472, 0.3980388770811303, 0.296135231597926, -0.8785277964292859, -0.8173481768127555, 0.20837358207094625, 0.9913406253312338, -0.49881068118048844, 0.328932241543066, 0.09109802491881247, 0.699075252369664, -0.6994794537045848, 0.7267466205272837, 0.31848860458125605, -0.5173305710770595, -0.36382337594860115, -0.5475923059867527, -0.1659027850723982, 0.8417331584260008, -0.2870050502856305, -0.5235718203305353, -0.5421959634166122, 0.7219560611017914, 0.058710836587239745, 0.18592757480363686, -0.08553093064090245, -0.18242015163732983, 0.8281881203479202, 0.9096863085190867, 0.15300336684196725, 0.720964180946527, 0.10396449678969755, -0.54750843113787, 0.27309715509087584, -0.6617247339632193, 0.07319255662257218, -0.4944404551385355, -0.25970979865878707, -0.19476263293390517, 0.7313744297759404, 0.9153077665933418, -0.5412669293953201, 0.47781578687982584, 0.09686618053390195, 0.3838565382885275, 0.567628282416913, -0.42900543569016003, 0.6535596727493442, 0.6207473172830449, 0.07155605852841518, 0.4028043017138454, -0.7173519209806813, -0.493433811846705, 0.9597939109935221, 0.31451758004159647, -0.7616740752765254, -0.8648545640275445, -0.311153387004532, -0.295580413719845, -0.10236104821384995, -0.27179152636519444, 0.46208348144694367, 0.18389483580247967, -0.8929931785563052, 0.5392492952548782, 0.758350664760983, -0.4234602845724138, 0.14931763225919648, -0.5714752536963497, 0.7522350228063761, -0.5453869962399196, 0.9847870467845934, 0.5100464584544098, 0.4429009343592323, 0.9633064983803681, 0.3189399588768729, 0.032235900616987445, 0.42544421698658774, -0.288496925149049, 0.9437842496050612, -0.21681206717236323, 0.8578436925993822, -0.060536861658692764, 0.22626466286130476, 0.006363579090769678, 0.35536534307303413, -0.7136482647757851, 0.8193256183642241, -0.1976504722940584, -0.6095061974307932, 0.5021026604364489, -0.4211761592278507, 0.1299013621990479, -0.15134873542110427, 0.2308723141511071, 0.5681714959785864, 0.6292929639699272, -0.8438187678142475, 0.7242917956095265, 0.859112236404344, -0.3697632209213735, 0.4374185490329989, -0.9150967416148945, -0.7613205407249151, -0.6015584099820539, -0.31431844628896743, 0.4419069926994421, -0.33601954676709567, 0.5566778316388825, -0.6161497523699491, 0.746743336299811, -0.2184651371382933, -0.2573179641018033, 0.41853554894526845, 0.028856998633393083, 0.35809221277791115, -0.3473002544406001, -0.19829039473974852, -0.07293752478229076, 0.07549998174832462, -0.8964820625788161, -0.5292395527459837, -0.981911976195734, -0.6630288148585306, 0.5958503285791517, 0.6774761617955236, 0.9541028257456596, 0.8600191010507703, -0.3659279428920559, -0.16462981174574254, -0.8611409210822347, -0.7616122797720666, -0.37782006528833234, 0.30326831789803865, 0.6121798787255868, -0.7726756624920013, -0.6797800401555347, -0.38397973351200343, 0.29673088662909874, 0.33698154748413844, 0.44336409183225434, 0.7551390440783559, 0.6231907132967018, 0.8755631545481743, 0.031345873181240824, 0.2857360720896007, -0.6335448099766936, 0.6951003711098085, -0.5804383138924223, -0.30084235781554636, -0.6150087038778007, -0.47800835386627694, 0.8373644933360083, -0.4007538592377753, 0.6845611918746948, -0.21421027184534136, 0.5467456760403246, 0.012361575030900518, -0.20379248446318843, 0.6043014592021922, -0.2874967788562237, -0.04489532697904308, -0.11173068313034218, 0.9718800466215645, -0.7588873428471081, 0.00470097924959223, -0.5919627100659806, 0.8067579451484499, -0.9121656400311475, -0.7068751795970747, -0.7625874369020404, 0.11348310701804332, 0.7742670097264701, 0.49260337786417985, 0.11272934529192002, -0.9339731523615982, -0.7908886970008495, -0.1889190118958608, -0.31201562699353125, -0.3351759942571857, -0.4358333402468755, -0.664224224985988, 0.9496579933767508, -0.7576825967142478, 0.8132636851192514, 0.6211951159421252, -0.18180490471514865, 0.09864365844779299, 0.9446832104923806, 0.4100735245868601, -0.46116688805315653, 0.3918665255787528, -0.6222119052947226, -0.2415404542974111, 0.7876208521598298, 0.6489650525712551, -0.47722242550941973, 0.7072205063938211, 0.5264245251559754, 0.6727997980022129, -0.41244203589736994, -0.5025174991608847, 0.401265505784383, 0.5098270993835179, 0.6219996928423481, -0.581731161085286, 0.22518409632054692, -0.06030675598273949, -0.25915040227553265, -0.9435534018034673, 0.18218699249574333, 0.43556655655392285, -0.029872933419591075, -0.6388978677486314, -0.012288681446297245, 0.5274609898208154, -0.9647821271190902, -0.3940710695817826, 0.41639573297770904, -0.39996879696428267, 0.7858136196729946, 0.388929533748986, 0.7463419935353746, 0.9860729380662374, 0.029471598435457214, -0.006706945442678691, -0.43582034783688406, -0.5765094521467276, -0.0573558431828316, -0.8204180891214194, -0.33507097632560967, -0.038819778558819795, 0.6872808304594433, 0.9238301687186672, -0.4139323806930455, -0.8860405248322465, 0.9845980622513058, -0.23189266929508046, -0.3015404556622394, -0.37234183609555527, -0.9998229223766617, 0.6892700848793489, 0.5653237031683258, 0.2890672075584291, -0.5792043640615889, 0.753542881387574, 0.856567341799723, 0.9145208641105667, 0.6729969677240268, -0.8539810803165493, 0.7592887815101084, -0.9449244539984578, 0.18448077005115482, 0.7045119832911182, -0.8802215390672135, -0.7291058125549179, -0.9615297830652887, -0.10627821648156965, 0.9684850706059482, 0.05594192118514352, 0.9434349991111497, 0.2935662576233091, -0.6579970754842956, -0.8376311406836787, -0.47981473490188065, 0.21951194355886972, -0.07823543803135413, -0.748940901110658, 0.6811368509045033, 0.9558265098237295, -0.6169519942845414, 0.4987847647518886, 0.17855686128677073, -0.4241685855391304, -0.4036089222948138, 0.2525288148438225, 0.869552289467739, 0.07440944164684815, 0.878830839501697, -0.06908595066692169, -0.6486897183619842, 0.275519688411624, 0.002435213020015148, -0.005289747807291745, -0.7094428966607595, -0.5795453916979485, 0.6004400197343094, 0.05201632648950527, -0.8357501126735583, -0.9645437597142843, -0.473917924887161, -0.7537249461840838, -0.6381870358714674, 0.48899777597753347, 0.2698292292007465, -0.038741413555667625, 0.8986680133910083, 0.0705211412586686, -0.4625094892892627, -0.714525536855072, 0.17332066393719892, -0.07531547439140862, -0.2826560287124462, 0.9254408606977378, -0.812033009297539, 0.572212627433828, 0.09651860441878268, -0.2395925911591752, -0.8813609075882678, -0.7433645857374016, -0.4883215048835243, -0.9585947621222071, -0.24994277117005015, -0.8633073484546918, 0.8072012106398789, 0.38558175104440795, -0.054760821006101024, -0.0890937225322801, -0.7850162822067965, 0.6416817196646489, 0.27835776077794305, 0.29360946628723017, -0.14202579173617047, -0.13467293884798504, 0.999152704474813, 0.8165711418402297, -0.4060122415841345, 0.2782681398790863, 0.24199054640759754, -0.7948867260027948, 0.3068538768315827, -0.13810769122173716, 0.6462468356947424, 0.21294307328556905, -0.26678620176563794, 0.17510201252204594, -0.516076023747982, -0.1330738807611409, 0.195563294346496, 0.7466625987212252, -0.37348416286806785, -0.27473100500789904, 0.43154266448440004, 0.053710232065685704, -0.8846725472498609, 0.4283383136292451, 0.6074059862056811, -0.8534639590722042, -0.19834056312000814, 0.11461683901593411, 0.5630939074561203, 0.8578490141394659, -0.47481904118278484, -0.5889335303425702, -0.34919341896589184, -0.634261770008961, -0.18281072613701754, -0.585386227729465, -0.6382922353007923, 0.8373146637839948, 0.09730915655536032, -0.005328860637740185, 0.16396787838113114, 0.7801640745643639, -0.755435108960538, -0.04189688620205345, 0.22141915013720292, -0.9233485759213793, 0.3695967348593776, -0.49219531774115066, -0.33755689194168736, 0.6859160935654165, -0.26410289789166463, -0.0555876770302981, 0.07661763772560426, 0.061364060589531366, -0.3368143312560159, -0.1645595393893451, -0.3246997474922755, -0.7502720738973105, -0.4459774721653349, -0.1209477406534416, 0.6306934240567095, -0.5793963417630801, 0.9734528251261918, -0.28385550465004106, -0.027210123235271277, 0.8181514215552295, -0.7545357930169303, 0.01591386036810505, 0.10306964489020665, -0.5380819368569141, 0.42284664480762313, 0.349848690132776, 0.7339162763889282, 0.6180361431977854, 0.2878878037838528, 0.3753939767300698, -0.2199339893527612, 0.8577923370907148, -0.6680293466759857, -0.05934336369601412, 0.540717834363116, 0.1179619199878228, -0.296606604992528, 0.09385579095341967, 0.0418787230201223, -0.4782322957415448, -0.004362500049954976, 0.3202468282269033, -0.6539909514087054, 0.10339231020162076, -0.5423154673289852, -0.6083890509094698, 0.47847655752571283, 0.07482015122783481, 0.05385597627488692, 0.5288604608917347, -0.8647100063398963, 0.41898554940879773, 0.9976427499810663, 0.8573242595746469, 0.40044414763993275, 0.502443016673146, -0.2764974226817867, -0.04614222796818401, -0.3085782411108251, 0.578103723865623, 0.6873846832361139, 0.6620310304205306, 0.1823788644271398, 0.007508866349905441, 0.24350876738661742, -0.7000140066741638, -0.5595855914946839, 0.6446996556159832, -0.43282289429409704, 0.12445672453926182, -0.25764310388239875, -0.995401755129641, -0.39207778262774284, 0.9158860734897476, -0.23322423702386108, -0.7788373112884472, -0.8475423394058925, 0.9960229079318144, -0.4720367897902433, 0.9627169065483674, 0.05376117924760293, 0.4753011547955317, 0.6034006490589809, 0.08351878637367438, 0.9244511627815006, -0.9670880113282383, -0.005977261773195908, -0.9405051106880871, 0.25468378732523234, 0.4894104322712165, -0.2495601933151821, -0.3115809216527272, -0.811146213262635, -0.8316286634862449, 0.20830712338770696, 0.25094281802354357, 0.7882196075444556, -0.55829044409708, 0.7365378967862506, 0.8582538894167253, -0.6287106589108673, 0.26089794962923385, 0.2312463405466132, 0.5412115971405473, 0.5801528011033326, -0.6409841882201306, -0.44252395522685495, -0.821767611287112, 0.9235246143885665, 0.9325669596010424, -0.5202058673338799, -0.6802303486939612, -0.09426742414532518, -0.5960213483586656, 0.6458520990345475, 0.10075361732054833, 0.3845558665333222, 0.5742530203923495, -0.47996664022909075, 0.36565639601009137, 0.9265845704905631, 0.1950485640124906, 0.20376624959771084, 0.6398783262408243, -0.4894663705324145, -0.09531604029890994, -0.0721078836001745, 0.6285670793079638, -0.7750212664314726, -0.31446657278641466, -0.31023955359205546, -0.07422628110173712, 0.07333624175292996, 0.6136453846221159, 0.9885871988214663, 0.860251358483352, -0.3785734950026616, -0.23020511224970908, -0.8442702319635107, -0.07421253443268028, 0.13892006102068177, 0.9690421679684491, 0.7606375375036785, 0.3506340527789473, -0.4960886720771045, 0.1433696144974077, -0.5730270331940428, -0.5213808172683931, 0.18635255450328514, -0.48883175676734436, 0.5365904307565292, 0.636451338952893, -0.37992715885571804, 0.5001782818422364, -0.014761809035823026, 0.9845588432630485, 0.843687382972369, 0.7529843199076403, -0.24235877806544148, -0.6629449537970655, -0.31595775037719864, -0.6498415399425552, 0.38662639144371314, 0.5523182087923479, -0.48435750313594106, -0.4523926753078853, 0.05698613521149731, 0.14164378069279704, -0.8864971630879221, 0.5682231672388489, -0.9249783461255541, 0.6017105007797456, -0.8004104330080901, 0.468598904656917, 0.020570888124127773, 0.3428986012683213, -0.6259772992809991, -0.5262606586500018, 0.2723975135954657, -0.39173144559502004, 0.5127131284460529, -0.4638322938475745, -0.041633539405758535, -0.5062174450308512, 0.3766837199583264, 0.6402810114309336, -0.44716714921332446, -0.8523253123705157, -0.5936970480180612, 0.7461064990069672, 0.8903986254836507, -0.04055639031980074, -0.7449255435357487, -0.14257066952967978, -0.4879922623479829, 0.08910106247485139, -0.5364654957980128, -0.8167480424890567, 0.9166885451496352, -0.5308123006372938, 0.4434884924059104, 0.8052720532115856, 0.8526870564695088, -0.8525508950477438, 0.5595019630400206, 0.5741252803661099, 0.6181299499579014, 0.39527821944069363, 0.7444490802302759, -0.2542207621785173, 0.6168699088696756, 0.3301880379987703, -0.7803954902465102, 0.7731213197603051, -0.19253026824366715, -0.38290099327695337, -0.8586120604163712, 0.42861362886573295, -0.7406225772809283, -0.6835444738175731, -0.635845453555685, 0.915160257355373, -0.6339872739556256, -0.2733177623553258, -0.5059183632929676, -0.8770540653067467, 0.46884959782177815, -0.09854820724850533, 0.7696668753195681, 0.15606924318958582, 0.08612373105021476, 0.39262768506721524, 0.3191113165441468, -0.464522665580859, 0.8027236249497673, -0.775332288103685, -0.45222077155051776, -0.3145321411592914, -0.08369072003917943, -0.5397776413802613, 0.6787685121106557, -0.8465399474091762, -0.1175537614066744, 0.4035429095314842, 0.6343028569534872, -0.48963706400065243, 0.7281707026317021, -0.3771439240935619, -0.7839730649075269, 0.9969456727201602, -0.7941390908802859, -0.19740278213460893, -0.561786921097758, 0.3672628747496556, 0.8297651875499639, 0.2145368475594398, -0.25480947476476823, -0.5759653350006284, -0.43815271757354757, 0.9340330283307852, 0.8574432778587187, -0.44333286920350923, -0.7737293139824502, -0.05401286050112808, -0.6277290121989247, -0.19542776179615418, -0.533602533748923, 0.1384183785319728, -0.7061565336609934, 0.9411077759093331, 0.6199476021445138, 0.03588132169571123, -0.5359361513201029, 0.5918966992772587, 0.3773941421682221, -0.6899321883794665, -0.08337903640418043, -0.45804934086943616, 0.8909046636629985, -0.21356089526596778, -0.3368366281235675, -0.4079463551359208, 0.7293260860788124, 0.8567261067658309, 0.4921429056196047, -0.33512295773648715, 0.42191112850892853, 0.2950651493158185, -0.06305302607924257, 0.9956744252806302, 0.08514746174141696, 0.11529822548298752, -0.7066954475907805, -0.6835724069553906, -0.7232429158954738, 0.34115817903190715, -0.46120249447579464, -0.8395679674138934, -0.2375422932970308, 0.5466261237097376, -0.5272384014970957, -0.4143994627970755, -0.9512438737945212, -0.4576945224119462, 0.6322359968085574, -0.2968805692534582, -0.13431986647179261, 0.5538700393327218, -0.6453614624071251, -0.05067163189237012, -0.5760236480702563, -0.29282291807541383, -0.31640832550686704, -0.07280419388135528, 0.36706880531420283, 0.2937503771344676, -0.8016500886589104, -0.04434888062994724, -0.7975619114105403, -0.8836712041077108, 0.9924838454333849, 0.5907615269573012, -0.5810562603389942, -0.5067762468908525, 0.7407745472368816, -0.3651704750622613, -0.39089069822288436, -0.6636456990780544, -0.1017887788189138, 0.4209854646015503, -0.3647072456999867, -0.28796985677285414, -0.02765390930131706, -0.3768761637353988, -0.5312535934264988, -0.45122905013441805, -0.13786945817152252, 0.820948474504684, 0.058398206825179155, -0.03900075067086295, -0.07343622854530563, -0.49745336798485296, 0.5321036949429099, 0.5039347660475317, -0.93879346212942, -0.1294383943763231, -0.7896858730674714, -0.631454264529191, 0.043798856632176975, 0.9287042124173064, -0.5254114092869242, -0.9304803442660086, 0.8952720249037796, 0.4315447680465134, -0.7194865405544308, -0.6908731107069566, -0.6226872707530304, -0.817518812644519, 0.18761883600493134, -0.6779949223747144, 0.7750009235108075, 0.10989606628633908, -0.10421654041335704, -0.8605225342981935, 0.2345055345251621, -0.4596500782932478, -0.9785928597676059, 0.48012162540778736, -0.4085988749612144, 0.8711161756232384, -0.5566146018501974, -0.7212501194371885, 0.5472771604828164, 0.4106630470607284, -0.7772955675411284, 0.8004515540319754, 0.9204337891506218, 0.28061874229031636, 0.8247616552323009, 0.19232543158719428, 0.8908986060488053, 0.4906122224866172, -0.6149843359349514, 0.19638376907899402, 0.5377211297684799, 0.9758248702785408, -0.11067802564318474, 0.9989449992567625, -0.9203053448439922, 0.16571383740624412, -0.11762861643373634, -0.7373845386359756, 0.4168314359399361, 0.11526857585764749, 0.1279141019054011, -0.25715593433825923, 0.33522257334012395, -0.4800799427640441, -0.28094595344132856, 0.3253608349965118, -0.6934718198520551, -0.4845998206901039, -0.8607878096379677, 0.18146021514005484, 0.24139347269633493, -0.9328900087143368, -0.3358721069957946, 0.19861330629212914, 0.7688094023183514, 0.7113882212684404, -0.3924048142674552, -0.1403201929171345, -0.5202379161649677, 0.5932157298981062, -0.3660592587607445, -0.901726018653628, -0.3030354116090366, 0.1423264017120882, -0.25725145510664715, 0.8261143053161157, 0.7726751810554517, -0.7712012598914602, -0.18583725426294895, 0.43012069961195376, 0.5881246133749294, -0.45922690335887806, 0.8934880821247542, -0.5839211449509818, 0.14579255391672885, 0.6647696152882763, -0.6685754333240266, -0.20247855282391214, 0.65772667838115, -0.02880127673362831, -0.5485800129815321, -0.6017044154733435, 0.48364866662457984, 0.3224596466920777, 0.8545306377326833, -0.9574113645085438, -0.056385221058593116, 0.22062630127355298, 0.2510375320945517, 0.23913483530261992, -0.214809317997942, 0.19027730219835126, -0.3325355707153803, -0.09900469070174522, 0.48871268134376233, -0.9602903769401623, -0.6821892883223506, -0.4637798788658398, 0.694103124847453, 0.5052567658169278, 0.03465740113288551, 0.6575989055934712, 0.9421827912714493, -0.8290494115468554, 0.7749326342885383, 0.2487654983133245, 0.8449177327269564, -0.6125039005830692, -0.24866258138153374, 0.9124349460352252, 0.4709896186090261, 0.23221630405369686, -0.6062875859252288, 0.793470164857532, -0.09788666786006006, 0.08079721146425123, -0.5014755134975384, 0.31010027651252314, 0.5756576275666012, 0.0006241228784160668, -0.07317199505318683, -0.34080854660410176, 0.4991060385283064, 0.762871267288731, -0.049851796993676256, -0.736102305081356, 0.3948130536193608, -0.7020937064104507, -0.5222628515647483, -0.7627026336789291, 0.7843562501099173, 0.10922735959768937, -0.6395799535264339, 0.9924571286111752, -0.7347636851292332, 0.22235293421393143, 0.73614233341704, -0.10700556830489338, -0.8559604097429314, 0.8715995945660353, 0.5794904494071591, -0.6779066971022099, -0.2436738763361479, -0.5895645587233718, 0.857055948137714, -0.9773001649262287, -0.7352537907679595, -0.13587683935776496, 0.5394721715800539, 0.7067373767363803, -0.2634648595899367, -0.1230601481163307, 0.9747123644379769, 0.3553380024835855, 0.4805485897126194, -0.3406271260764513, -0.6753312868929688, -0.46442358876871936, 0.3710512657547076, 0.8839498252564228, -0.07897926713087933, 0.6287463902194352, -0.2416600545489005, 0.4837437806037037, -0.2633359667148045, -0.41428777063746525, -0.6570803693539007, -0.8537185950180479, -0.9014631101604591, 0.8207227417745373, -0.9119925769436144, -0.0030427122134177864, 0.6368597620777527, 0.9057832738638587, 0.5344821043459775, 0.1775737606992227, -0.022202137942442635, 0.39017851202240816, 0.34559584647733144, -0.7283188326968668, -0.46648683164323224, 0.5167789557079872, 0.27480030681552803, -0.4134401084568564, 0.19709951530832104, -0.9556649694089867, 0.1125017024825441, 0.3413958845973464, -0.9922109688183434, -0.12918076358823116, 0.731074164926143, 0.5134564103713564, -0.5504466666045127, 0.15272698205443147, 0.028872312876920825, -0.8219739933151722, -0.5992121634560148, -0.5108585303399402, -0.9392488436640547, 0.182348363390485, -0.42421501038860043, 0.012569775997316235, 0.2645582029304514, -0.09003622135373579, -0.5388642456921371, -0.22269410685831348, 0.13632144662903656, 0.31477988825383885, 0.6817184005983157, 0.14180355413327916, -0.8124317779981358, -0.1480146184236979, -0.6414501129938253, -0.10041266290013384, 0.041617626924934426, -0.4411819556975498, 0.33684238991418525, -0.03325715516448313, -0.8037361805956442, -0.6233255037448007, 0.5270117084330106, 0.8890433252039287, 0.28206379351040445, 0.7974988211589471, -0.8676890420535961, 0.2523574725140636, -0.2077087914254454, 0.11673951092028978, 0.35986486052141653, -0.9528892860732578, 0.84755221018602, -0.26600662429829636, -0.17434259779351535, -0.5250294114999228, -0.45757682169893243, -0.33950144904171675, -0.4120631285362246, 0.6554365854760698, 0.6532469554783551, -0.9707885235749982, 0.55854435235784, -0.18873265309164533, 0.0939124976345882, 0.4589386049195654, -0.8378399145835183, 0.3257586583743779, 0.3897694014904951, -0.5464246982411691, 0.3154525907537704, 0.008150446884930274, -0.21372694354428945, -0.7029907264995201, 0.3353644432613161, 0.026585775403659806, -0.332849011980928, -0.7549358896520897, -0.49705766993936074, 0.7795706689840556, 0.7746520079576773, -0.2657833156998699, 0.31376640178504256, -0.9762557031725292, -0.28073385605834544, -0.33642147699938696, -0.37934073872441854, 0.7209040441339636, -0.01669864720130576, 0.10395603626466232, 0.3200720492297664, -0.5051014858089078, 0.26676592138446087, -0.35014172743249006, 0.4051120641339827, -0.7829292381799022, 0.9957565019662378, 0.13637430832396058, -0.36930762372887505, -0.4175080824230091, 0.0037016629183967975, 0.6941253748608989, -0.21216450221158167, 0.6388807863123713, 0.333646604828691, 0.8074236544456461, -0.7302637255437034, 0.5433718771874645, 0.6320739116291059, 0.8007290901089694, -0.3124020911925032, -0.4587739799478783, -0.5275095446186553, -0.6610628536985366, -0.32346413936561813, -0.9276351688720443, -0.15332962559948782, 0.7659445884440708, 0.17876404766841447, -0.25893581391395926, 0.12828967380726763, -0.5092576510500482, -0.23854820454522252, -0.052452745004566204, 0.2450224600041102, -0.21319613912100865, 0.48316057285765845, -0.831308122888178, -0.2504537687024231, 0.7208040861169889, -0.8630836476993033, 0.43085849626441663, 0.26602675087431615, -0.9852384943085304, -0.6986041804527607, -0.5632063483881893, 0.5891773183223541, 0.4321912774217964, -0.3491869997163617, -0.19305276174757946, -0.14288970061975914, -0.6043948165017055, -0.6981273993482688, -0.9981556843424044, -0.47545454844601354, -0.1890646228574957, -0.4498201203916683, 0.5390837515018869, -0.27205244821850383, 0.1506812076919568, -0.5343246332845677, 0.05867571164729957, -0.6141924617648578, -0.7278405086414586, -0.7049904961195461, 0.6636306271005805, -0.6217766906112099, -0.30725856507144655, -0.841232273178617, 0.519398369704897, -0.269445929576253, -0.23690497487983242, 0.09383915838304246, 0.2887372751902162, 0.7357306868521396, 0.5797552618649116, 0.3994426108227809, -0.5504239985106709, 0.04242985867462146, -0.3106449191528322, 0.7881088061526547, 0.7463787846056644, 0.18892450434336694, 0.045358166603967076, 0.39757382714716405, -0.3300185158070361, 0.3212561618641492, -0.6084825106911353, -0.8386008198082844, 0.5005980840837365, 0.5853399576995153, -0.5160474427401018, -0.6117074330478522, 0.29975687997833966, -0.5364917218030261, -0.3017835318427795, 0.6599597024876349, 0.40487251557810056, -0.7113945529944594, -0.0007022510676855109, -0.3227445128941273, -0.8882891929070771, 0.9492953065907026, -0.5421308824534852, 0.7588161745836992, -0.9008645927327843, 0.6563496151061579, 0.8047708772448021, 0.21736333902988325, -0.45472687635186815, -0.17224089288958222, 0.5652922794868407, -0.7831777882457842, 0.4301633095475548, 0.014222368144184205, 0.6340344148834529, 0.6214220337209015, -0.9867348840275725, 0.9299202313704114, 0.2832940598150966, 0.3050599616675973, 0.7107378077750044, -0.8042937498114076, 0.9379716313919992, -0.07183366080204157, 0.8409873602897393, 0.8245501508770368, 0.5093913401485992, 0.3624437929844053, -0.966191371136697, -0.9585511716457544, 0.4365318531298077, 0.009972207595819738, -0.5970815835600836, 0.6124823675250344, 0.8813279175727078, 0.10996244367473285, 0.4268985370967353, 0.907207445847392, 0.23406130552002735, -0.3924956250720497, -0.7777613490651969, -0.8066145812938874, -0.46707856587247343, -0.7872957527285129, 0.9717305979715862, 0.3232259255475953, 0.5741063152001096, 0.6326204619593365, -0.4267820024463165, 0.09867116466267278, -0.8855173535385741, -0.4849633842299079, 0.38183412282960627, -0.16310360422399794, -0.973557421453896, -0.705064007041043, -0.8481331739853257, -0.9053529425516553, -0.21427052073583797, 0.2479048600835927, -0.5969904588236603, -0.17446673703976256, 0.7797528255019317, 0.3254558310865401, -0.10036009684710501, -0.12726629758087538, -0.5267259822563986, -0.4549355204681176, 0.5356927670451255, 0.18417450307236716, 0.10667575124665074, 0.32078135386127027, 0.8888790087368568, 0.07656764187140119, 0.41648679471511296, 0.7700232192118333, -0.2799729107406861, 0.2700190928396333, -0.09345239123246518, -0.47332057257153104, -0.917572114674555, 0.7593050239043404, -0.4541650054772681, -0.34025346707954296, 0.3360671818042451, -0.2310387514732657, -0.9104782967054794, -0.2194095486563199, 0.5858550583376072, 0.8740195670450395, 0.3375243701481585, 0.8257419387141192, 0.7508771546677102, 0.851473611418051, 0.40914729303903474, -0.5480165452267398, 0.3384556060734354, 0.10760894465048487, -0.7801906598934825, 0.2629212967461305, 0.1381187092138214, 0.08181406139882874, 0.04665282288565664, -0.6321104006821103, -0.5423201740884971, -0.3795498123316343, 0.1078305371786985, -0.3547620470245487, 0.20973850906374314, 0.16222915665634963, 0.09997069115855339, -0.5663055631320855, 0.7647402553843856, 0.38651907097797467, -0.12531139027396265, -0.12169418963940193, -0.642044231191546, 0.7837921884100287, 0.9292473313212752, 0.9446360643849065, 0.11172368046076264, 0.8869256276063084, -0.37353167808161936, -0.890817549084449, 0.7661500151501714, -0.5416265397174256, -0.37980417608418526, 0.6655974641329481, 0.9629817452297884, -0.016232273136689734, 0.19437890797817414, -0.207694129982533, 0.22396457434336603, -0.4650651171698741, -0.07886550066652664, -0.0016409347449302114, -0.6125450162585417, 0.8132816506722653, -0.2581191882196374, -0.3273596153339977, -0.9695003107483842, -0.37953476646087947, 0.6548076819971478, 0.6896081462744883, -0.434169457762394, 0.7133240466732773, -0.24063559560082193, -0.2989555349838917, -0.5769822938404221, 0.1983990338392645, 0.30351147873079865, 0.503913894425295, 0.3005478608733221, 0.871525142342388, -0.08426999682458791, -0.6588933309593801, 0.9771728697189201, -0.4982644838655321, -0.6059224236612737, -0.5090511185020792, 0.011261323591484684, 0.9097577468543363, -0.852836754619398, -0.2261464023938482, 0.2675158210284818, 0.9267747887828119, 0.6755674172294956, 0.3730541581644222, -0.04459432213551562, -0.9098211702492105, -0.5828282985606397, -0.012805309667054843, 0.947002657252356, -0.23580996442724111, -0.5840845653477424, -0.8834815973053287, 0.6035182089158904, -0.5838235211131002, -0.6335367335177937, 0.35767589617094186, -0.3149882805904023, -0.9813934840905378, 0.3249754521395156, -0.4169221694904741, 0.27053754838541066, 0.4290448398542819, 0.7597542395391998, -0.1395431826712994, -0.8362183854785192, -0.9093605865966441, 0.1535743957821647, 0.16260758513355444, -0.8333029546574431, -0.19003630740883937, 0.21858593935680415, 0.16992631666434943, -0.4350953479691855, -0.3385515885370898, 0.40532954491534445, 0.658050433941193, 0.3987395380623857, 0.11604627172847426, -0.09844916355300337, -0.11206110047215878, 0.6564496750616584, 0.8831577592920463, 0.1829823906351371, 0.5206978298268039, 0.4354759635421037, -0.019047494080309813, -0.6116348725645218, 0.2581454729950481, 0.09914662891217585, -0.9919421825282118, 0.3072994854781206, 0.8965218892064759, 0.5201975122883371, -0.311477042058824, 0.8964280968111824, -0.7791007189247827, 0.03801587071777268, 0.2264579814722214, 0.05409224485969055, 0.23390047850831808, -0.8350847405083668, -0.5487096221340064, -0.003641855507255487, 0.9821658517529159, 0.9940176087852723, 0.3320409120515848, 0.4410542275770115, 0.9550766135253994, 0.793846566811524, 0.9714280136015743, 0.8814583137489609, 0.6101213790005433, 0.5844100469635993, -0.14324286300602518, 0.007541524311575065, -0.30324040455502654, 0.21945729975827355, 0.8021552177625693, 0.48722959578648917, 0.2413918525905998, 0.6192465468789876, -0.03415108773176012, -0.34545229360437535, -0.24541066358350294, 0.6334390451623593, 0.22866544385373677, 0.9409138002806587, -0.5114514364400693, 0.37898385626488973, -0.5576904581145394, 0.31926518011356353, -0.44498406557105885, 0.44861409832045807, -0.4662919754885695, 0.680117836150866, 0.4146701767128096, -0.3603154585674331, -0.7203771900096647, -0.032352712102003967, -0.591743888464362, 0.20801993585944167, 0.7852459710203927, -0.5373350682313691, -0.7250765101655288, -0.41489352854417083, 0.26805967102805117, -0.45050613104416404, 0.43801547969762944, 0.8040150409450686, -0.05949061742686523, -0.02813096252483005, 0.22181869954987188, -0.32681137276515915, -0.5140824156803394, 0.8090935131326085, 0.4010678027013166, 0.7435060994470364, -0.14187372647111385, 0.14049008423154352, 0.21891990761802393, 0.1379043684507828, -0.5199068212149667, -0.3954657719716079, 0.13290050942356668, -0.8371720278910353, -0.6140965186871608, 0.8068692943590168, 0.38972824031021314, 0.13041383512911198, 0.6059801236982734, 0.4531942477275748, -0.0028140340801785513, 0.8729213368263224, 0.15320365821443116, 0.1425001934141974, 0.7250295887367533, 0.48056156974807673, 0.9112811060095856, -0.6746564150555454, -0.8349968391116795, -0.7108866295724683, -0.9160755745997364, 0.41149215887577006, 0.2866541601685946, -0.6370290994746881, 0.07645875007172598, -0.05345492819015529, 0.8763818879384011, 0.14538137344411384, -0.11956557938524282, 0.2320770835884407, 0.34528600080464944, -0.891700945195099, 0.3779694409752645, -0.5129597708701983, -0.5596463942955512, 0.5814701542157905, 0.5232285698320029, -0.7481654692954962, 0.9788139545362051, -0.5190534844274322, 0.22854954565873586, 0.04172805323276818, 0.9871295829866717, -0.8389451227450311, -0.7072266126931701, -0.8088309077861873, -0.21985155471855133, -0.4806389903245276, -0.693151319393084, 0.32901870836835334, -0.22307449692619086, -0.7778970305001429, -0.5117578257992024, -0.3230524718799461, 0.009870507493074943, 0.742069187901647, 0.02375469323925894, 0.7734249876528758, -0.3366367684147624, 0.6694276793004268, -0.27025892792371, 0.0065635099048046985, 0.001427619139200198, 0.22993579247590246, 0.6914135044759389, 0.49479703191767777, -0.4758168341430362, 0.6906067073107649, 0.7034185792213854, 0.4032253513894908, 0.8846417490896095, 0.6283686897795742, -0.629509333313713, -0.4477467332383771, -0.344808183554294, -0.44918709186739747, 0.9777858623223039, 0.27195709058049977, 0.04903401073916802, 0.8856358684919905, 0.23302734846447826, -0.0933913365661263, -0.5249736073104061, 0.7929714821578868, 0.5534229028543836, -0.4391021709122407, 0.6798706743294196, -0.7023964088467256, 0.247464416938195, -0.032996965223831776, 0.4509068630101454, -0.1688132952588668, -0.492691279641319, 0.9932024660699499, 0.1044393676692894, 0.7658843648669258, -0.2048110991745351, 0.6395334411979214, -0.4838545388323787, -0.2826937331999233, 0.7685183340138904, 0.18058051098643868, -0.1693159972726641, -0.10332447299502245, -0.6915070832086687, -0.9092764565637212, -0.36964335174793983, -0.9986841137741318, 0.02286850246665506, 0.27133663758959936, -0.18178087298519285, 0.9637464084283993, 0.4388525130500449, -0.9489433793608275, -0.7843460400092135, -0.8430057486461795, 0.31390585003691807, -0.3224756019817183, -0.6365851041400201, 0.16818489181435714, 0.18457106705598814, -0.7293414056899337, -0.26630058484167596, 0.709312343452021, 0.9877402057359308, 0.7687842151749984, -0.8620246163240548, -0.9799524486945879, -0.3026485244235808, 0.5547370495283641, -0.14826505620438946, 0.052504016682317634, -0.548610191170515, -0.1708093501653185, 0.713280707574705, 0.939099759336077, -0.6623638854015719, -0.385973910771243, 0.23136084013459168, 0.010035742352594301, -0.4944014571640378, 0.3272037680061284, -0.9873520320407789, -0.26009331437853267, 0.581482817549956, -0.09524030910685366, 0.8342140369015216, 0.35181536870112207, 0.21920734681106757, 0.7257183680078523, 0.6151861953843254, 0.5682160372819991, 0.7346452699438712, 0.9832941481187012, 0.39192653208416384, -0.0021055248484884714, -0.469258980347822, -0.5275919112863785, 0.20407466842269173, -0.8744681497624682, -0.39716687130608186, -0.6052376716691601, 0.7491720918897429, 0.6283798219209524, -0.43194182005411164, 0.8313716794636294, 0.8731657824571288, 0.5791748223980138, -0.007393489027764266, 0.5798519779760274, 0.8867211299647781, -0.6808816448975799, 0.2652157478181112, -0.9282521001856547, -0.6350902070450419, -0.5741084269454384, -0.33723604524137385, -0.008928692078621348, 0.3574241261380784, 0.16372888292359145, -0.8735422702970104, 0.5800076171279733, -0.035828796331621726, -0.7653109792061807, 0.745063949914256, 0.7009240574849813, -0.8758040474366326, 0.2886997522487562, -0.7094562215396893, 0.49742477283648845, -0.0680160649889745, -0.5420082552106367, 0.3474222297797094, 0.7817388330010464, 0.5990534118534145, 0.857722459092022, -0.4075814095760857, -0.80557812614907, 0.4613979141626723, 0.955351025353937, -0.723350921130161, 0.7579240107663596, -0.44573157855030265, -0.7447470445319786, 0.6095010455088656, -0.12001744927148561, -0.8517115553318166, 0.07385437726466404, -0.08930356667556172, -0.58456022332506, 0.7294645708064076, 0.7137461022351737, -0.4430349367336426, 0.5935935214395183, 0.47985474426063135, 0.8771154605389859, 0.7572227952421184, 0.8800347982238264, -0.2813073741445662, -0.06910220057199101, -0.5649704044069015, 0.21837468304684227, 0.7258875118364361, 0.251196766239999, 0.6871827005886078, 0.16276681055706566, 0.011313017548593596, -0.07030842932239167, 0.1977197754812785, -0.4099669935757715, 0.509734159056578, 0.938007015955908, 0.9831685959258716, -0.07373868293149033, -0.9613497931920163, 0.7033043675084016, -0.7940766656496725, 0.8855691739896936, -0.2616843081742515, -0.7040702048481495, -0.8516216064407696, -0.8976335064420646, 0.39682352838938795, 0.9897352103862618, -0.7035703484455489, -0.39345018507675533, -0.871927759057942, 0.7261321923741979, 0.0029707265592466836, 0.31503826209690633, -0.25221073037132746, 0.9549062102042263, -0.6967056524882853, 0.09706805742598568, 0.010354518885253405, -0.8838764968299131, 0.1860483604895753, -0.8680069093262115, -0.9467911125073687, 0.244168834452972, 0.47721131053037236, -0.6426807869311453, 0.3337579429932003, -0.8358634240093086, -0.01898174276658171, 0.9829815704790721, 0.7622837947889833, 0.05444135232645175, 0.8240102297284622, 0.23509471039356766, 0.4713933076855019, -0.14021398678850128, -0.46422428264835003, -0.3812802112857656, -0.3322875223361128, -0.6137819308010055, -0.9366917349701156, -0.5882617679028883, 0.24657394547973932, 0.31659965024493797, 0.24073618993733414, -0.3556498722689716, 0.8400039529790773, 0.4268638118354151, -0.3910993941228409, -0.9791904360367298, 0.5533555271809649, -0.8789967782770514, -0.555688553315439, -0.054429387025631604, 0.36368146533501755, -0.04373761306870794, -0.7919127864823519, 0.5957964735660304, 0.07630840088680402, 0.4175214431130634, 0.7404045254954523, 0.7371463728944647, 0.8802207451869539, -0.8783524030115561, 0.18301724171248313, -0.7094646335848085, 0.5943949097815913, 0.07460416204846121, 0.5806512260597221, -0.1819106513515527, -0.17665796401128775, 0.47530816694410793, 0.1441780561007957, -0.6406686303245106, 0.7413518881769883, 0.617509898853273, 0.04017581418386218, -0.8345034982720907, -0.758929005328536, 0.101646230471365, 0.2719987655661922, 0.028012084718544772, 0.011833622616629569, 0.4065694145193992, -0.8652595717725338, -0.18347643523456392, -0.8709881442666909, 0.6895868795785953, 0.7068242118076107, 0.7881091929305748, -0.6429100954548106, -0.6127872911079846, 0.9017648849144964, 0.7535408208605812, -0.24641940853879363, -0.40176184739706855, 0.6083421312271375, 0.4881317837775747, 0.7288363287363595, 0.2984933353024033, 0.2637990191639126, 0.5098980135496853, 0.5452906774514676, 0.7787078300429513, -0.9859850211538332, 0.21895086883033388, -0.36313947218896847, -0.131441015428033, -0.8117011807371917, -0.8207285882113129, -0.026738848466751675, -0.7545324927360351, -0.5282335999825161, 0.8588496310196925, 0.11400202379569824, -0.8771903446022622, -0.8967238286948451, -0.32399717039638465, 0.5056414157681608, 0.9119142957565365, -0.36765719879980163, -0.3115782203940296, 0.5802280725504616, 0.20322970543158903, 0.5924538133771375, 0.2501703467851366, 0.9556730562900038, -0.6501910224886589, -0.642835397546214, 0.03860486875184588, -0.540043540147203, -0.24041859155007983, 0.06344166511047056, -0.08790428281660989, -0.6461210770883961, -0.9701002857444874, 0.09611838430699571, 0.8331104270089948, 0.1991256172115876, -0.8654305055806082, -0.9340859818094571, 0.7184623663325416, -0.30475143694102513, -0.8814985506958526, 0.9617961161404851, 0.9723817717994823, -0.017100556228048847, 0.007549704210248587, 0.18584854212094482, -0.37078613940127547, 0.5220267034668147, -0.10483009735027737, 0.05403892107560471, 0.43963571723522543, -0.7613109969418761, 0.5087133100920009, -0.9395756876315859, 0.6249438974800985, 0.7534910206524357, 0.45649173685605104, -0.14647451415109392, -0.6167020945608386, 0.2346455822594078, -0.2764544466236125, 0.4441325028171821, -0.8721196028830778, 0.08971504728779145, 0.473365068538244, 0.15491815485257554, 0.012274349438697207, -0.7875460645863825, -0.027397084122655402, 0.3976462272755492, 0.041383482755112944, -0.6410306455504129, -0.1761550531955023, -0.3203427861941126, 0.8543609471680655, 0.9828468248481634, -0.28463185958819115, 0.1664941580753978, -0.6467296001034888, 0.14647938117452686, -0.22183110363780933, -0.49661860555101733, -0.9405056751830365, 0.3462681517122952, -0.9627452617530807, 0.08329650546521616, 0.19687807409311375, 0.253645523562426, -0.8440308964422987, 0.5976976457507919, 0.6229870098466626, -0.15681246347285804, -0.12985199824728566, 0.09111276511690392, -0.934748181063827, 0.7846952281911721, 0.8401505690605977, -0.12662932946183458, -0.9760148935031348, 0.2532170605535726, 0.38694481248910173, 0.4153105905072627, 0.9566189393798343, -0.6446536788605193, -0.6677168621171032, 0.21549056324008364, 0.1253882306123999, -0.571738195022051, 0.48026499514986476, 0.922134493919105, 0.6643066466833027, 0.9594692658263935, 0.5486617835248568, -0.531262208292377, 0.7390107057578377, -0.8700529998798605, -0.16923218476432234, -0.20305013514045522, 0.38209726431833, 0.003376134929415464, 0.373055667928327, -0.6140313669347706, 0.12829517657711653, 0.3229109696619117, -0.8477675380365337, 0.1631997878219429, 0.4818275344145342, -0.6240924239650698, -0.05685224118082699, 0.3259561434913283, 0.4717797855573227, -0.7258538364769214, -0.6097773362783725, 0.6828590610438179, -0.45117327627920245, -0.746138468103732, 0.2032283578764722, 0.5569948864147356, 0.9878453447685789, -0.9311314366807459, -0.6625209014026863, 0.15416810997850927, 0.9866341770925189, -0.028406769032649937, 0.24136024694492209, -0.4640028117648438, 0.3119916305081165, 0.8143943521382635, 0.14670358515658455, 0.6715933689599722, -0.2346734487406117, 0.6580758693062447, 0.7663765094232045, -0.020889898291465814, 0.3051215579153179, 0.1605055357255285, -0.983379156326373, 0.33522125472447084, -0.8252494622488165, -0.9030332620498183, 0.9833685721390601, 0.420128576940211, -0.4083482377538854, -0.8273788065296599, 0.4333328868537989, -0.2692437848524676, -0.5449899755462155, -0.9617612988240496, 0.2699141658867259, -0.6216358291214803, -0.5650009270501068, 0.9216901444574528, -0.07047726784810515, -0.32819841510344694, 0.7993701843394834, -0.332302785678839, -0.5569637989596139, -0.49021142707065324, 0.2532761220894477, -0.6461534028065963, 0.5241244830426524, -0.2706354722719162, 0.6040013151387273, -0.21345263134545167, 0.3328621477501068, 0.42085268782696694, 0.4952653972700418, -0.098598729010829, -0.7284640598945986, 0.4299656216531582, 0.018191418051756836, -0.7764191682468018, -0.9041816905437094, -0.5867501967817161, -0.740501149740797, -0.13997652804655147, 0.8439469869729017, 0.10270747412334513, 0.9407547099624454, -0.7411322908924454, -0.2893401806432574, -0.7533945178215515, -0.9507902978238667, 0.4069450031734194, 0.2419756948930578, 0.9896331432599155, -0.5007525842138136, 0.6839317274648595, -0.4677651880167586, -0.37291754814968314, 0.946441264765038, -0.06379294847245132, -0.2263153765037227, -0.7763359997616579, -0.6558808203265518, -0.01361479468568616, -0.5021349730500624, 0.4556062519169841, -0.6266472930356255, 0.6033173860550705, 0.49801639786136476, -0.8192532405644404, -0.24457633103603205, 0.9077670293928171, -0.5164343010741572, -0.9932166458581677, -0.8674963117003633, 0.5111343593158117, 0.3409462314938514, -0.8474981553754397, -0.04447811532554291, 0.5598729615440856, -0.7187478543068608, -0.29220151522592674, 0.4988601523327989, 0.5368955658882195, -0.8467887806555814, -0.5785527536653923, 0.8446760794279211, -0.15709804833141172, 0.9642718698487813, -0.7916447988252417, 0.07254096886055583, -0.5797552631917973, 0.5334719490116322, 0.8088797462053241, -0.2580252774649585, 0.2899959753293606, 0.8632235306181981, -0.8535279647999361, -0.06653189572673868, 0.5922942310914356, 0.580371222951332, 0.2950501130165841, 0.0965524990153881, -0.04515315793734942, -0.05239421934663979, -0.5077637262326125, 0.25101108013042484, 0.2299086281024325, 0.9557156300704044, -0.5689939311247474, -0.30425392303178267, -0.48602535936210933, -0.18183999043010513, -0.614501720601055, 0.20842380633398494, -0.621278933132454, 0.5077543409693779, -0.7972835561056277, 0.7852231112060049, -0.6950621137824557, -0.1918198337742132, 0.06970964525875067, -0.817555564870347, -0.9385929951335026, -0.1954348961393022, -0.5798689173907092, 0.27248298838374607, 0.7849760163545783, 0.9566306766749502, 0.989212817707849, -0.6609148274078274, -0.6963631324460313, -0.3070285077413697, 0.9016590470010315, 0.6256994700775904, -0.9442728006665713, 0.6756385837237568, 0.19235459155284884, 0.49165085168271117, 0.6978803446910251, -0.6225200581295125, 0.09887950354717279, 0.30169650673121584, -0.38254743993981455, 0.7657024389590439, -0.6576716701637357, -0.8502725163837517, 0.5546226999444188, -0.8059466188932631, 0.8451058358526824, -0.10859271525182956, 0.23555559519838232, 0.23007083760026625, -0.6215798193012463, 0.07789229016330479, -0.1880295541950625, 0.791289665346268, 0.21166877268467954, -0.8796653322677883, 0.2501783632205492, 0.6631052202146488, 0.871624352138652, -0.8869992228502952, -0.841821628798098, -0.1903182913272503, 0.14029005442488307, -0.2571854918046501, -0.3151030567789823, -0.2053162712086385, 0.34881816353790085, 0.012766698833196033, 0.6771360830641002, -0.7295124572689207, -0.8081617240611227, -0.2553566910305336, -0.40570754890754834, -0.29775378153319143, 0.45751769338581205, 0.5273171022446272, 0.09434762723308743, -0.8091456078904307, -0.05066661908311976, 0.5616597583879981, 0.7542876533931835, 0.8522338388574247, 0.7020130625782643, -0.918485079033581, -0.8322144799524238, 0.5931154023021603, -0.3731785469080837, -0.8482257175234988, 0.921320650988876, -0.885034966068549, 0.3040725069969059, -0.817888817798285, 0.8505371483697788, 0.28359534432066735, -0.3093142916025531, 0.7708256886007683, -0.6365512857743059, 0.9054148659734216, 0.8697119660186847, 0.9017558966104036, -0.6788544913688259, -0.007649886130093986, -0.22183573794592015, 0.6395964876814455, 0.8809550109822839, -0.7295209553599395, -0.8799686372640885, 0.8331465192364578, 0.22752556239542931, -0.1614591005390329, 0.9267383676240868, -0.2084863207794767, -0.46409233207942546, -0.8931707062797951, 0.709066606272543, 0.9823694094457491, -0.9793654995073913, -0.19053865037238227, 0.4185227500581079, -0.2680774730674427, 0.37076824762626703, -0.28590560601372306, 0.3058267491781763, 0.5105829431515498, -0.6142022188621816, -0.9895143869702703, -0.0606433181530901, -0.6743290816880834, 0.9511376532034446, -0.33547119147797044, 0.03325309070759763, -0.5461331465893862, 0.034891967211262864, -0.45922072551469584, 0.01687271354187181, 0.5840722477759823, 0.10851170194806259, -0.2917181605736947, -0.064735469520536, 0.16085744174459826, -0.856638288247572, 0.8035078038434968, 0.16207359792230913, -0.12659071010876888, -0.35736918375980453, -0.9638645456700914, -0.13085821668812736, -0.8342721921780092, -0.6530331705199428, -0.8239657925644939, -0.3943599066631467, 0.6074461533285123, -0.49220708361078236, -0.35863643251778243, -0.15743340424022256, -0.2884589112916496, 0.6624029221849896, -0.7847716720917393, -0.7546307756856212, 0.742670782221897, -0.8810430571695984, -0.9993981950307584, -0.04678406580197536, 0.5037833044643845, 0.06755069396976654, 0.3370599675170656, -0.47596611721215565, 0.7096661033443634, 0.9351915833267732, 0.7872781833420668, -0.9911669599858715, -0.45959905668235357, 0.3153381988308819, -0.8179943641081169, -0.6323618830081181, -0.15950275777482648, 0.3705537991085075, 0.8321734929019484, 0.6084519898337613, -0.8237712052430424, -0.2989416853298916, -0.7618924807091427, 0.15367918580756657, 0.27484699191080164, 0.33814807569157335, 0.18862010241663207, -0.92424044949313, 0.42256095154363105, -0.3182975830240409, 0.9587627392236324, -0.38810821112329275, -0.04201233566604423, 0.18089178704105313, 0.05390525794420942, -0.34037209284791814, -0.6399710077183636, -0.870287440320707, 0.2875387572318293, 0.8633646234524157, -0.7735402576417203, -0.7738505393151021, 0.7734045681041597, -0.10845587069518214, -0.14812154737406868, -0.24214429069204058, -0.36672231417242385, -0.3422598733513669, -0.8326395617079818, 0.10868093772846144, -0.9187979298991287, -0.935783382445966, 0.4642326597387536, -0.9631128364955495, 0.27501774070659923, 0.8546954444690609, 0.18591581970745752, 0.5647371700594845, -0.9157848218020888, -0.44058593689312486, 0.861018383921605, -0.31860346333750744, -0.5886152098628255, 0.4170081897404183, 0.8180153469988907, 0.5558219949069079, -0.9153103318057414, 0.8413765728749272, -0.5174419863163575, 0.6921445445157173, -0.45775987702963583, 0.47689377802590704, -0.7750437519684248, -0.8775301487245111, -0.39563128597613373, -0.767060861379786, 0.1604488194532372, -0.8397326630962179, -0.5885496464104023, -0.5215461090418803, -0.44841880892730135, -0.09024875348786932, 0.41204520591965754, 0.25908674552828503, 0.12115899726787505, -0.8393712385826038, 0.06592114945421867, -0.9478492452682632, -0.37126363064227497, 0.6876693280797237, -0.7665381858995828, 0.5332813736892061, 0.32063142990139815, -0.9825927782464341, 0.8030135353770151, 0.45117675538110746, -0.2742480301607786, 0.4005659304715452, 0.08389870056425286, 0.19330328977895084, -0.7951523606531639, -0.12991050500292434, -0.3827066147401128, -0.08201628199927402, 0.29832893298198426, 0.6657915838462738, 0.23173222568296215, 0.8094960996439595, -0.8083822049473262, 0.8233343745068042, -0.7666785336348299, -0.44699827294468397, -0.871663547519127, 0.8555196013524924, -0.4105141242638335, 0.8576510691751604, 0.4974850397825079, -0.6310155989681727, -0.8006432203351577, 0.8207940378964458, 0.2941708846111224, 0.9333862961022723, -0.9386469433376778, -0.5506952159387353, 0.6072949239296961, -0.7302013110138947, 0.5404244282357402, -0.07784266801248996, -0.49846305691093673, -0.34837841661367275, -0.691550890012784, 0.4236658129612936, 0.9192003694258246, 0.46418864100951085, 0.3446300464082592, 0.04381749402427304, 0.41662716347967055, 0.04683201543405091, 0.5010866048056062, -0.552047327877208, 0.12449917028272428, -0.7194162256191186, 0.17471261114348957, 0.8223397706590441, 0.35738211234288064, 0.8336302645559486, 0.5605006474901217, -0.40957283445527737, 0.7832211536573723, 0.6528615790797294, 0.11965034305583644, -0.3785691957746107, -0.41352353385810936, 0.6718731313520316, -0.9012409844147689, 0.38288354145244985, -0.1399173488130636, -0.5966529773217208, -0.504548621552092, -0.9467045162306269, -0.1912731202318021, -0.6403105809368159, -0.9237739191507888, 0.779768174199656, -0.9213275634666949, -0.6896659732390076, 0.7745547848201491, 0.7579268681705384, -0.6451844385233145, 0.17474818623356092, -0.3853407911325466, -0.006497355518810144, 0.35144660178413933, -0.9731483876087548, 0.3967659252568392, -0.368807833705153, -0.4104570787865316, 0.05238413849438239, 0.4854594183755494, -0.16426171441427573, 0.4494437096145121, 0.5102359139502342, -0.7735593044990126, 0.928000281982091, -0.6969308995430568, 0.4885083783058559, -0.7807788328197034, -0.1821431269780589, -0.6866500155713646, -0.10013779788294341, -0.5306701248941106, -0.7960724376875825, -0.47226620959188104, -0.47939412073859144, -0.9725332378449887, -0.2785883796590023, -0.4935967361896725, -0.5693407071434802, 0.28418280326301293, -0.49145169743131234, -0.8209084912809064, -0.03582476876034035, -0.7270863841503308, -0.7688466551832218, 0.6303169940741014, -0.09365931395391258, -0.8140495100577292, -0.0415699885470584, 0.9241386497003063, -0.031469221734605535, 0.6457539245546022, -0.7314745694117499, 0.15058565817272784, 0.3873462538728758, -0.6246917362018964, 0.3090561864466661, -0.9889515314010202, -0.6091525744609794, -0.5036859461112413, 0.332125967004246, 0.3698671171330097, 0.7363316871216663, -0.6130388778861073, 0.1703165414597092, 0.45312263850230505, 0.5683612762682348, 0.44439829735470804, -0.16893290596177546, 0.8618968697803016, -0.8498866820481046, 0.05137357846192536, -0.40979524645324705, -0.5509901329734277, -0.7656773158388981, 0.9528289287650777, 0.7731513731335318, -0.9977687754063391, 0.36610051490956397, 0.7911977274005164, 0.03568307241152402, -0.8154510570933748, -0.27351987609621076, 0.37672588451082856, 0.49527845980619145, 0.20420706064583527, 0.7181984031414559, 0.9953806722312477, 0.3482207311369663, 0.5528882200048739, 0.9920467927544159, -0.6154182294773596, 0.2548739446594781, -0.422005381966172, -0.24265786773830444, 0.6181158055861375, 0.9353992083731719, -0.8791589222878062, 0.01921211193074046, 0.8762329882957496, -0.5821024607278944, 0.9517127279456488, -0.5643604894764971, -0.4864921968376352, -0.11871519866540137, -0.8765783810701362, -0.36078966785042077, 0.7155735377255081, 0.6994231268981623, 0.6177869698777338, 0.5645821800014366, 0.8179000616651946, -0.5901763339727695, 0.590590455355587, 0.44247441518939223, -0.034786709388661574, -0.8779502239002817, -0.17216447007362823, -0.03073783512072925, -0.6735463044310026, -0.2073417818680312, 0.039382109699560575, -0.38186307712203527, 0.70126897164312, -0.9978642895029919, 0.015692126536887008, 0.5810689717891566, 0.6104499315579943, 0.36144030646573677, 0.05749565089548625, 0.11273795705447287, 0.6024836751894564, -0.5079663281907008, -0.4573841050826273, 0.9460788548163204, -0.06307171638893516, 0.8805272489128744, 0.4265866379211103, 0.7489189445469882, 0.5035345642347264, 0.7200738579387513, -0.6777452002437667, -0.9316868709816275, -0.23264900473748384, 0.5061131469194746, 0.13019877907344402, 0.9091443831306816, -0.2420367683550675, -0.7083643891636411, -0.7464784305128651, -0.593986966891096, 0.6405005751440296, 0.7196856524499853, 0.6953416096391543, 0.6178650919219946, -0.307894801120314, 0.8578868412987162, 0.9298264999184451, 0.7618761284455222, 0.22662944241708427, 0.6450553297632473, 0.010000557896064244, 0.13386812955226324, 0.5710154688632434, 0.7172824853606097, -0.3008928412246872, -0.755646851917751, 0.31008739819519904, 0.05342260962243017, 0.3087727038973953, -0.2972670566432205, 0.15611462806546128, 0.9742222834136331, -0.628229958823588, -0.24432071425280588, 0.6656733140994411, 0.6959802949046252, 0.5226250862256918, 0.46910315821935256, 0.6607322391708519, 0.9905231703657136, -0.7778228949489197, 0.03398932729765858, -0.5891566851456274, 0.9565022605586782, -0.5410859415212728, -0.9805286453057107, 0.2470901094788691, -0.16267364857725908, 0.13446298354496977, 0.8406499761074091, 0.8323067877973058, -0.34800759838137685, -0.1656687144174369, 0.3238250088049124, 0.8359098168071697, 0.4775281916021932, -0.4795071397834445, 0.1781820428575409, 0.7140279447207867, 0.8235654842156555, -0.2455712660145799, 0.8400845061831488, 0.3466451008997087, 0.16504278725977084, -0.45740791120283597, -0.9537627697923274, 0.36570930390399337, -0.24344698004073106, 0.5353560068286494, 0.3443734453939047, 0.6354466100966942, -0.8561629189637996, 0.8583260862520108, 0.29237089103239655, 0.07242879107496991, 0.308391820856486, -0.366691574978125, 0.4263601840871729, -0.1981784395651831, -0.06110654231898405, -0.6988861202671168, 0.04133483121455672, -0.4976041992140048, -0.7930697648167708, 0.5914854687964972, 0.3063066780055619, -0.78077636601356, 0.01854369555816615, 0.04190371302587059, -0.8645944223816087, 0.5040178175239578, -0.7474354896654491, -0.9310566117153731, 0.6421383800775784, -0.9041982652192151, 0.4040726570203388, -0.3523923623423315, 0.8877470164043664, -0.20779845670509856, 0.8610237124321773, -0.7380195297174985, 0.7331261001308462, 0.6888449696848098, 0.755707611770899, -0.9878469106142436, -0.3790308262211506, -0.2389046010778597, 0.961461150156824, -0.29260544559499646, -0.9666910015137984, 0.5353075990567582, 0.3880890655124867, 0.9122851947204778, 0.2284716042849262, -0.10536095797997902, 0.9069572537179984, 0.7427777070972441, -0.44115007653761285, 0.0868034569605125, 0.7071999893390859, 0.17615082734454068, -0.7975515831752669, -0.05200447220208049, 0.5702632576591642, -0.218545640770772, -0.6955151903788603, 0.9281661639967511, -0.5396654097586815, 0.5035505103061191, -0.23083242705440776, 0.8150100287542805, 0.2773808384126797, 0.8470357436993499, 0.4720773976989243, -0.06351266944657663, -0.48263018683880876, -0.037000758789136334, 0.5026348269540599, -0.5464468976909398, 0.9147598417066523, -0.3294416845984085, -0.8551162730348629, -0.27980556720691463, 0.40742258942204024, -0.3270388620047733, -0.8914239935552877, 0.7637247728973526, 0.014126607614620701, -0.26004995770413286, 0.2238943770650954, -0.883192577602741, -0.0789868182565332, 0.9441423042462402, 0.5812606168128682, 0.17034959786324522, -0.38992145240201914, 0.17986166817533156, 0.23790759963573893, -0.2604226255143489, -0.48175453856337325, 0.7676161669295998, -0.1408520020610684, 0.18771554345930985, -0.47036778564061255, -0.025898320561940746, -0.9430289123399098, 0.14753344814139457, -0.7687730692989609, 0.7330427596468834, -0.708771245198687, -0.028648975500562557, -0.2749847642264325, -0.7919700640062082, 0.8603286030977091, -0.35783353151165165, 0.16227424422997672, 0.9531955015875706, 0.4892819550527572, 0.46505288167184444, -0.4482814720728836, 0.16212167672458588, 0.23254252333348568, 0.7897391218510788, 0.9193661195392511, -0.17088437091097508, 0.8853477170881219, 0.3986264810254698, 0.20050861090129435, -0.13343769826098262, -0.30064402656582545, -0.5253736073788713, -0.09523396847503163, -0.9832355050409061, 0.9009646825656132, -0.6299007750336652, -0.5817578872326983, 0.650023079555194, 0.7629132761213475, 0.13763394251414063, 0.7606179734833873, 0.815775155948889, -0.9328891064096718, -0.759001750831787, 0.36968900813127314, 0.680979430204115, -0.14597686886158745, -0.43824484685273224, -0.44220584446040867, 0.8926100640514925, -0.6539842895548889, 0.375212882904572, 0.0122460777735649, -0.560319466184872, 0.6895025380164654, 0.4902231916172217, -0.9385688303004096, 0.5112002859475773, -0.12689283668116036, 0.6950288449007822, 0.5458752669927083, -0.5376428994791449, -0.029596418138425395, 0.1711830569079671, -0.5229833512411808, 0.4044594107187962, -0.0072241511281243564, -0.4377655844215833, 0.5790936055095319, 0.572379353166633, 0.730161136973208, 0.19338595105822454, -0.6763597842557831, 0.475885858930188, 0.5898614523270114, 0.10397302237697059, 0.8899401498768658, 0.9502083913238895, 0.9905153030938154, 0.6452899948085826, 0.11540963110837921, 0.4555927598099383, -0.13710587091975834, -0.13715643065297112, 0.4851786103106943, -0.9514219142781388, 0.7755070342719876, 0.13470787424216035, -0.41995159505528834, -0.5662199310318281, -0.763654447061695, -0.6835182075131039, 0.6112947508408502, -0.19988878996104686, 0.5234141129938332, 0.6416026897023703, 0.052943009020067766, -0.1931920374165872, 0.9014795278751517, -0.010942798589439029, -0.025347713400953875, -0.24561290201721397, 0.6839822545866632, -0.31574474624362114, -0.770496190369975, -0.6933764595876177, 0.42506578076934987, -0.7059481158443726, 0.0661156435361625, 0.011195926331561967, -0.08259225828378569, -0.6164521890704115, 0.75206951233644, -0.1287421834391853, 0.36678000228511, 0.38014940494366534, -0.6575180055849499, -0.4714368710851664, -0.9255747815477537, 0.6916776197135166, -0.3377914256968406, 0.22147736661117268, -0.23475065611236845, -0.5574242763426898, 0.41008864592123584, -0.2821265946956517, -0.6402684825079425, 0.8718959510739515, -0.12349983613089188, -0.041436878471469374, -0.872756551234438, -0.8795738440730492, 0.18198337602388026, -0.1607521200250046, -0.17374716015097103, 0.31011796560844074, -0.08207826578304012, -0.7339696586383035, 0.3514334950329825, 0.3088844851381416, 0.6045393222035182, 0.23602612556715497, 0.45646432466916353, 0.5245319711695551, 0.5301380570156242, -0.6916538570431237, 0.2565505509047932, 0.757879279954724, -0.8233992992039094, 0.5138882238382512, -0.20174057738748274, 0.2885323342591961, 0.8946970274549773, 0.6950922902058165, -0.08275135539049217, 0.12983797147554932, 0.34705279577212145, -0.8016208592533003, 0.42774318566213076, -0.49855794689075483, -0.5819669848115849, 0.45521234199487726, -0.5928865569935964, 0.27945837847041766, 0.39292924807370544, -0.06435474218184645, -0.8909181716011143, 0.9591131008338774, 0.7888293087585816, 0.5068572944934224, -0.2911580436662944, 0.4503546453482945, -0.5757711103341652, 0.012649878151515637, -0.974880170334302, -0.9299126792369414, -0.6542045964311252, 0.8508388491638506, 0.02552292133299927, 0.816584580509927, 0.8125763896349247, -0.8391149105020126, -0.1714402888315103, 0.7278530561339125, -0.6314957123461236, 0.9044606342787205, 0.430854612252517, 0.6561477742265747, -0.6244056432043261, -0.13130170560280519, -0.3736329547022723, -0.3523159311014141, -0.4255025992254686, -0.3265325422516412, -0.3603551657423434, 0.9639068990052084, -0.31787315896885016, 0.7091661026313685, -0.9260481736755204, 0.047249424725105404, 0.814344076271321, -0.9254932154549962, 0.9065536752746355, 0.5468050049922937, -0.30169280548219546, 0.6907366205173346, -0.05124737858751516, 0.7213979000892998, 0.0384114468036314, 0.167675848985811, 0.10580170442428583, -0.8295504758349943, -0.552867164357111, -0.0446686698601193, -0.8220843270780547, 0.8970964339674017, 0.19950678514146047, 0.1746594960102288, 0.06755294340178053, 0.646982931230683, 0.6660194635301617, -0.7307329242318497, 0.827910039615487, 0.11281091291063494, 0.7829976141261279, -0.4183121473135112, -0.8771622110903614, 0.7573645946590004, 0.5408325901206441, -0.0946092015015012, 0.06753665005832232, -0.2752089110607716, 0.9450201835486964, 0.07756604568584025, -0.7784329768472389, -0.03995554587323791, -0.08488721536198662, -0.17616983841293354, 0.501196841406063, 0.6461349762047646, -0.12858485617811088, -0.5825117849305645, -0.48402041468477397, -0.4243173334355965, -0.48945124003147966, 0.9948699843119839, 0.4577910335817701, 0.9072470837151063, 0.8829560101722866, 0.5882763609095574, 0.9611194360517299, 0.2832616773901939, -0.9690429816113035, 0.9189972490100784, 0.7349284507900347, 0.5376379165685639, 0.41334465209194926, 0.3216809302021264, 0.8098358562731842, 0.8935070992933927, 0.65735261587369, 0.38812067167635433, 0.48033479403868196, -0.7001423020647737, 0.9372292656982522, 0.32951259674256783, -0.5084279953737345, 0.7371735973207469, -0.685897633451706, 0.7905868744973319, -0.5708890099860409, -0.9625739429314069, 0.6979042038776224, 0.5864189937335069, -0.41906607382778605, -0.32417620015180604, 0.2351948047820569, 0.0815641750106324, -0.18813138718257516, -0.16032976043367841, -0.477119267983781, 0.1044140309904309, 0.5014376999292891, 0.2658456902912749, 0.14334311857406346, 0.9423684152088501, 0.18278476046883552, 0.645452979688248, -0.164561265961366, 0.3168088523122785, 0.822057977511601, -0.8107024025895937, 0.9025273366130762, 0.5848151062355924, 0.38306510577901576, -0.8913750114484953, 0.016193765276701466, 0.3845229601199789, 0.3325389114295385, -0.22883955253710453, -0.9579751163495642, -0.24065202604614777, 0.937661323718372, -0.14395628747037947, 0.15047736401627154, 0.8916443025663834, -0.09375577662032897, -0.9898048641465584, 0.7180588186776038, 0.8177855735744055, -0.13230994277851305, 0.1892822719536713, 0.1931559977934818, -0.04047960838528164, 0.16517696107805024, -0.5385341213405253, -0.6614013699294836, 0.4204122277732776, -0.2995033031925225, 0.2607186318121648, -0.6935741658801959, 0.2829193757638091, 0.10752014022233425, 0.9037360054828558, -0.015398297983139297, -0.2227691757091379, 0.04694263780767294, -0.3227946218288811, -0.47297275808956285, 0.8758288358595574, 0.5690751672401413, -0.48696629855078766, -0.061424077985365866, -0.46283907836291194, 0.3329863401052888, -0.9081264942380867, 0.5281333705877069, 0.9411633510880955, 0.3648277427132891, 0.7360132570002242, -0.25274738891199355, 0.13905661347767717, -0.7897950986394318, 0.12448433929485869, -0.3197640332353804, -0.5444624420709259, -0.368255011627699, 0.8673124699824102, -0.024220064191632362, 0.4523486960685754, 0.4301503545202887, -0.777394987474501, 0.31180052334619224, 0.5354296296948187, 0.21428803437512767, 0.18778517402686146, -0.7048684518358819, -0.5164093870055682, -0.6315203760841555, -0.6663859554785962, 0.8340424559474622, 0.8740933235804234, 0.31415386118612787, -0.807350812394394, 0.7272565261459787, -0.9040639608794216, 0.28207864400646376, 0.012277645196375087, 0.7175841458307501, 0.9085637144975496, -0.8282146641721952, 0.6406833375344705, 0.28045231098511025, 0.8918853951459362, 0.5041630864514566, 0.6702328601359633, 0.26315381411232397, 0.6677889087353883, -0.6570957051341031, 0.6377781725746605, -0.48822990442202996, -0.4162514563434625, -0.09636833546148282, -0.5313821487692232, 0.5281391077389019, 0.5730929987820439, -0.07827813255365257, 0.7952384712697382, 0.2602240818323842, -0.8634692294872401, 0.2977634973704326, -0.4013219212913759, 0.8490342115269627, 0.07978280761814838, 0.44554669214946796, 0.5027598079641618, 0.557577854615483, -0.6111357861630076, -0.47259343443392066, -0.032265436604843156, 0.2546871083687343, -0.6093597855743687, 0.4611612844000008, -0.013335226616631557, 0.08427446103842762, 0.1494909609894437, 0.18108252140338754, 0.5432593943365365, -0.185366656674677, 0.3379496370458641, 0.5254032907985957, -0.9259885471233202, -0.885111191878895, -0.5968833487368179, 0.8696776088262888, 0.9441566508165891, 0.6846089773767139, 0.44860432054200294, 0.8700465309964374, 0.662501142277738, 0.6028885688585304, -0.33302976031257114, 0.20774557825665108, 0.0536468492679294, 0.21279226654462846, 0.40785021163921154, 0.5688146028774379, -0.5132691219364078, 0.1561055925959376, 0.5608082607652842, 0.7377789464905071, 0.5363028758340376, -0.060009477709077164, 0.6175054519685494, -0.8341255392905014, 0.3931570180862618, -0.24825168268123754, -0.3023907829331296, 0.13036783637916138, -0.6516667943142884, 0.18387057275923624, -0.3295564272621394, -0.41285507963407264, -0.5433221675443858, 0.4344293320299524, 0.29486399923340345, -0.41984900125722, 0.9813028687266196, 0.2351320887634707, 0.6901931242023525, 0.5926839153261474, -0.9877995706705189, 0.44618367774495926, 0.12187447982107935, -0.891802011661813, 0.21635892535023382, -0.5157384531700273, -0.982087490200146, -0.25010018076251006, 0.22626234303506387, -0.34355072279355814, -0.5993613491284251, 0.8765714484574012, 0.4783480745896673, 0.20446076112266254, 0.5275551578636513, -0.24019350872371836, -0.12676883096540248, -0.7001654946633944, -0.5733510706550233, -0.7143780577459895, 0.21463914220697666, -0.5627055595222508, -0.24544430746733226, -0.8364726003650078, -0.2834987652518952, -0.3556296205078864, 0.927618568269776, 0.3564073138198176, 0.6318829914402755, 0.9761388874024646, -0.14219318972152317, -0.8723186260485227, 0.2599399817013983, -0.556378730411687, 0.02685150129088476, -0.3432433042630103, 0.28712830141889056, 0.1968895567031408, -0.5840183251909754, -0.10357473087963709, -0.7144040211155809, 0.7411250407538474, -0.2257446346044738, -0.1940238835958179, -0.6884365710236906, 0.11591212290598985, 0.852620874904146, -0.07177918569263686, 0.7066524439422774, -0.1914938836106297, -0.17247132842980584, 0.9350169050049308, 0.3207657696086581, -0.9550755575255538, 0.027567914779138336, -0.31391386320313863, -0.495201583205523, -0.494912769122851, -0.26279537848067425, -0.19426265335649506, 0.8858870198540885, -0.08588713274889859, -0.5268692297563, 0.9235827221512509, -0.8817904940574031, 0.6900377293977715, 0.9835962005241261, 0.8496724539534828, 0.8549174972816542, 0.19722975008036836, 0.8869075135113542, 0.8831930106979347, 0.5147630814836708, 0.5945011617568325, 0.02487629257844559, 0.501990454663517, 0.4279726967822173, -0.7181660020818927, 0.3498336602418102, -0.5814133540179809, -0.9843107314289123, -0.411265560810294, -0.0652965245796604, 0.9120801638542295, -0.26037420847370774, -0.9193725630803009, 0.6037522155838346, -0.48991104452505296, -0.05726904948882594, -0.4259010156397083, -0.6169220037312233, -0.46983151798439327, -0.6697758502723934, -0.44081066904925836, 0.4456815629242705, -0.05810705293198426, 0.292656968170824, 0.04016750716193762, 0.4277952050066536, -0.707540394716291, 0.37577054449588565, -0.5047499898116137, -0.4695520873153711, 0.7185019277325753, -0.9280259210984696, 0.020467828320961612, -0.8250367472374533, -0.6168265489227951, -0.8875930635129172, 0.024593512123013417, -0.3905170698645686, 0.011882630818289774, 0.16442575542575555, 0.534717985580075, 0.23318917364627012, -0.31932596802824875, -0.30765002263521435, -0.7310800696842648, -0.46906029823782025, 0.6997290880533025, 0.1331329679032336, 0.8113263404655837, -0.8566326245165763, -0.9295769615417613, -0.4917759631331826, -0.7356531704826681, 0.743701080496177, -0.6772727077963172, -0.30974658165078583, -0.5217262325608922, 0.12788693412074736, 0.35983472027059604, 0.1712126522438715, -0.1323822311569669, -0.7464482271264581, 0.047761257650929645, 0.6275388253659362, 0.5783330934086537, 0.9413171089122372, -0.20473530004096463, 0.8056306309818897, 0.8766121163963849, 0.3310164492998626, 0.17158599955439402, -0.3910110438219334, -0.8713605249970864, 0.48309840301516815, 0.14799072754073328, 0.9118164863604468, -0.5696993713014926, 0.4330735340884171, -0.9496988976181786, -0.5518501944394478, -0.05461699629284533, 0.01076964249694834, -0.7370284191440208, -0.21938027220703593, 0.7558781049767456, 0.8308312390107946, -0.9754538579399039, -0.11819029615512822, 0.4037823416825401, 0.38473930829148273, -0.18567568465257422, 0.6874838945067172, -0.9501882751755126, 0.12158154806061816, 0.059336245094607554, -0.1757188514844834, -0.37739071091596776, 0.49556778984894323, 0.3246912008324363, -0.7897218904512142, 0.4512767303705336, 0.008463610362861695, -0.26887844308635067, 0.034084812241200524, 0.18950744283497345, -0.023808045082666096, 0.632679801062479, -0.5360333662144099, 0.6101108570754992, -0.5023150019473543, -0.3669195954251476, -0.2596237077003558, 0.5201583805506043, 0.12767829030414002, -0.6292085612071496, -0.3104948015456439, 0.5331792864085056, -0.8351940712674382, -0.4571718970555616, 0.05096307458031224, 0.4992068173342701, -0.5202846253942959, -0.5621263255318549, -0.22945589699595192, 0.9605166819016773, -0.4345170629566413, 0.3988637551605556, -0.8201626266605759, 0.6122681840284927, -0.536649796072614, -0.49073542134358017, -0.1930263908741463, 0.5237767518330294, 0.8150780365702637, -0.7737162403553561, -0.48888662162107477, -0.8261252311821141, -0.1939906896205279, -0.7471252402040964, -0.8030570134902661, -0.8120768002883048, -0.6449263308733302, -0.44866706923359456, -0.940467658956053, 0.14934249982161218, -0.8424380290575124, -0.19524318841795885, -0.16853419746771037, -0.7600117947150933, 0.26560514312846006, 0.725796946805938, -0.42378972505389534, -0.5212154579078236, -0.6788983964420297, 0.619433385731031, -0.5877683032146506, 0.24295871439364802, -0.03275958916049371, -0.12355960895562057, 0.36666336158681556, 0.9126637020828845, -0.5515683405555365, -0.45252067095006665, 0.06027414136332565, 0.5112636791589857, -0.2588322700531398, -0.8879479258452292, 0.7732382953886119, 0.31626197617550855, 0.7349094884819727, -0.897432533776962, -0.5949506556156268, -0.4843487821679098, 0.28540892641847737, 0.47628562069410685, 0.6990898775753005, -0.6556371010493329, -0.22085755979482347, -0.04418017207182334, 0.21505688233265996, -0.7884386742023501, -0.42939272435756126, 0.40607974058381147, 0.5171233696342736, -0.6316407262900441, 0.33947365782904604, 0.5355280188364258, -0.17854982586695356, 0.3777956017961379, -0.897232525708384, -0.02440720553676745, -0.7952814117877862, -0.08613295709353297, 0.3233947540360116, 0.8181864986286327, -0.782400330247236, -0.8969468260505347, -0.15813086040095303, -0.0756496163249265, 0.9670298423128889, -0.8734538046648994, 0.41917020157901885, 0.9688649677160761, 0.6055919130660192, -0.26331112967116876, -0.043626777284540896, -0.3047595025247827, 0.10913756632070437, -0.043273529588601756, 0.493894902640609, 0.410310782130086, -0.987499885106675, -0.05517176976751337, 0.6946152739078328, 0.3024827927582636, 0.94044510024308, 0.8923582596328004, -0.8480708325084025, 0.7240724055337797, 0.7037795103628306, -0.07050106242224041, 0.9611832705390915, 0.9349411257888856, -0.6503663653687366, -0.7069975111626325, 0.9731566651078742, 0.2376565072908361, 0.10875977471484677, 0.03537229754201876, 0.35603570843256827, 0.528598066756298, 0.4013325831290675, -0.41065021576963767, -0.8267590838412235, 0.3647175480813891, 0.6255448720873931, 0.8712855034218896, 0.9803373816471526, -0.6555194334542604, -0.9466338244235044, 0.5623298468244735, 0.6669351350881145, -0.2482121896682843, -0.04044576380013232, -0.160808680947659, 0.5314053520417859, 0.35522133396536715, -0.9928613813415357, 0.947366510432897, -0.8569652334835824, 0.5305905514866374, 0.655074331566873, -0.5605555524314705, 0.24627719044528718, 0.41305583028702864, -0.41022231684974786, 0.9958748201048582, -0.6163355317276886, 0.38327889135898063, -0.002988384053919102, -0.486635350562977, -0.7135291569298026, 0.09181331845625462, 0.9106550687278094, 0.7293226504124746, 0.506959780899189, -0.6137565033199932, -0.11845651316898698, -0.12176062720649061, 0.23694901975288918, -0.1243602648386386, -0.30362145036601307, -0.8164171999880905, 0.17325985343838624, 0.3505377069293574, -0.6111223202343332, -0.27962284778674995, 0.42019149507244746, -0.01945436900747821, 0.4507857322863882, -0.623876433512577, 0.566219264430917, 0.3020710356987557, -0.32849643734816447, -0.7835934627426975, -0.46668616654480966, -0.27955406099916735, -0.22823904007942475, 0.21556835416129516, 0.4649095779361876, 0.42347469441341423, 0.9465355384045966, -0.4569890560298113, -0.21958075300419888, -0.07103531607909708, -0.5638406460420256, -0.6962545949945176, -0.42051905959854174, -0.9202447557568856, 0.8455982140209426, 0.6565382628114369, 0.7895325648428961, 0.09873485279167404, 0.8061593700700851, 0.2873196760882515, 0.21061923122568094, -0.742617119665337, -0.22979004779619094, 0.4707830587640458, 0.23132401284064041, -0.10483823002167769, 0.3660107692572234, 0.335278094389482, 0.08250457034584047, 0.7494631066957835, 0.45799202799551275, 0.2681027762026005, 0.7942207873277087, -0.543901482809805, 0.3698983349631155, 0.5608114528499644, 0.0462631888374625, -0.24643272449450815, -0.5222207475770924, 0.8705945563526143, -0.7412858569616965, 0.24806644841337633, 0.3227054336817883, 0.8017943714263183, -0.3802209285936484, 0.4213874258566348, 0.48472427752657965, 0.9412485774628196, 0.014270875863992893, 0.1760663619693439, -0.2769422859596786, 0.9691603809141436, -0.293795842354307, -0.47098635653984067, -0.28242932967460677, -0.5281775794056724, 0.4726202428971076, -0.9101761337044314, -0.40749653443037226, -0.3180029629878558, -0.04802701958236222, 0.03407658641981204, 0.126339535674169, 0.28023003509178523, 0.009260848657830945, -0.8618048552912834, 0.594516984576781, 0.34066512357380985, 0.3065524355607072, -0.9228493413498433, -0.1286180787890039, -0.8898407762820146, 0.200206245053, 0.6355465461189547, 0.45643670685172233, -0.017693611545351207, 0.7258380859273525, 0.5644166969619178, 0.21487986604758502, -0.21810060840170764, -0.8463886951523749, -0.8534772249441496, 0.23032757104742574, -0.3110752436207225, -0.9339816257252007, 0.5591024215460139, -0.1521308894497646, -0.38712033511619004, 0.04245656065444403, -0.8008342105418882, -0.05025422322969941, 0.1322889576126467, 0.9429068531558824, 0.43141308297763103, 0.3044828287093553, 0.8762931919633079, -0.26134844151752046, -0.5132579610248595, 0.6043354872021112, -0.9291392583248905, 0.05603286238141658, -0.7348153817182828, 0.11282370425254817, 0.8251670474086132, 0.014996764403532259, 0.9457591299276671, -0.36822710488624844, -0.35546624501614943, 0.6037785259153947, 0.17850214104557227, 0.4487590481283896, -0.8389749365214956, 0.7499300591987517, -0.7342943342517689, -0.4070840792577797, -0.010628348827176115, 0.6561252239571056, -0.49808378841404344, 0.13124018861785247, -0.5420242089016787, -0.44733019476670166, 0.49649695223627965, -0.5966960049171468, 0.08066083642883459, 0.8449439439316611, 0.6395877964731704, 0.2985030060591656, -0.09864591029911995, 0.07791394311705879, -0.9347749392739968, -0.6513764447278905, 0.33488586039657653, -0.8614262339453438, -0.44275827860343253, -0.9644251402909993, 0.8078500925525671, -0.7222936319691389, 0.20315899837695772, -0.32064686854502944, 0.5539606625806732, 0.8642012500292326, -0.180537464888898, 0.26366111141079474, -0.4434127843657787, -0.21633570076782926, 0.21711844093623434, 0.3878523057281371, 0.15193128060214312, 0.9139338022881671, -0.779573009945965, -0.048035344020149306, -0.12410690151836401, 0.9041343746235324, -0.37311729143725136, -0.8510106502301154, 0.5426803327780783, 0.71932177348657, 0.9552398413639971, 0.8486770023673593, 0.22235334001841567, -0.4416923734778935, 0.8255719537899522, 0.3378897844078528, 0.8788213691571396, -0.0801905940952452, 0.630657664823999, -0.8558180354522522, -0.9249210030343225, 0.15593536690850685, 0.8787032736801166, 0.9737744924669498, 0.334406084821127, -0.49755192777153057, 0.3772039038829371, -0.8578236248470625, -0.16151636369083167, -0.75713431727117, -0.9670385362182086, 0.9964845981049204, 0.1595551346207349, -0.5476794343358156, -0.825848508889629, -0.7941918556040439, -0.42520164642469016, 0.8942411195994764, -0.555965555030753, -0.9723623001841366, 0.9691808216381617, -0.9987179424533532, 0.11998196424164309, -0.9537160304218963, 0.8524097032397215, -0.2679089959552412, 0.4254411096407662, 0.8619122900049605, 0.05217064974624663, -0.65959824689904, -0.32531059638000537, -0.32590576060525756, 0.5112492250411413, -0.8683041121842598, -0.6461315125882743, 0.6962389235175093, 0.9464769676947258, 0.07223586331255882, -0.5037478691719937, 0.05141234601530642, -0.4576585686794412, 0.8000378376796062, 0.5186274869586562, 0.5209793294972431, 0.7944517605027257, -0.33519336039068515, -0.306042088075154, -0.016065567852848517, -0.6689286305394886, 0.9993204530542206, 0.7820084964320817, 0.11423656754886746, 0.7338036915744877, -0.9026770258238439, -0.2313595551335026, -0.6093766202964186, 0.44456940404644163, 0.14171889141398464, -0.5951674149342878, 0.8251559646982627, -0.8352911925432194, 0.186188347379054, -0.20033856786174642, -0.7724747605611209, -0.8896657259027361, 0.21672400399904634, 0.8797056407947244, -0.7529911238464779, 0.4200863900468019, -0.3048945299061061, -0.9875269676136953, 0.6819685234187218, -0.09950289887578267, 0.7284988757981927, 0.98860547708548, -0.8394349390430436, -0.35721685690390337, 0.2905777121791435, 0.46284328075346104, -0.19983456957720325, -0.9144278164761361, -0.13820146320263782, -0.5169796067242234, -0.9578729758924422, -0.5964926092799838, 0.9630270346699286, 0.47947524237977523, 0.5232517267368606, -0.3429131201558202, -0.9826755106512539, -0.7792960611891495, -0.8611383536464308, 0.7691358268309625, 0.90125057575285, -0.7826326989928536, -0.8674670695705686, 0.7921659348520049, 0.0909836266648496, -0.6266969723405871, -0.9123861888981954, 0.0369595211861411, 0.21101814629596216, 0.9648677798789373, -0.3095212297963623, -0.11791045730742478, 0.08384576821490497, 0.593744910557958, -0.8384997887969596, -0.5381372991256463, 0.18716636247332863, -0.5858823770370982, 0.6561158208826314, -0.6741942259136626, -0.464729782604242, -0.2995190410490558, -0.08761010919721901, 0.5765716755259753, 0.2634017306169123, 0.8080034391059225, -0.5328155476822776, 0.06834986797782006, -0.2802286717727336, 0.9575970572642551, 0.5847845977308808, -0.03869395542510201, 0.7233477965255632, 0.05301017151276777, -0.017857428066968994, -0.2907639209790509, -0.5058240795198039, -0.1488189283334047, -0.21713846674827897, 0.7423135943985602, -0.4789103476716967, 0.6298721511396601, 0.5257252327412734, -0.8433851322817314, -0.27547306021481965, -0.907521326175549, 0.715471895382803, -0.22596195753481463, -0.10720381894370767, 0.9086954096274766, -0.3470005136330703, -0.6874407172902102, 0.8571125177642844, 0.8273072191381912, 0.8246127619257482, -0.5419234837762241, -0.5092516855047464, 0.520472265873742, 0.7175335935020422, 0.7095393655173456, 0.5642273634320847, 0.5425448258619412, -0.6052649714272744, -0.7664399056744535, 0.35406202862243474, -0.5117084316263658, -0.27121554683086546, 0.4175989841721657, -0.7303960163090162, -0.7510320817123604, 0.33775527535407024, -0.24748241603587817, -0.9907455001843162, -0.6598342775647221, -0.8322205501840492, 0.5788362150325621, -0.24480650238508117, -0.7030598815936713, 0.3743725734351211, -0.795037273703793, 0.16716058932553501, 0.7242092001260805, -0.4319153413296153, 0.21476560929669963, 0.7932351813644913, 0.8590203340021041, 0.9555883009562234, 0.27448498915328745, -0.8573269360195286, -0.2866182013245915, -0.6670656031695232, -0.7806172182281494, -0.5185237404222387, -0.04081827511123137, 0.20908200611336158, 0.9315338347964577, 0.471438035710142, 0.21800112517618508, 0.7260389458075982, -0.023610623314629153, -0.8358957820795703, -0.08641476739033616, 0.7043342660410878, 0.4551823740028773, 0.23132704171451657, 0.49465121435453074, -0.7760515221559605, 0.9402928756822777, 0.31884558932116813, 0.7567961407697414, -0.2526912380241899, -0.07989251392352936, 0.032362937547615545, -0.6180546648624381, -0.03926700734607258, 0.9649413765283037, -0.17154214783396782, -0.5963696770174993, 0.4266371812178045, 0.07273552883270096, 0.703324896774844, -0.95068169091474, -0.7835749696142489, 0.7872103010513924, 0.9962517073774315, 0.3827223157231965, -0.7443047435247556, -0.7257416725426951, 0.7909173087366572, -0.03651144185762445, 0.7043377652703149, -0.0015574150472188464, -0.7826202273793976, 0.1723927500739879, -0.7929334649427999, -0.024055491867995604, -0.55481102740502, 0.8212931468834646, -0.9430868337536245, -0.4497617212225864, 0.17110368119647035, -0.7772960756156753, 0.4520944519856971, -0.337175370424476, 0.14260750136302147, -0.36318760875724965, 0.1590846543184028, -0.19503698614467369, -0.6139075808674739, 0.7350589997205716, -0.05584949421260399, 0.670606391104261, 0.12817088643993868, 0.07355771470462891, 0.4550724031525615, -0.716382214836069, -0.9765342108961272, -0.29691570208097384, 0.17741476579275406, -0.6275770478862299, -0.8160495121661682, -0.7783567858350429, -0.6239974427984052, 0.4525351751793387, -0.043196340035438485, 0.8203815941692865, 0.17388489519982997, 0.8133903773785891, -0.8830221817122517, -0.548309378507386, -0.691886186433952, -0.358811381379339, 0.7791238195138463, 0.5177362247724595, 0.49167152568079686, 0.3323751065557623, -0.35847392987783566, -0.9732336859387323, 0.08664926582007548, -0.8291327471512873, 0.3189134471765125, 0.8920014438617576, 0.9244471839623203, 0.1971593294875842, -0.6774107206414826, 0.5667744215063228, -0.9084980003923053, 0.6942743699764262, 0.8892011712399779, 0.6503339663898737, 0.38839025761746204, 0.1350835725594366, -0.6592457301515626, 0.9449201586542249, 0.9334307231902008, 0.2593724487050826, 0.44950270325250985, 0.6329890500600952, -0.016446472316939298, 0.14512689587785288, -0.9301359334121919, -0.28709701224217543, -0.45442630769729275, 0.3031860609094241, 0.9336444499571872, -0.6595849283467938, 0.24380152201222582, -0.5506582647370233, 0.6048094627244462, -0.972969766178984, -0.8956074500011935, 0.8632414501165295, -0.7522821835602296, 0.05608367791352564, -0.7919959849749452, 0.3467333447019607, -0.5804590953111, 0.7584313052344969, -0.012715895219418627, 0.0768255896629575, -0.013900688274436357, 0.5746949131947123, -0.34786706573394865, -0.46388801349069886, -0.16436489745989546, 0.43869432114472673, -0.6451244617315044, 0.27010050630253524, -0.8497510809780782, -0.15849840130945947, -0.23649702942635176, -0.024669647585258803, -0.8007128182594359, 0.11727392653761881, 0.006532214520095447, -0.36316717563204537, 0.8378151760747192, 0.06629552224590984, -0.6945257536385814, -0.26496885287349636, -0.7078702343119117, -0.5394185920177197, 0.18681799475258165, -0.5881550535355053, -0.047266918302783845, 0.8427210764668382, -0.9996797675280684, -0.014852502404195667, -0.9887085016355825, -0.7546939432890614, -0.03110151058463151, -0.8858487654679581, -0.9256457382538343, 0.04640826972509826, -0.5619720758043922, -0.8001696436812014, 0.7063625556217441, 0.2536165064731932, -0.8083771183813258, 0.3956103544633196, 0.2375282989736358, -0.14163638315119198, 0.1700999881369405, 0.8149480772130195, -0.6310193434927316, 0.20212269064988164, -0.5797102821109712, -0.4215083701014901, -0.4917516992625339, 0.1486347291796215, -0.3271524183992409, 0.8459930876455601, 0.16003825719843578, 0.868008360092404, 0.38350299890208417, -0.7961397625060229, 0.9509589959479612, -0.7392836459041454, 0.9553888372729582, -0.905838815796564, -0.8520402705893562, -0.33298327889630186, 0.09485990259239796, 0.03514812762036712, -0.4710203490008389, 0.18388220892304852, -0.6443753722007521, -0.037976646400515124, 0.14271798026455418, 0.9832680258451747, -0.5137904057378371, -0.5953106179329175, -0.12800053983216086, 0.11875394680357121, 0.9141419703198681, -0.4869695027664518, -0.11600840706420334, 0.397655038914311, 0.3622158646599609, 0.6809525715927236, -0.39277302237010825, -0.301452360481655, -0.1059189657196975, 0.4914246297553744, 0.6260744198432093, -0.7558537336677358, -0.9597262240776361, 0.012364702264377359, -0.5758276472548443, 0.6051420471217079, -0.41830119188459025, 0.5718920255912623, -0.4728891452397832, 0.16353114895463383, 0.26152207702930563, 0.6995802628829868, 0.8954331578545554, -0.24591865694751158, -0.10368355892584247, 0.30269247116560805, -0.06738944283577908, -0.5188454281380894, -0.07183558985454952, -0.23173621732012162, -0.10709708122641204, -0.6936096555219116, -0.832363671986263, 0.09821380781733802, -0.8656525685458765, 0.4509729841971686, 0.8987909125090365, 0.004158570658048477, -0.9360880073475433, 0.7385895254094714, 0.11820007475072702, -0.25232970776737207, -0.39495047317425835, 0.5780706974394907, 0.9125400450772352, 0.5491922005194121, 0.7088950315016183, -0.7539626736367839, -0.8904874587091876, 0.9859264647827874, 0.9865671224669925, 0.3905091249207704, 0.0030569733373151387, -0.6144517357594745, -0.7881814462919146, -0.3252721972595083, 0.6989789293134654, -0.1665952398753825, 0.2145344293859499, 0.9538135673890469, -0.24837678474839553, 0.13423226367638486, -0.030245636758762906, -0.8701978040853031, 0.7923251391414488, 0.29923422192399785, -0.6968752818900883, 0.6621062790483028, -0.2733772658392146, -0.09923532463801443, -0.6676407478931972, 0.8905367983855821, -0.8306634749590722, -0.7209532997428947, 0.9927643084470037, -0.47838019186517733, -0.050743970956505624, 0.25440518096009, 0.9932449922162931, -0.9830394619126701, -0.10571063611173948, -0.11908040024305766, -0.5598412300836839, -0.6945262183921024, -0.8885616821588516, 0.07266254321110766, 0.14150485504245358, 0.6231491504687714, 0.30797207206197186, 0.2226230094287287, -0.05914540480481656, 0.7428687050116869, 0.03248792560554037, -0.8593300959299777, -0.7045522407908225, 0.3091975703318941, 0.12833337081402152, 0.3688507986748213, 0.7956086628500554, 0.8016766919235692, -0.16928877575641854, -0.28311208697151335, -0.44730168926166924, -0.12703129733806917, -0.45968909034415195, -0.7328546555651589, -0.6174786043893605, -0.44920129967738687, -0.3791628650570775, -0.6703808379060154, 0.4192918857228731, -0.6948198305678783, -0.4008121715631363, 0.4851925454151491, -0.8962345433064682, 0.23337906682933118, 0.6774334855089237, 0.8382658600579445, 0.9217639641786282, 0.20387871090865772, 0.4935962860214591, 0.6580271549789387, -0.6057748431862224, -0.7840289162990706, -0.9700884662451983, 0.1007554792170764, -0.9149545855288412, 0.7664989533102615, 0.34520438427593514, 0.6090525724789972, 0.5926631978230199, 0.9467441296893166, -0.4980849862428809, -0.6260341045294064, 0.40961660825386814, 0.5011396042758174, 0.08409827068784037, 0.9435617645954033, 0.03034244824126131, -0.9398868398134046, 0.8084740445552834, 0.9433662880124565, 0.7423536707290841, 0.9416910195547079, 0.6935313832559222, 0.12272656367318735, -0.21587318396973498, 0.15281952903369578, -0.11444249597635925, 0.3970663323813395, -0.8542876341110051, -0.2104866374177745, 0.6448221968387868, -0.902350953020576, -0.35550378760928303, 0.7039824763948226, -0.4932591410340237, 0.9647210547228806, -0.6925416937692843, 0.33283471418921295, 0.6438872234649569, 0.16104478168694358, -0.1278030858618957, -0.9880379320746226, -0.2910131100262636, 0.7939376513220628, -0.2362658235891819, 0.538688352628935, -0.6262887311468954, -0.09041489633607114, -0.6038724101245347, -0.8741972066552517, -0.014187351094789635, 0.26083857273424105, 0.8122580987164907, -0.5923203910439012, -0.6046494696199829, -0.5962425038765293, 0.32394709121316123, 0.8527440340780463, -0.5609429950442122, -0.8312223326760733, 0.05211628383444, 0.743202426978349, 0.703249217898041, -0.11833319662302721, -0.12790341097818314, -0.531614695846097, 0.3548814098259825, 0.5782538765765932, 0.21963874210339274, -0.012320559828813193, 0.0649423921722092, 0.24828935589605927, 0.8522381041843037, -0.004299412044403539, 0.6906525194506792, -0.11279904540952668, 0.7551574519608251, 0.252590694221958, 0.3024804413573825, 0.09552102956411646, -0.2598761985488778, -0.2517947669991023, 0.8530956399278209, 0.20724214197616675, -0.1399502499054357, 0.6872603541148461, 0.6980120983312916, 0.1395521839156717, 0.9023253378281055, -0.8230014588049392, -0.49531655858644474, -0.3772896718152008, -0.6879681944020863, -0.3037507381406188, 0.4659719741229076, 0.49783599809127965, -0.5555703635723905, -0.5246651961133846, 0.554966167765941, 0.15221860058719128, 0.37256981075063256, 0.5382765046172646, -0.7854100874455292, -0.5400996458479606, -0.19930312407588868, 0.9541197530797949, -0.8976135968301702, -0.10394894835030999, -0.1934678094031861, -0.8196688993535999, 0.2240723450463511, -0.2908379505359864, 0.9503616285432863, -0.22970421141387143, 0.9185953761607559, 0.7499485291598915, 0.6536776501604746, -0.620411881723645, -0.6647228517112993, -0.41740824662604714, -0.7726734182892205, 0.5639905164996479, 0.5505469370442431, 0.6230746551562631, 0.588730454919123, 0.8897915031500132, 0.16209508431318254, -0.1435744125848879, -0.09559395154189199, -0.553312132469185, 0.07238864710538317, -0.04069563554423028, 0.2043690146043955, 0.4734422128346061, 0.9662407691357797, -0.5668037148483376, -0.4622978621413767, 0.3586309716803193, -0.2301782987804184, 0.4909902314519765, 0.04162968265585709, 0.4217149876887001, 0.7962096147766333, -0.2145081883411375, -0.836002246915881, -0.7934655161435562, 0.7429722312946452, 0.2517271583888987, -0.9077770805115633, 0.91544397592236, 0.8071581721675434, -0.8422943597515702, 0.9091742715990916, 0.1135888431306682, 0.8716750134284577, -0.7816911738221712, 0.8516970905292605, -0.057692869622641085, 0.5011974335328047, 0.1911816437089051, -0.04005862982594888, -0.022233340543679203, 0.40072650717658265, 0.1443685334421747, 0.8972058868746164, -0.8833800781665766, 0.894541953671868, 0.12591977347726324, -0.35597482592649143, 0.6340548573242231, -0.7178482935605226, -0.7410312930445626, -0.4442375537233991, 0.8119040140848814, 0.30888942174488543, -0.4742741801293675, -0.546889739515892, -0.584793996580778, 0.9042016311882937, -0.0011320106705059452, -0.8159535912374709, -0.8178231143539021, 0.7523577136319832, 0.776066961372198, 0.1300748864656145, -0.367325890136301, -0.30838270763352127, 0.6866799305003626, 0.4448638207479578, -0.9915174345710149, 0.1707764544699002, 0.37179339716176907, -0.7029658972062871, 0.7142884958928746, 0.11716374179604183, -0.33171581122175264, -0.7742973537603508, 0.5590300863252009, -0.7066748735773991, 0.21252783998463487, -0.9309854664175465, -0.7427695140570301, 0.10707343079394649, -0.040685633637888685, -0.6413957904814329, 0.7812087694854917, -0.32500050722177054, 0.2834420527979493, 0.6437067148410531, 0.8001537088443877, -0.7755448059982308, 0.04779888547826916, 0.46227207831997874, 0.3310040117558124, 0.7695498440111768, -0.3080427585155643, -0.8987416954899055, -0.20873922377609122, -0.9167505361977879, -0.29145886472166294, 0.9813047207618772, 0.8294512327131138, -0.9918849126260711, -0.7324480593043101, 0.5734895603475358, 0.39898846693326573, -0.6874352496582128, -0.3486696004286902, 0.271479890393749, 0.5459911691200277, 0.2042495751925033, -0.15182896095492282, -0.3046736901647922, 0.4252472593321366, -0.5993159653558826, -0.9094156402473905, -0.15115248803780812, 0.4711755254661394, -0.2713589942113299, -0.8330525724518478, -0.7925019194200451, -0.6807577626695831, -0.36433891351557945, 0.6934744129322159, 0.2320564367649922, -0.6418139756686316, -0.34278856387660794, -0.43988486721325737, 0.05464816826558305, -0.8843623663191327, -0.772641247313566, -0.6736637599833324, 0.14775648817706966, -0.7305908788894688, 0.40060933765564544, -0.1502637275807861, -0.25134617635844814, 0.9180326814868818, 0.7297400717109623, 0.5700239990452423, 0.5449103973861655, 0.6635740056820703, -0.6030257495046765, 0.586742034268992, -0.07472462026088644, 0.3917810865352045, -0.8203363631581595, -0.6096477576959931, -0.4057019083438764, -0.40763925033727166, 0.4008609047702727, -0.2702420152877256, 0.6714647279610453, 0.5443899678761666, -0.40016794603890427, -0.06178053589306476, 0.11082232746501086, 0.9481053213114745, 0.5055357105263607, -0.03091580806543459, 0.16697464700430786, 0.5032117036654937, 0.11411669103426658, 0.5034586439651496, 0.8586681654045041, 0.033117563759041735, -0.7868724113494903, 0.4391421823682107, -0.16262029928101707, -0.9129035134823034, -0.21904146263893787, 0.9723483939151978, -0.3833876730242285, 0.44905836190216397, 0.09615805124390042, -0.9763889253461584, 0.8373454096777226, 0.14352405390252465, 0.03885822682092854, 0.0427844359657068, 0.03121357295969429, -0.7267619700099974, 0.2740265210163848, -0.062238192763502065, -0.37315822091243156, 0.8578831103143139, 0.9564525417241889, -0.3482534919527591, -0.8180878893094679, 0.41780449062708436, 0.9611488590260471, 0.845275557751552, 0.9212988957077304, -0.3030977817264242, -0.1379385686165293, 0.9413552762650925, 0.736660792701243, 0.08833091955602357, 0.13851199316993945, -0.13546188828330585, -0.7919821973109729, -0.9366403721477587, 0.48277745304049535, 0.5463656751915227, -0.7567153836310969, -0.9650418819046174, -0.7123106862717106, -0.3219298856860726, -0.3130589273555908, 0.9999125095075181, 0.3083944661092357, 0.5082180136065659, 0.5579778796452037, -0.5736132470400406, -0.5782945159169857, 0.07481916579511139, 0.1990847236437412, -0.09652862916823435, 0.50115356022204, 0.31306120706544593, 0.5219683371827253, 0.8528464733980774, 0.714972319874053, -0.9651091493270822, 0.37181711973321385, -0.1498870030458841, -0.24298758145480992, -0.15681978293943866, -0.7261402020049259, -0.2260926349844099, -0.39145508280669605, -0.5068953448456945, 0.14181985122399499, 0.18228818360464838, -0.7581808714996434, 0.08542384677630022, 0.15525483122195505, -0.7282172456893863, 0.004587892159375162, 0.5999424877653017, -0.5434296829353584, -0.17115416292407604, 0.25189556879246777, -0.1075269121981175, 0.5292787150479632, -0.1636674836561416, 0.6046821044473545, -0.36629133421610405, 0.695783121693822, 0.8062536879964093, 0.9484994984408539, 0.3639726571652606, -0.01576548080013329, 0.7813438181834298, -0.7630709483588352, -0.032748901466328695, 0.8471523069455527, 0.9695120252968616, -0.13607931280681318, 0.6089651047608335, -0.9972994447241792, -0.026912664604368253, 0.42463545899638655, 0.7067275901865169, -0.22041985070859282, -0.7060391353872113, -0.19897538456578778, 0.5941506001707528, 0.1083527340833279, 0.6756450484461565, 0.1888147946203338, 0.5916471624000139, -0.531037137109446, 0.8387853455028362, -0.07596926896029155, -0.16776452277579645, -0.2774041113540733, 0.39594401199991625, -0.4440110750557438, 0.4682436355462447, -0.3531811804436893, 0.22684481934054013, 0.3357010923790944, 0.6470429151069161, -0.209844221486428, -0.6320211282757433, 0.7343655071947643, 0.4085349689998352, -0.23320031134672936, 0.05975560486779674, -0.6114368438771178, 0.498463805145807, -0.20262390532203023, 0.986363014579366, -0.5879182291046112, 0.4754360917646492, 0.07057820709833096, 0.5566306830783507, -0.8156133249642752, 0.39553121246616385, -0.6648163534472427, -0.276757095937437, -0.1663756148419544, -0.2725462573298656, 0.9505636112692815, 0.11216307042267792, -0.7437830791503397, 0.6765487285696066, -0.22972175149032426, 0.646213888682299, -0.5572114684889571, 0.9085780060008308, -0.5148461906785762, 0.3549115426403566, -0.7788381241453575, 0.07579711470697781, -0.5756662579662546, -0.2258781906843368, -0.2812870129092748, -0.26666509467086286, -0.9005050052774031, 0.43926325451779835, 0.3066349601249678, 0.0538117187594489, -0.48197728754016556, 0.10995290260724055, 0.060606421712323666, -0.4425460913593562, 0.03843564564589519, 0.44618206725848797, -0.08288863429615057, -0.8240375789312, -0.8589282095902178, -0.8982835072245736, 0.18498067912651472, 0.2965362537207683, -0.3572728724444869, 0.4402840526274261, -0.9328311662339404, 0.5260297776009386, 0.2941780196453354, 0.9049345537251561, 0.4319820941650123, 0.6868094384316379, -0.7438131378734538, 0.9835550548849601, 0.5071709314025776, 0.6182460828454606, 0.388541645202233, 0.09879651409305756, -0.7896001142283688, -0.7663556570770964, 0.8476484439932765, -0.6881176355753174, 0.5327955748581861, -0.6649454767421019, -0.72020158049182, 0.38685905487407, 0.9646342318712777, -0.43360866878254356, 0.37457791325952705, 0.5745342015084274, -0.9427269364280753, 0.7382154446843343, -0.9622815480441202, 0.7678544877494915, 0.6377218430446134, 0.5445545108251304, -0.48199670581375886, -0.1480830792885217, -0.05025338661218082, 0.0052322660202532845, 0.5962008713129254, 0.19974025827015218, 0.7960535092844181, -0.6005533559484366, -0.8399245945896026, 0.5156397655318532, -0.7679509559495132, -0.17358752772373243, -0.7299887468059161, 0.19604678300993106, 0.8628542080921542, -0.13904635051036762, 0.5064251900984713, 0.1701247922597069, 0.37317699198567533, -0.6366006221963081, 0.291417875258454, -0.7344711112403768, 0.5178257715844299, -0.03538331369864367, -0.3052719559881558, -0.17839090763559917, -0.7313866920887357, 0.4083006498984698, 0.36267572380276913, -0.5584896683289189, 0.5275284226504813, 0.3605109183182096, 0.8132057142745506, 0.879435714126737, -0.5303337102465568, -0.9967367440608492, -0.47738707173411865, 0.3553937216800096, 0.3208868778081171, -0.6952708139864499, -0.9262143250984938, -0.6272145828498228, -0.04189706763037382, 0.1190442440258519, 0.17020094616581938, -0.862420435309228, 0.05513041637884042, 0.8930511845415954, 0.9182694454699654, -0.4528247310700657, -0.36096742871959764, 0.2609804504062403, -0.8008923864105537, 0.8822012488645627, 0.9755850633637124, 0.7101144379716728, -0.7752374945513252, 0.3211324545652243, 0.2445370224594141, 0.7131724227477247, -0.9832306974716358, 0.4165671033192577, 0.2987785788990105, -0.5660690976318132, 0.48296758161205444, -0.09280982382451741, 0.38037233825608796, 0.5248332546137904, -0.5675069037856035, -0.5389630508206089, -0.5359286322053958, 0.2668749297099726, 0.8474588130019121, -0.23431621416822468, -0.2619278090398789, -0.009090337007610083, 0.34114803036272123, 0.08502832545401162, 0.2549327135471513, -0.20914276049269853, -0.3662835202279924, -0.5585427113140042, 0.8287844582664126, 0.6983448560850214, 0.6694766587791943, 0.32517524946680054, -0.2213159122868782, -0.2822041359424565, 0.8732018425721115, 0.15862246574420924, 0.38865406433641203, 0.14254948741159557, 0.4876737933951276, -0.17931648164726166, -0.3803596327197467, -0.5489501218984314, 0.5604582150174313, 0.43792458384317756, -0.35296702327547225, 0.6013761609212565, -0.9598608764031948, -0.7351342372591707, -0.12240762444614783, -0.9188422561616167, 0.8052388662139207, -0.6743251563130281, -0.3843904957501947, 0.11803256358689307, -0.2056416778024388, 0.8321984138950682, -0.6447228469904245, -0.7288239962394825, 0.4581723365410766, -0.5064378567905612, -0.41293041054752866, 0.06547869132024431, -0.15196642005848138, 0.20496220902450335, 0.6057638130734788, -0.1837357026509323, 0.383497669366073, -0.39524855149657623, -0.606741874074828, -0.11505365930298184, -0.7614661646265548, 0.6245837222666828, -0.42703620894842476, -0.6340518865021212, 0.4506255829346113, -0.7347885128068539, 0.09408086774875213, -0.47436544999439056, -0.20878569071622, -0.3881354799060168, 0.5883349925496266, 0.5253531357692738, -0.8432656290794809, 0.33361350568219605, -0.7490571801573798, -0.030446248993216685, 0.1651289429326579, 0.5006294836715055, 0.7096985244546696, -0.8638959632194942, -0.3320687034025356, -0.12773295739390989, 0.3587487133738647, -0.08501896273178677, -0.721174634334782, -0.3065714050506201, -0.07904364045528722, -0.03138355854522645, 0.9395623272761451, -0.5478862340710633, 0.30248375471903, 0.7348070681737018, 0.7609405199034625, -0.5094448636507514, 0.012968699756152668, 0.0543255629590651, 0.006263033305165422, 0.7094106589856346, 0.7538150582130059, -0.8517434882045538, -0.6812118567758008, -0.3406185429056272, 0.007050283008328773, 0.10292965329392723, 0.3849797797190395, -0.9444814678411264, 0.5784295245310427, -0.29357504048488225, 0.3750060885945179, -0.9856833679915387, 0.09627479881242618, -0.49285597543335546, 0.3146647649991974, -0.06423771433171943, 0.3655536485328157, 0.48754549587125373, -0.9765449412237956, -0.16548046364865843, 0.7272374896780842, 0.21400372796087397, 0.9259707943736057, 0.4437148036161047, 0.8726669969891108, 0.18749934472539054, 0.8744518596890274, -0.46037149719550086, -0.15389409355401584, -0.9455963495553745, -0.2969070308579347, 0.5841899041048026, 0.36651950888019114, -0.954794888794442, -0.6367567349672993, -0.3451900927188811, -0.841503613502731, 0.2435695997101539, 0.22512636891694493, -0.49750624299222634, -0.0541393191280779, -0.9860033318901036, -0.08559102903307836, -0.8201051003010598, 0.9445745108068411, -0.1788233158399808, 0.7657185151224322, -0.9910362145656488, -0.37702517534448177, -0.06255352569064643, 0.5931738225213057, -0.6594879526113246, 0.2625786599550377, 0.2651553846405341, 0.7473887671682036, 0.728011620207949, 0.3835864605929087, 0.872272530762374, 0.19450605704133062, -0.7219323714675616, -0.5934145770677222, 0.2303327195528968, -0.9211560726581749, -0.8413732712913968, 0.478781024340446, 0.8606685940956067, -0.42470098932450795, -0.26329281997213605, -0.8378950008225656, -0.5942553329199938, -0.35275497530528144, 0.15843351212906032, 0.26886979559794466, -0.670025314760667, -0.06828054106957193, 0.038532135518916544, -0.35526364716519643, 0.07761405941429311, 0.4029670640581877, 0.23618704974300053, -0.5399343001260701, -0.2689977442863103, 0.26684460226412443, 0.520241419997878, -0.5240722177626622, 0.7100412547967694, -0.6090321748911414, 0.052483412773309146, -0.3668800896352775, -0.4397373085846443, -0.12181158266182113, 0.19854930323349018, 0.21146362292890974, 0.4475637014700171, -0.3217460807326906, -0.7087107135835231, -0.8071603600091046, 0.95098659377555, 0.21074541587651563, 0.8746291382689284, 0.621340845628156, 0.19541854146689763, 0.18246772745960804, -0.45658339552045435, -0.9225598962298989, -0.025473724086799487, 0.19840811233218547, 0.06191472430540257, -0.8362527037251608, -0.13032935298027115, -0.9680643336489336, 0.6959214781039118, 0.299497198772823, 0.5109279500079891, -0.4041456539974597, -0.25574403670041357, -0.4924070816115007, 0.13990955910945302, -0.8867204738016157, -0.594637904976729, -0.7315605328713113, 0.7680853748127672, -0.358439130354792, -0.1960528163587829, 0.7440129965988247, -0.22771234255895756, -0.7868891466647665, 0.67134532623717, 0.05907770121532652, -0.919717599284674, 0.19812878788671018, -0.10197588619029596, -0.4627184446886079, -0.9530819758691456, 0.23630208785103446, -0.0857500294820226, 0.5332658844778491, -0.5754757453123582, 0.6034429694963093, 0.27982660116632463, 0.8635719689317853, 0.6587315012750803, 0.5677652501550889, -0.6343846793555379, -0.8749338728742089, 0.5755139803020299, 0.3966457352418651, 0.5794707838596214, 0.3362134250273048, -0.5823828544177054, -0.8645887652942967, 0.2566746362598513, -0.9267927556002933, -0.726880347017903, -0.10304594231723296, -0.45104125621680935, -0.13007691894630224, 0.2554647090108366, 0.34880334831973947, 0.01771971741747458, 0.4653456430235845, -0.8580867545920805, 0.6936950825346586, 0.13134788889804683, -0.678628212909727, 0.4711606752736286, -0.022472084608964327, 0.5270782923788415, -0.20434096366261123, 0.6112428633035374, -0.5395560570863518, 0.44754686765004004, 0.12080842626041566, 0.9792392030794899, 0.43580894293370287, -0.5191017383165484, -0.8367384548688865, 0.06414114676682492, 0.24986380246660622, -0.32835990369020873, 0.6389004983553941, 0.7993852708502758, 0.3291989300475018, 0.2899721917930491, 0.8250261305391273, -0.3698767543268402, 0.10519610152437053, -0.21348658891495154, -0.7925390704642441, -0.07157562702314668, -0.840958142136373, 0.8908518308177147, 0.4615104297108521, -0.6741351535249653, -0.9442165634932602, -0.7239597479109172, 0.23537696493339832, -0.8306196222921141, -0.0030796989330390545, -0.3780013006230625, 0.38240687731996514, -0.5825334696943147, 0.33103974478013654, -0.22691968097777626, 0.6698329138759089, 0.48696522849970036, -0.5325708008053245, 0.4014358033836558, 0.45865434410660577, -0.8318852788299638, 0.3309358074246944, 0.6743692738257205, -0.3147065797782074, -0.9483230405333292, -0.7816884415179295, -0.008030004879021613, -0.406573913188935, 0.5240562928990207, 0.1649973790388064, -0.5464137995732452, -0.6609207944008926, -0.06616542715061957, -0.8346734314677644, 0.03244689142747936, -0.30219019519850754, -0.6207985590868779, -0.5257516716854975, -0.014963097272092662, 0.21677913555355066, 0.6659002042603321, -0.6870554540503895, -0.23350553684100883, 0.33962045967928844, 0.4102396830311794, 0.11259981483174619, 0.28363424303601836, 0.329102118270604, 0.7197397521067703, 0.1779367240939289, -0.6826848604756239, -0.8655163649729487, 0.2455462550589398, -0.18280437655321924, -0.26959201377057473, -0.4417220733550242, -0.8761671839688909, -0.13479669774280212, 0.7638264063873306, 0.5510205863948803, -0.45310139040419717, 0.25029158062237156, -0.7630106551576017, -0.042242951943797236, 0.9169948221357653, -0.7493338687159912, 0.7593536791150608, -0.9351290473627571, 0.371270349384377, 0.3113240939760804, -0.5458416461827063, 0.7944524815426279, -0.7238841698016283, -0.1123329401853399, 0.005167421376470749, 0.4290751165278548, 0.6234869081704186, -0.3011686099825899, -0.93544983279179, -0.004890057498665001, 0.048545154101275134, 0.6138387809032042, -0.856690986670321, 0.7145398771575795, -0.9675040885749717, -0.07048109143844417, 0.9525325698135363, 0.9781112642962719, 0.21309813699784286, -0.3518604740871365, 0.6574614561120307, 0.6736470157656336, -0.3159855116830814, -0.9891385581737349, 0.05435201046451277, 0.20715622111563547, -0.4812705208989314, -0.34314210692592195, -0.8156998061308864, 0.40382558868796226, -0.8357346488330397, 0.232086958777181, -0.8069751853373912, 0.31991551487504855, 0.1705335554261307, 0.19130251595241998, -0.5933868309739618, -0.9852536124326599, -0.8555040800170177, -0.271795921522453, -0.0920592257839552, -0.6102795928900819, -0.01896867171990735, -0.5624098105705067, -0.393449315497858, -0.9590990938893862, -0.9573765404608034, 0.22973155628706143, 0.49976695256255543, -0.8600976172476125, -0.5303130741209614, 0.011677420299573305, 0.9099166818479316, -0.3599385549055458, 0.9978048554408498, -0.7717175729294956, -0.14198111135423552, 0.2412978138198123, 0.15092881094105914, 0.6210238064164466, 0.8057118399736731, -0.6498445482475181, -0.20192744615416847, -0.15600662419052846, -0.7829943993425332, -0.6569501259708699, -0.2194279145638729, 0.021813451728460498, 0.9434350534072511, -0.4965712092839034, 0.30358983616737323, -0.501874370170702, -0.00655862121197659, -0.17308134563465272, -0.8398146242070028, 0.08997169456677834, -0.860278081090804, -0.03317501910079934, -0.45007063499638367, -0.1241983011133021, 0.9828560303835256, 0.6349503649438186, 0.799998060548673, 0.9792317034946905, 0.5689913875569745, -0.5263253643159238, -0.7314987391975736, 0.36644977943793666, -0.882763057022897, 0.8027549601746484, -0.8016836847074915, -0.2777469356380422, -0.37015294271442745, 0.3125542263374792, 0.3536487129107977, 0.1327035724245087, -0.2761924037550336, 0.5752792858371036, 0.31024458705321933, 0.8658955544741678, -0.39139446677133716, 0.04010262945266563, -0.3698986164098219, -0.5631149795201627, -0.6424314116080556, 0.3591222951595525, -0.014326110895214406, -0.11695461551649755, -0.2945846893404904, -0.35250821175748603, -0.7664954777424002, -0.6331717357220994, 0.9707000806822663, 0.6856231443285541, -0.9936895135455188, 0.024903127100477374, 0.5374760943286732, -0.9012603994479156, 0.5294734041795914, 0.932818389263834, 0.4901203727523322, 0.9852200018448409, -0.2606586627836418, 0.13483952889983186, 0.1785834241376265, 0.8490917944717169, -0.3135937014245047, 0.034881389210060565, 0.590267771041733, 0.7013634441820384, 0.9710228369911866, 0.7762749700812221, -0.08730880456193613, 0.5483035799509266, -0.45148756571690973, 0.44968684887547217, 0.00020776454432680502, -0.09403926640793592, 0.1536679672277801, -0.34483289143691365, 0.4679141849180659, 0.028546892546056135, -0.12482783514096352, -0.4560512417280298, -0.7094134110941495, 0.9594669844312866, 0.33544902031531976, -0.41523182286294147, 0.5877268863994765, 0.023963483504160443, 0.15738240029323958, 0.6750435949166909, -0.7147308202303249, 0.9083001340546557, -0.2891348984278175, 0.42851206823291665, -0.689088813073113, -0.09866695640985701, 0.532438158778064, 0.4402799870759917, 0.43107360495764024, -0.0391672248389634, -0.08071175174361223, -0.632129858763103, -0.10620744948479022, -0.1331957918669231, 0.3847224795335187, -0.2582028600467976, 0.2984364702422555, -0.29530173891950584, 0.32504248742683006, 0.7916468930087821, -0.009336586580272144, 0.2891402246210466, 0.9818875682616541, 0.44380389549357324, -0.2692454455536408, -0.4389756467401178, -0.9865319326360895, -0.19601841756710492, 0.8427798252755554, 0.4389520547028154, 0.6794351470986528, -0.7124510711951757, 0.19740375839263957, 0.741418770626336, 0.4456879290179345, -0.41642186616224364, -0.644377121316047, 0.7724544063464549, -0.7577313880438619, 0.015351240189185322, 0.7102858257135407, 0.8808634101399155, 0.24046573008910066, 0.9411555526266957, 0.8979046518610383, -0.36189522835368537, 0.2188214615481061, -0.4850942350752996, 0.6761713217651011, 0.6375059538586796, 0.276202726692105, 0.16560672585539038, -0.5578775338718813, -0.5495803529769814, 0.7035822710598139, 0.5105807102912874, 0.34645538490358607, 0.37098755579411913, -0.3385825287154416, 0.7660646654124155, -0.4257505276355822, 0.8095417911945431, -0.5231613224096399, 0.7990519255243704, 0.8588141707664463, 0.03306010805082016, 0.5121829012203003, -0.8029167529953813, 0.8305138120954911, 0.5910862875472087, 0.10995290320025752, -0.9547727668974337, 0.3549673434019891, 0.44480455219928117, -0.27822252996351104, -0.814384450549339, 0.2097158279719329, 0.35153720894709384, 0.6292103375486047, 0.9686873773051741, 0.3697276313163569, 0.13111798091483995, -0.062169516292400306, -0.1005349614680835, -0.7888991706478181, -0.9956001132052228, -0.07609444518436237, -0.7825517433374007, -0.7749619917820436, 0.4032821495184622, -0.3668328781430332, 0.838550316537755, -0.5220561358389024, -0.9395888744090535, 0.835344352539555, -0.9231863228347577, 0.3411835843145965, -0.2526581920579074, -0.3196694285554491, -0.15105847537723727, -0.7847725493312181, -0.49632085695748396, -0.5391925951279937, 0.7235703114701815, -0.5109905936361303, -0.6196190413289142, 0.9567727212089527, -0.1338013419871502, -0.4921192260195082, -0.7363612693825998, 0.263070386102759, -0.46899540792040617, -0.8980881390679742, -0.14890958723801417, 0.839233733591499, -0.7797004704591841, -0.24903572546195019, 0.970793823976452, -0.6911599615279953, 0.6642554561728897, 0.22661296106699824, 0.9898654794483399, -0.1623903657596415, 0.6124952026616066, 0.08089317632956927, 0.6257369800737811, -0.6859372401726003, -0.6853151168520801, 0.09120284790973221, 0.8695582092489396, 0.908471700925576, -0.7978162637868134, -0.47393025341486283, -0.38138385908602723, -0.32498735613870133, -0.5610971514228746, 0.259574113867693, 0.11778190854216986, -0.384853250678576, 0.11635085509634546, -0.5218091276029473, 0.13001967724809904, -0.29973001897584806, 0.5257868402297339, -0.7403174966577704, -0.5631506740294709, -0.46043065666759797, 0.7745004702271396, 0.2579771979975418, -0.16716556120903459, 0.9168073985255194, 0.6617821506099129, 0.1959049621926836, -0.991893810635889, 0.36483909131635617, 0.764335486048316, 0.664094645795319, -0.4807779255207665, 0.809253037330532, -0.5785421796600445, 0.5779852537245711, 0.5841279729905797, 0.48908524703672485, 0.9034107608543473, -0.6944117869212281, 0.8279408803813622, -0.8621136812055077, -0.9959019570577841, -0.21409537312694193, -0.31520096110180496, -0.5588036288147289, -0.49248767494281886, -0.9432607679223888, -0.010375360788782872, -0.9358186288777832, 0.7006517589817316, 0.7883361535982587, -0.17257728231359182, -0.229849076457721, -0.41364551358959556, 0.7404733023942409, 0.9632518402678245, -0.8176554107236578, -0.6086143973083127, -0.9035727776512898, 0.8593819445876478, -0.7648464393657797, -0.5119897255772141, 0.9845917044563584, 0.19989432192555845, -0.28606324777907965, -0.6909224378141339, -0.6942678366578681, -0.13972652390970408, -0.8599823343283144, -0.8981117074778462, 0.9717910987491254, 0.1288184369694585, -0.763612273787541, -0.8852552483245275, 0.8496114577232012, -0.771562506278531, -0.5500892535293993, -0.23782553195313438, 0.43096907084378744, 0.8441529960982208, -0.6644254947458625, 0.558887267176825, 0.35735979386235184, 0.379879751333577, 0.7052726212231861, -0.38030723359168106, 0.828323915319773, -0.5219408770027443, 0.6466420149398779, -0.6374049957675074, 0.115679517886766, 0.5314081996468538, -0.7424215685119935, -0.6658099018334742, 0.517692073949306, -0.5594835511963763, 0.15492313545196312, -0.05791134498344852, -0.10192883357684268, -0.1921946887160073, -0.08347896854477077, -0.12684894260723478, 0.9574808120949316, -0.662009762759731, -0.20827050000283354, -0.6383803699345281, 0.31466111404876784, -0.17644922324948276, 0.1753120243858186, -0.3933887730465746, -0.38660672077687597, -0.5769413328383857, -0.8480059366021733, 0.219499314942317, -0.006647614337486507, -0.11276182342901375, 0.5992656664362948, 0.38176878260587954, 0.584470176141181, -0.9745766713092123, 0.5348770795048454, -0.6004042633827968, -0.5672758482865345, 0.7733951646262529, 0.573475091951275, 0.4512130244421617, 0.5571541694463533, 0.7851651572590042, 0.33441630899605435, -0.7846869326595487, 0.21251635976061078, 0.5356208258925008, 0.5869522940839071, -0.841962772328043, 0.8793991075763401, -0.9041309062798886, 0.49662887209764417, 0.15554295031073684, 0.09925963582183983, 0.0846918513794146, -0.12128309971922757, -0.5505191509366074, 0.49357380458638733, 0.22851317773227353, -0.02262657749445074, 0.13735630202703852, 0.7041284307200488, -0.5544560651133981, 0.6282408356257059, 0.09051509081321885, -0.3544907493792844, -0.6228962510599692, -0.1385811634289107, -0.47068109613922604, 0.8522903487717559, -0.6338241155063677, 0.9563759146240147, -0.8945620473033586, 0.16261695195048942, 0.25861558392229855, 0.0800804073014334, 0.8531836725400339, -0.820357697935354, -0.013424005482439272, 0.02633888693858677, 0.6898861436175481, -0.31324021525578893, 0.9595551755919383, 0.7526718251261686, 0.08413127256078079, 0.8416564935546149, -0.9926355087405672, -0.5922178750382958, 0.21879863816305378, -0.1664163065709774, -0.30344052161973, -0.7298327743807487, 0.585621347707471, 0.831669911083958, -0.7005108360855978, -0.7698384775597036, -0.6753629126475715, 0.8908012527936138, -0.19194971831639807, 0.5111912412833719, -0.5039624045179609, -0.34495689724296463, 0.31608393258739675, 0.35918053327549404, 0.11264216317161924, 0.10068890927154794, 0.49837464505642215, -0.26761688541080497, 0.3263017752366637, -0.7830831701520184, 0.971513164453695, -0.7249662402778982, -0.11036811955437997, 0.2709500891192389, 0.9079738401743636, -0.03468684961457824, 0.12326205353605424, -0.46432721614001204, -0.6508845869711963, -0.9557580635344767, -0.9129504841176874, 0.3726849237515384, 0.9580577650095405, -0.0681944508686847, -0.09301449667032569, -0.6203050927869964, 0.6697017382649633, -0.11268073458198247, 0.2383395645410098, -0.06827711040144635, -0.7645762014597997, 0.05707415642830682, -0.9193953587309427, -0.26422828913226004, 0.6556682268958443, -0.8397985014174874, 0.26111111309761426, 0.1732681203723656, 0.9689492289515937, -0.07559578627502717, 0.40130342417215603, -0.7048576205070751, 0.9481532394267433, 0.7800918292559582, -0.8011117320222927, -0.9739191222385071, -0.9511069493967199, -0.9519585265336226, -0.8107291053329007, 0.2842398096652963, 0.8234018783620816, 0.11737931582451577, -0.6262974357250575, 0.7427320886882416, 0.5229540083944637, -0.4978583429205339, -0.9937562045069921, -0.11741818775996737, -0.5975524555203775, 0.08887679067019882, -0.9030847745999713, 0.5474706327598953, 0.4545028756106373, -0.9837500310331271, 0.1884065856494599, 0.6412388067644434, 0.5994146754578049, -0.35438922743122747, -0.7446566482808692, -0.3825578188074332, 0.6167718677638852, -0.1472161201590776, 0.47471699346404606, -0.8301344534200092, 0.9113741616692763, 0.1731498324086036, 0.853997009960181, 0.8516137650054709, 0.7571396188399799, 0.3209767415107714, 0.4919725483266868, -0.5275789999452702, 0.5571806621583533, 0.6183852821053772, 0.9074952639661615, 0.6502634160285972, -0.5878531936148574, -0.4117990165659642, -0.06288461726194128, 0.2896486913750389, 0.02230088307473399, 0.2422096496781374, -0.3052347051460751, 0.4520314027021821, 0.8631300314227375, -0.2728083435717996, -0.2599133561143496, 0.9971280364977915, -0.8322040393908439, -0.34470353504858875, -0.3718458544551537, 0.7329277880507996, -0.1617170119748026, 0.5087352439117716, -0.3201389112356543, -0.06246174344970212, 0.9356114164144138, 0.1267867709707633, 0.4351962040962436, 0.3207403154772004, -0.9852623757599797, 0.10813152749996058, -0.9583914265473226, -0.22110158371740818, 0.17953591832671556, -0.3301815856048802, 0.07100115693028641, 0.9211452638296618, -0.38130318518413686, -0.7806819781421561, 0.18884027234919443, -0.512863265028876, 0.4174898717998248, -0.6399184204749842, -0.607589237703521, 0.8964571551660416, 0.9262082178130737, -0.3469692064139658, -0.45719338139182075, -0.8308246103928383, -0.8473950768245702, 0.0010099304387722885, 0.7372768850615783, 0.1315034745162833, -0.46445716133058346, -0.6275392025135951, -0.8593506291088358, 0.5701737454800988, -0.04773956277933222, -0.02962513198337957, -0.0070390875362433825, -0.9105504969729725, 0.5697220897712705, -0.41233633556369753, 0.6542725656321049, 0.13640907122158952, -0.8083169206018026, -0.7598195817359996, -0.844666151931424, -0.5181367258825467, -0.16853144012003796, -0.8231880632938646, 0.2751963422031152, 0.2615112075326256, 0.42497072687369797, -0.4904780974676832, 0.16715997296302065, -0.22776139299890508, 0.6646670934269878, -0.1990378345429369, -0.30681680140212775, -0.5657765967247226, -0.4101781567025873, -0.3431031322628655, 0.023217715487541035, -0.61599149528206, -0.5882790868429404, -0.42571509925685236, 0.565547131144778, 0.5683891009599762, 0.49716397632875164, 0.39038386854056206, 0.7758048853228996, 0.7783872356673249, 0.5095177809336926, -0.4359339487897047, 0.5559008748382668, -0.2389649905442326, -0.2147131318829032, -0.37383390260130267, -0.2768091923640268, -0.15365533851631885, -0.07357186055221576, -0.46813474846316194, 0.7462371263322314, -0.049887462786263104, 0.30195148611985556, 0.9687860810088216, 0.38434358041026395, 0.9288268224230536, 0.17604889211128416, 0.9381684953734064, 0.39900128010159963, -0.07427138436654834, 0.12443903488772134, 0.944829892195816, 0.17221518863956775, -0.33577669359053464, -0.47794809926602544, -0.08857468060733997, -0.8747080761602122, -0.7147395110158594, -0.05470599799636533, -0.3426023495086925, 0.05428107047489328, 0.8701316959377947, -0.5302411311302404, -0.8866642300755936, -0.16877882544429457, -0.7254079585610209, -0.16423608024487568, -0.21402585381096872, 0.0959463519062953, -0.8877152940648485, 0.36187862953194894, -0.23137000152249065, 0.11275862394114045, -0.573940771981106, -0.21358312499510546, 0.6711970620156167, 0.8225725742938159, -0.26182377586550154, 0.6658084315944652, -0.07993824967908969, -0.019849480111902684, 0.8428429689854491, 0.6530783636723378, 0.03653568503007887, -0.4829347373487547, -0.042188785408239005, 0.8044351440380404, 0.529372067381148, 0.42625795430897506, 0.20495406322176746, 0.8266659133114145, 0.3035514001565289, -0.05655263823085033, -0.043729876784023514, -0.9877385558899914, 0.6285112893582587, 0.6523112412120147, -0.6617179677033189, -0.7885438285999922, -0.543323613461711, 0.2158571157253093, -0.5907348783781818, 0.8528444823948387, -0.6732524392591637, 0.6920715261551282, 0.4862543694296684, 0.07836901753652503, -0.8610658941901195, 0.926804351342646, -0.2291941060416205, 0.6960320726335321, -0.32240060871167886, -0.639011921929904, -0.18317439835603633, 0.42540963941256593, -0.028078309859698836, 0.4204481530990125, 0.8447000745063007, -0.5962398470893853, -0.5140690256526066, -0.34544003152118985, 0.4062950169516122, 0.8089814981084282, -0.2754379467094712, -0.0050601576307283125, -0.3658253393165094, 0.9525853501407078, -0.7217894825340072, -0.7844413428669978, -0.2777978215755914, -0.13804422630420965, -0.48527349483055593, -0.6951722639521529, -0.04587372961008329, 0.4853302112265334, -0.13038576470285634, 0.7198513649991369, -0.5778831046987678, -0.07866710303086744, -0.9389858221976297, -0.23058100328136422, -0.5417978015656764, 0.3882681882238326, -0.026219999394296867, 0.33908520279504284, -0.7189995466906587, 0.8224112719958934, 0.393365838203416, -0.06897322548381601, -0.852157371105037, 0.9154055137139485, -0.8265755405792246, 0.5705134971894044, 0.003809524740074677, -0.33017402211084, -0.6255105469787166, -0.07691085747165105, -0.7570935777736663, -0.4584490474802283, -0.9207628884015553, 0.16257040101940334, 0.5320373017688267, 0.2766680726959505, -0.03719506132798567, -0.0751458527274591, -0.39334587574604774, -0.34302489764059185, 0.7853900893192043, -0.49668755475117643, -0.31280692635299, -0.17637600962327937, 0.7943906480655814, 0.7343167227180747, 0.1160261075062674, 0.7263668326425572, -0.9755548963167715, -0.057371737232192466, 0.0569500899337545, 0.23886283065348723, 0.6627054368164653, -0.22907381940217442, 0.796185935768652, 0.8959981871797589, -0.41935415835986944, -0.9739105016760452, -0.6510653630617513, 0.8571018790673293, 0.7679865633121346, -0.10614712837991824, 0.5018821412694379, 0.5699468476625948, 0.2981308966916032, 0.3274865028528138, 0.27167089729000193, -0.7894312853141141, 0.21134816525133115, 0.330626071927802, -0.5384413104202068, -0.6906211672667526, 0.48058660909443685, 0.05413441842704714, 0.060766148397198405, -0.5783543532995084, -0.2967483646875835, 0.5290704840032308, 0.46436044610131355, -0.4860745768410628, 0.9134048617239527, -0.2508558889720629, -0.6616498680994587, 0.563531078475888, 0.5633910935094446, 0.905044697494481, 0.0024226770537132403, 0.3119052784028955, -0.9676919461511824, 0.44828699401421135, -0.15731950408971018, -0.8771443517697428, 0.6741108952410448, 0.6655862192287754, -0.76191742241198, -0.13281816375780986, -0.7931526052431812, 0.6117561904096844]

model = genotype_to_model(genotype)

model.save("best_model.keras")